In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import math
from scipy import ndimage
import matplotlib.patches as patches
import cv2
from skimage.transform import radon, rescale
import abel
import os
import glob
from scipy import constants as sc
from scipy.optimize import curve_fit, least_squares
from scipy.signal import savgol_filter, correlate

# A bunch of ease-of-life functions here:

In [ ]:
def save_data_to_txt(x_data, y_data, file_path, header=('X', 'Y')):
    '''Saves x and y data to a text file with a specified header.'''
    data = np.column_stack((x_data, y_data)) # Stack x and y data as columns
    np.savetxt(file_path, data, header='\t'.join(header), comments='') # Save to text file

def crop_and_rotate_image(image, xmin, xmax, ymin, ymax, angle):
    '''Crops a specified region from the image and rotates it by a given angle.'''
    cropped_image = image[ymin:ymax, xmin:xmax] # Crop the image
    rotated_image = ndimage.rotate(cropped_image, angle, reshape=False) # Rotate the cropped image# Rotate the cropped image
    return rotated_image

def splice_middle_elements(lst, N):
    '''Splices the middle N elements from a list.
    That means, if the list has an even number of elements, it takes the N elements centered around the middle two elements.
    If the list has an odd number of elements, it takes the N elements centered around the middle element.'''
    is_odd = len(lst) % 2 != 0 # Check if the list has an even or odd number of elements
    start_index = len(lst) // 2 - N // 2 # Calculate the starting index for splicing
    if is_odd: # Adjust the start index for odd-sized lists
        start_index += 1 
    spliced_elements = lst[start_index:start_index + N] # Splice the middle N elements
    return spliced_elements

def change_ticks(data=object,pixsize=float,axis=object,change_X=bool,change_Y=bool):
    '''
    Changes the ticks of a matplotlib axis to represent physical distances in microns, based on pixel size (my custom version - there are probably better ways to do this).
    Here I specifically wanted to have ticks at integer values in microns, and the ability to turn on/off x and y axis changes separately.
    inputs: data - 2D array representing the image data
            pixsize - pixel size in microns/pixel
            axis - matplotlib axis object to modify
            change_X - boolean flag to change x-axis ticks
            change_Y - boolean flag to change y-axis ticks
    '''
    (nrows,ncols)=data.shape #get the number of rows and columns in the data array
    N_x=math.floor(ncols*pixsize/100) #max integer number of hundreds of microns that fit into the horizontal axis
    N_y=math.floor(nrows*pixsize/100) #max integer number of hundreds of microns that fit into the vertical axis
    new_xticks=np.around(np.linspace(0,N_x*100,N_x+1),decimals=0) #generate a list of integer ticks for the horizontal axis
    new_xticks = [int(x) for x in new_xticks] #convert to integers
    new_yticks=np.around(np.linspace(0,N_y*100,N_y+1),decimals=0) #generate a list of integer ticks for the vertical axis
    new_yticks = [int(x) for x in new_yticks] #convert to integers
    #find horizontal locations (in pixels) corresponding to the desired tick locations (given in mm), rounded to the nearest pixel:
    new_xtick_locations = np.around(np.array(new_xticks)/pixsize,decimals=0).astype(int) #also, cast into integers
    #find vertical locations (in pixels) corresponding to the desired tick locations (given in mm), rounded to the nearest pixel:
    new_ytick_locations = np.around(np.array(new_yticks)/pixsize,decimals=0).astype(int) #also, cast into integers
    if change_X: #if changing the x-axis ticks
        axis.set_xticks(new_xtick_locations,new_xticks) #set the horizontal axis ticks
        axis.set_xlabel('X [microns]')
    if change_Y: #if changing the y-axis ticks
        axis.set_yticks(new_ytick_locations,new_yticks) #set the vertical axis ticks
        axis.set_ylabel('Y [microns]')
    return

def rotate_with_fill(image, angle, fill_value):
    # Rotate the image with reshaping
    rotated_image = ndimage.rotate(image, angle, reshape=True)

    # Find indices of zero values in the rotated image
    zero_indices = np.where(rotated_image == 0)

    # Replace zero values with the fill value
    rotated_image[zero_indices] = fill_value

    return rotated_image

def get_delta_eta_H2(lambda_um):
    '''calculates the wavelength-dependent refractive index of molecular hydrogen at STP based on R.Peck: https://opg.optica.org/josa/fulltext.cfm?uri=josa-67-11-1550&id=56823
    Also referenced here: https://refractiveindex.info/?shelf=main&book=H2&page=Peck '''
    A = 14895.6 #constants from Peck paper
    B = 180.7
    C = 4903.7
    D = 92.0
    delta_eta_H2 = (A/(B-lambda_um**(-2))+C/(D-lambda_um**(-2)))*1e-6 # The difference of refractive index from 1 at the specific wavelength (true refractive index - 1) 
    return delta_eta_H2

# Abel transform function used later in the plotting script:

In [ ]:
def pyAbelTransform(PhaseMap, plumeOrientation):
    '''Performs an inverse Abel transform on a (pre-processed, rotated) 2D phase map using the pyAbel library.'''
    # pyAbel requires that nrows is odd:
    if PhaseMap.shape[0] % 2 == 0: #if even number of rows
        PhaseMap = np.append(PhaseMap, np.zeros((1, PhaseMap.shape[1])), axis=0); # Add one more row of zeros at the edge.
    
    # pyAbel assumes symmetry around vertical axis; if the channel/gas plume/whatever else is oriented horizontally, the phase image needs to be transposed first
    if 'horizontal' in plumeOrientation:
        PhaseMap = np.transpose(PhaseMap)
    # now the abel transform can be performed; using pyAbel library: https://pyabel.readthedocs.io/en/latest/readme_link.html   
    AbelInvertedPhase = abel.Transform(PhaseMap, # must be a 2D array
                                    direction='inverse', #transform performed from 2D projection to cylindrical coords --> this is the inverse Abel transform what we want
                                    method='hansenlaw', #abel transform method; I tried a bunch, they should all give the same result if done corretcly!
                                    origin='image_center', #specifies where the center of the image is located
                                    symmetrize_method='average', #method to use for symmetrization; average is safest
                                    symmetry_axis=0, #axis along which the image is symmetric
                                    ).transform;
    # if the channel/gas plume/whatever else was horizontal, transpose the result back to original orientation
    if 'horizontal' in plumeOrientation:
        AbelInvertedPhase = np.transpose(AbelInvertedPhase);
    return AbelInvertedPhase; #return the abel-inverted phase map

In [ ]:
def two_color(delta_eta1_arr, delta_eta2_arr, A, B, C, D):
    """
    Solve the 2D linear system of equations (two-color interferometry):
    M x = b
    where:
        M = [[A, B],
             [C, D]]
        b = [delta_eta1,
             delta_eta2]
        x = [n_e,
             delta_n]
    All values are assumed to vary over radius, except A–D which are constant.
    Parameters:
        delta_eta1_arr, delta_eta2_arr: arrays of Δη₁ and Δη₂ over radius
        A, B, C, D: constants in the system matrix
    Returns:
        x_arr: Nx2 array of solutions [n_e, delta_n] at each radius
    """
    N = len(delta_eta1_arr)
    x_arr = np.zeros((N, 2))

    M = np.array([[A, B],
                  [C, D]])

    for i in range(N):
        delta_eta1 = delta_eta1_arr[i]
        delta_eta2 = delta_eta2_arr[i]
        b = np.array([delta_eta1, delta_eta2])
        x = np.linalg.solve(M, b)
        x_arr[i] = x

    return x_arr

In [ ]:
def gauss(x, H, A, x0, sigma):
    ''' Defines a Gaussian function:
     inputs: x (independent variable), H (baseline), A (amplitude), x0 (center), sigma (stddev)
     outputs: Gaussian function value at x
    '''
    return H + A * np.exp(-(x - x0) ** 2 / (2 * sigma ** 2))

def moment_guesses(x, y):
    """
    Robust, sign-aware moment guesses for a Gaussian fit (my own gaussian fit was failing to converge too often)
      - subtract robust baseline (median)
      - flip if the feature is a dip (instead of a peak)
      - compute center/width from positive residuals
    inputs: x, y data arrays (numpy arrays)
    outputs: H, A, x0, sigma (initial guesses for Gaussian fit)
    """
    H = np.median(y) # baseline level (median)
    y0 = y - H # residuals
    flip = -1 if np.min(y0) < 0 else 1 # determine if we need to flip the feature
    y1 = flip * y0 # flipped residuals
    y1[y1 < 0] = 0  # ignore negative residuals
    # fallback if no positive mass
    area = np.trapezoid(y1, x) # compute area under the positive residuals
    if not np.isfinite(area) or area <= 0: # fallback
        x0 = x[np.argmin(y)] if flip == -1 else x[np.argmax(y)] # center at min/max
        sigma = max((x[-1] - x[0]) / 10.0, (x[1] - x[0])) # arbitrary width guess
        A = (np.min(y) - H) if flip == -1 else (np.max(y) - H) # amplitude guess
        return H, A, x0, abs(sigma) # return guesses

    x0 = np.trapezoid(x * y1, x) / area # compute centroid
    var = np.trapezoid(((x - x0) ** 2) * y1, x) / area # compute variance
    sigma = np.sqrt(max(var, (x[1] - x[0])**2)) # compute stddev, ensure non-zero
    A = flip * np.max(y1)  # restore expected sign
    return H, A, x0, sigma

def fit_gaussian_robust(x, y):
    """
    Fits a Gaussian profile to the data (xdata and ydata)
    Bounded curve_fit with robust least-squares fallback.
    Returns (params, covariance or None).
    """
    # light smoothing of ydata for stability of guesses of the initial parameters
    y_sm = savgol_filter(y, 11 if len(y) >= 11 else max(5, len(y)//2*2+1), 2, mode='interp')
    H0, A0, x00, s0 = moment_guesses(x, y_sm) # initial parameter guesses using the smoothed data
    # bounds: sigma > dx, x0 inside x-range, H padded by data range, A unbounded
    dx = max(x[1] - x[0], 1e-9) # min dx to avoid zero-width issues
    lower = [np.min(y) - abs(np.ptp(y)), -np.inf, x.min(),  dx] # lower bounds for curve fit
    upper = [np.max(y) + abs(np.ptp(y)),  np.inf, x.max(),  (x[-1] - x[0])] # upper bounds for curve fit
    p0 = [H0, A0, x00, max(s0, dx)] # initial parameter vector
    try:
        params, cov = curve_fit(gauss, x, y, p0=p0, bounds=(lower, upper), maxfev=10000) # fit using curve_fit
        return params, cov # return fitted parameters and covariance
    #exception here: curve_fit failed (e.g. due to poor initial guesses or ill-conditioned problem), so use least squares
    # this works by minimizing the residuals between the data and the Gaussian model
    except Exception: # robust fallback
        def resid(p): # residual function for least squares
            H, A, x0, s = p # unpack parameters
            return gauss(x, H, A, x0, s) - y # compute residuals
        res = least_squares( #compute least-squares solution from initial guess and bounds
            resid, p0, bounds=(lower, upper), # bounds for the optimization
            loss='soft_l1', f_scale=np.std(y) or 1.0, #parameters for robust loss function: soft L1 loss to reduce outlier influence
            max_nfev=4000 # maximum number of function evaluations
        )
        return res.x, None # return optimized parameters and None for covariance

def center_shift_from_profile_parabola(profile):
    """Subpixel center via quadratic interpolation around the min."""
    k = int(np.argmin(profile))
    i0, i1, i2 = max(0, k-1), k, min(len(profile)-1, k+1)
    x = np.array([i0, i1, i2], dtype=float)
    y = profile[[i0, i1, i2]]
    # Fit a parabola y = ax^2 + bx + c
    A = np.vstack([x**2, x, np.ones_like(x)]).T
    try:
        a, b, c = np.linalg.lstsq(A, y, rcond=None)[0]
        if abs(a) > 1e-12:
            x_vertex = -b / (2*a)
        else:
            x_vertex = float(k)
    except Exception:
        x_vertex = float(k)
    return x_vertex  # pixel coordinate (row index)

def center_shift_from_profile_symmetry(profile):
    """
    Estimate center by aligning the profile to its mirror via cross-correlation.
    The lag is split by 2 because mirroring doubles the displacement.
    """
    p = profile - np.median(profile)
    pm = p[::-1]
    cc = correlate(p, pm, mode='full')
    lag = np.argmax(cc) - (len(p) - 1)
    return len(p) / 2.0 + lag / 2.0  # estimated center index (float)

def estimate_center_index(profile):
    """Combine parabola and symmetry; return a robust subpixel center index."""
    p_sm = savgol_filter(profile, 11 if len(profile)>=11 else max(5, len(profile)//2*2+1), 2, mode='interp')
    c1 = center_shift_from_profile_parabola(p_sm)
    c2 = center_shift_from_profile_symmetry(p_sm)
    # median of the two, clipped into valid range
    c = np.median([c1, c2])
    return float(np.clip(c, 0, len(profile)-1))

def compute_snr(amplitude_abs, bg_std):
    '''computes signal-to-noise ratio given absolute amplitude and background stddev
    inputs: absolute amplitude (float), background stddev (float)
    outputs: SNR (float)'''
    bg = float(bg_std) if np.isfinite(bg_std) and bg_std > 0 else 1e-12 # avoid division by zero
    return float(amplitude_abs) / bg # return SNR

# --------------------------- your main function ---------------------------

def plot_2d_data(n0_mbar: float,           # initial ambient pressure in mbar
                 RED_filepath: str,         # folder with 1030 nm phase maps
                 GREEN_filepath: str,       # folder with 515 nm phase maps
                 plot_name: str,            # name of the saved plot/txt
                 plot_title: str,           # title at the top of the figure
                 dev_angle_override_R: float = None,  # manual rotation override [deg]
                 dev_angle_override_G: float = None,  # manual rotation override [deg]
                 flip_sign_RED: bool = False,         # electrons should give negative phase --> flip manually if needed (one channel copy is positive, the otehr negative)
                 flip_sign_GREEN: bool = False,       # electrons should give negative phase --> flip manually if needed (one channel copy is positive, the otehr negative)
                 repair_abel: bool = False, ## manual rotation override [deg]
                 xlims_um_R: list = None, # list of two xlims for the region containing a piece of the plasma channel in microns (red)
                 ylims_um_R: list = None, # list of two ylims for the region containing a piece of the plasma channel in microns (red)
                 xlims_um_G: list = None, # list of two xlims for the region containing a piece of the plasma channel in microns (green)
                 ylims_um_G: list = None, # list of two ylims for the region containing a piece of the plasma channel in microns (green)
                 xlims_bg_R: list = None, # list of two xlims for the background region in microns (red)
                 ylims_bg_R: list = None, # list of two ylims for the background region in microns (red)
                 xlims_bg_G: list = None, # list of two xlims for the background region in microns (green)
                 ylims_bg_G: list = None): # list of two ylims for the background region in microns (green)

    ######### CONSTANTS for this function:
    pixsize = 1.06  # microns per pixel
    lambda_red=1030e-9 # wavelength (IR --> 'red' nickname) in meters
    lambda_green=515e-9 # wavelength (frequency-doubled --> 'green') in meters
    N = 300 # number of radial points for Abel inversion
    mrad_lim = 250 # mrad limit for horizontal projection plots
    rlim_um = 120 # microns limit for radial plots
    ##########

    ######### convert ROI limits from microns to pixels 
    xmin_R=int(xlims_um_R[0]/pixsize); xmax_R=int(xlims_um_R[1]/pixsize) #ROI
    ymin_R=int(ylims_um_R[0]/pixsize); ymax_R=int(ylims_um_R[1]/pixsize)
    xmin_G=int(xlims_um_G[0]/pixsize); xmax_G=int(xlims_um_G[1]/pixsize)
    ymin_G=int(ylims_um_G[0]/pixsize); ymax_G=int(ylims_um_G[1]/pixsize)

    xmin_bg_R=int(xlims_bg_R[0]/pixsize); xmax_bg_R=int(xlims_bg_R[1]/pixsize) #background
    ymin_bg_R=int(ylims_bg_R[0]/pixsize); ymax_bg_R=int(ylims_bg_R[1]/pixsize)
    xmin_bg_G=int(xlims_bg_G[0]/pixsize); xmax_bg_G=int(xlims_bg_G[1]/pixsize)
    ymin_bg_G=int(ylims_bg_G[0]/pixsize); ymax_bg_G=int(ylims_bg_G[1]/pixsize)

    # lengths of ROI and BG regions in pixels; useful later
    xlen_R=xmax_R-xmin_R; ylen_R=ymax_R-ymin_R
    xlen_G=xmax_G-xmin_G; ylen_G=ymax_G-ymin_G
    xlen_bg_R=xmax_bg_R-xmin_bg_R; ylen_bg_R=ymax_bg_R-ymin_bg_R
    xlen_bg_G=xmax_bg_G-xmin_bg_G; ylen_bg_G=ymax_bg_G-ymin_bg_G

    # load data (2D phase maps, stored as text files; a simple array was saved via np.savetxt())
    data_R = np.loadtxt(os.path.join(RED_filepath, 'AvgPhase.txt')) # load RED data
    data_G = np.loadtxt(os.path.join(GREEN_filepath, 'AvgPhase.txt')) # load GREEN data
    if flip_sign_RED:   data_R = -data_R # flip RED sign if needed, according to the flag given manually
    if flip_sign_GREEN: data_G = -data_G # flip GREEN sign if needed according to the flag given manually

    # subtract mean background (find the mean in the region designated manually as background, and subtract it from the whole image)
    data_R = data_R - np.mean(data_R[ymin_bg_R:ymax_bg_R, xmin_bg_R:xmax_bg_R])
    data_G = data_G - np.mean(data_G[ymin_bg_G:ymax_bg_G, xmin_bg_G:xmax_bg_G])

    # compute background region stddev for signal-to-noise ratio estimates later
    bg_ROI_R = data_R[ymin_bg_R:ymax_bg_R, xmin_bg_R:xmax_bg_R] #the whole BG roi as a 2D array
    bg_ROI_G = data_G[ymin_bg_G:ymax_bg_G, xmin_bg_G:xmax_bg_G]
    bg_std_R = np.std(bg_ROI_R)
    bg_std_G = np.std(bg_ROI_G)

    cropped_image_R = data_R[ymin_R:ymax_R, xmin_R:xmax_R] # crop to ROI (red)
    cropped_image_G = data_G[ymin_G:ymax_G, xmin_G:xmax_G] # crop to ROI (green)

    # generate meshgrid for plotting the ROI phase maps later
    rows_R, cols_R = data_R.shape
    rows_G, cols_G = data_G.shape
    XR, YR = np.meshgrid(np.arange(cols_R),np.arange(rows_R))
    XG, YG = np.meshgrid(np.arange(cols_G),np.arange(rows_G))

    # rotate ROI according to the given angle (manual override)
    rotated_ROI_R = ndimage.rotate(cropped_image_R, dev_angle_override_R, reshape=True, order=1, mode='nearest')
    rotated_ROI_G = ndimage.rotate(cropped_image_G, dev_angle_override_G, reshape=True, order=1, mode='nearest')

    #1D projections for centering (only centering for now --> median is fine)
    # median is more robust than mean against outliers/edges
    ydata_R = np.median(rotated_ROI_R, axis=1) # 1D projection of the plasma channel phase map along axis=1 (columns; radial profile)
    ydata_G = np.median(rotated_ROI_G, axis=1) # 1D projection of the plasma channel phase map along axis=1 (columns; radial profile)

    # coordinated axes (transverse to the channel) in microns, centered ~0
    xdata_R = np.linspace(-len(ydata_R)/2, len(ydata_R)/2, len(ydata_R)) * pixsize
    xdata_G = np.linspace(-len(ydata_G)/2, len(ydata_G)/2, len(ydata_G)) * pixsize

    # Gaussian fit. Using a modified function defined above instead of basic curve_fit to avoid convergence issues (sometimes it wouldn't find the right fit)
    params_R, _ = fit_gaussian_robust(xdata_R, ydata_R) # fit RED profile using the gaussian function defined above ('robust version' to avoid annoying failures when the fit struggles to converge)
    params_G, _ = fit_gaussian_robust(xdata_G, ydata_G)
    H_R, A_R, x0_R, sigma_R = params_R # unpack RED fit parameters
    H_G, A_G, x0_G, sigma_G = params_G # unpack GREEN fit parameters

    # OK now I have fit parameters for the Gaussian fit on both colors. Now I want to roll/shift the 2D ROIs so that the center is at zero.
    # compute signal-to-noise ratios for both colors so that we can decide whether to use the fit center or the data-based center
    snr_R = compute_snr(abs(A_R), bg_std_R)
    snr_G = compute_snr(abs(A_G), bg_std_G)

    #shift in pixels along axis=0 (transverse to the channel); x0 is in microns
    shift_pix_R = x0_R / pixsize if snr_R >= 3 else (estimate_center_index(ydata_R) - len(ydata_R)/2.0) # use fit center if SNR>=3, else use data-based center
    shift_pix_G = x0_G / pixsize if snr_G >= 3 else (estimate_center_index(ydata_G) - len(ydata_G)/2.0)

    # actually shift to centre based on the computed shift
    rolled_ROI_R = ndimage.shift(rotated_ROI_R, shift=(-shift_pix_R, 0), order=1, mode='nearest', prefilter=False)
    rolled_ROI_G = ndimage.shift(rotated_ROI_G, shift=(-shift_pix_G, 0), order=1, mode='nearest', prefilter=False)

    # subtract fitted baseline (H) from the whole ROI; this is a better estimate of the true zero level
    rolled_ROI_R = rolled_ROI_R - H_R
    rolled_ROI_G = rolled_ROI_G - H_G

    # now that the ROIs are centered, compute new 1D projections and re-fit on centered profiles (for plots/metrics)
    phase_projection_R = np.median(rolled_ROI_R, axis=1) #using median again for robustness
    phase_projection_G = np.median(rolled_ROI_G, axis=1)
    params_R2, _ = fit_gaussian_robust(xdata_R, phase_projection_R) # re-fit RED centered profile
    params_G2, _ = fit_gaussian_robust(xdata_G, phase_projection_G) # re-fit GREEN centered profile
    H_R2, A_R2, x0_R2, sigma_R2 = params_R2 # unpack RED fit parameters
    H_G2, A_G2, x0_G2, sigma_G2 = params_G2 # unpack GREEN fit parameters
    fit_y_R_2 = gauss(xdata_R, *params_R2) # fitted RED profile
    fit_y_G_2 = gauss(xdata_G, *params_G2) # fitted GREEN profile

    # error-to-peak metrics
    err_to_peak_R = abs(A_R2) - bg_std_R
    err_to_peak_G = abs(A_G2) - bg_std_G

    #####################
    # PLOTTING ##########
    fig, ax = plt.subplots(3,4,figsize=(20,14),gridspec_kw={'width_ratios': [4,2,2,4], 'height_ratios':[2,1,1]}) #create a grid of subplots
    plt.subplots_adjust(wspace=0, hspace=0.3) #adjust spacing between subplots (I need to pack them tightly because there is a lo of data to show)

    ax[0,0].pcolormesh(XR, YR, data_R, cmap='RdBu', vmin=0.8*np.amin(data_R), vmax=-0.8*np.amin(data_R)) #plot full 2D phase maps (original data)
    ax[0,3].pcolormesh(XG, YG, data_G, cmap='RdBu', vmin=0.8*np.amin(data_G), vmax=-0.8*np.amin(data_G))
    change_ticks(data=data_R,pixsize=pixsize,axis=ax[0,0],change_X=True,change_Y=True) #custom function to change ticks to microns
    change_ticks(data=data_G,pixsize=pixsize,axis=ax[0,3],change_X=True,change_Y=True)
    ax[0,0].set_title('2D phase shift map: 1030 nm (RED)')
    ax[0,3].set_title('2D phase shift map: 515 nm (GREEN)')
    ax[0,0].axis('equal'); ax[0,3].axis('equal') #set equal aspect ratio - important so that the images are not distorted

    phaseLine_R=np.mean(data_R,axis=1) # 1D horizontal projections (mean over rows)
    phaseLine_G=np.mean(data_G,axis=1) 
    ax[0,1].plot(phaseLine_R*1000, np.arange(len(phaseLine_R)), color='black') #plot 1D horizontal projections (mean); useful to visualize the noisiness of the background
    ax[0,2].plot(phaseLine_G*1000, np.arange(len(phaseLine_G)), color='black') #also useful to see if we maybe didn't keep enough spatial frequencies in the FFTs, leading to oscillatory artifacts
    ax[0,1].set_xlabel('mrad'); ax[0,2].set_xlabel('mrad')
    ax[0,1].set_title('horizontal projection (mean)')
    ax[0,2].set_title('horizontal projection (mean)')
    for j in (1,2):
        ax[0,j].set_yticks([]); ax[0,j].set_yticklabels([]) #remove y-axis ticks/labels for the horizontal projections
        ax[0,j].set_xlim(-mrad_lim,mrad_lim) #limit x-axis to focus on the relevant range
    ax[0,3].yaxis.tick_right(); ax[0,3].yaxis.set_label_position("right") #move y-axis to the right for symmetry

    # ROI rectangles
    rectangle_R = patches.Rectangle((xmin_R, ymin_R), xlen_R, ylen_R, linewidth=2, edgecolor='magenta', facecolor='none') #shwow the manually pciked ROI rectangles on the full phase maps
    rectangle_G = patches.Rectangle((xmin_G, ymin_G), xlen_G, ylen_G, linewidth=2, edgecolor='magenta', facecolor='none')
    rectangle_bg_R = patches.Rectangle((xmin_bg_R, ymin_bg_R), xlen_bg_R, ylen_bg_R, linewidth=2, edgecolor='black', facecolor='none') #show the manually picked background rectangles on the full phase maps
    rectangle_bg_G = patches.Rectangle((xmin_bg_G, ymin_bg_G), xlen_bg_G, ylen_bg_G, linewidth=2, edgecolor='black', facecolor='none')
    ax[0,0].add_patch(rectangle_R); ax[0,3].add_patch(rectangle_G) #add the rectangles to the plots
    ax[0,0].add_patch(rectangle_bg_R); ax[0,3].add_patch(rectangle_bg_G)
    ax[0,0].text(s='ROI before rotation',x=xmin_R+5,y=ymax_R+10,color='magenta') #label the ROI rectangles
    ax[0,3].text(s='ROI before rotation',x=xmin_G+5,y=ymax_G+10,color='magenta')
    ax[0,0].text(s='background region',x=xmin_bg_R+5,y=ymax_bg_R+10,color='black') #label the background rectangles
    ax[0,3].text(s='background region',x=xmin_bg_G+5,y=ymax_bg_G+10,color='black')

    # rotated+centered ROIs
    rows_R2, cols_R2 = rolled_ROI_R.shape # get shape of rotated+centered ROIs
    rows_G2, cols_G2 = rolled_ROI_G.shape
    XR2, YR2 = np.meshgrid(np.arange(cols_R2), np.arange(rows_R2)) # generate meshgrid for plotting
    XG2, YG2 = np.meshgrid(np.arange(cols_G2), np.arange(rows_G2))
    ax[1,0].axhline(y=rows_R2/2, color='black', linestyle='dashdot', linewidth=1) # plot center lines
    ax[1,3].axhline(y=rows_G2/2, color='black', linestyle='dashdot', linewidth=1)
    vlim_R = np.max(np.abs(rolled_ROI_R)) # symmetric color limits (maxof absolute value)
    vlim_G = np.max(np.abs(rolled_ROI_G))
    ax[1,0].pcolormesh(XR2, YR2, rolled_ROI_R, cmap='RdBu', vmin=-vlim_R, vmax=vlim_R) # plot rotated+centered ROIs
    ax[1,3].pcolormesh(XG2, YG2, rolled_ROI_G, cmap='RdBu', vmin=-vlim_G, vmax=vlim_G)
    ax[1,0].axis('equal'); ax[1,3].axis('equal') # set equal aspect ratio
    ax[1,0].set_title('rotated and centered ROI (RED)')
    ax[1,3].set_title('rotated and centered ROI (GREEN)')
    change_ticks(data=rolled_ROI_R,pixsize=pixsize,axis=ax[1,0],change_X=True,change_Y=True) # custom function to change ticks to microns
    change_ticks(data=rolled_ROI_G,pixsize=pixsize,axis=ax[1,3],change_X=True,change_Y=True)
    ax[1,3].yaxis.tick_right(); ax[1,3].yaxis.set_label_position("right") # move y-axis to the right for symmetry

    # 1D profiles with fits
    ax[1,1].plot(fit_y_R_2*1000, xdata_R, label='gauss fit', color='black', linewidth=1) # plot fitted RED profile in mrad
    ax[1,2].plot(fit_y_G_2*1000, xdata_G, label='gauss fit', color='black', linewidth=1) # plot fitted GREEN profile in mrad
    ax[1,1].plot(phase_projection_R*1000, xdata_R, label='data')
    ax[1,2].plot(phase_projection_G*1000, xdata_G, label='data')
    ax[1,1].axhline(y=0,color='black',linestyle='dashdot',linewidth=1)
    ax[1,2].axhline(y=0,color='black',linestyle='dashdot',linewidth=1)
    ax[1,1].set_xlim(1.1*min(phase_projection_R*1000), max(20, 1.1*max(phase_projection_R*1000))) # set x-limits to focus on relevant range
    ax[1,2].set_xlim(1.1*min(phase_projection_G*1000), max(20, 1.1*max(phase_projection_G*1000))) # set x-limits to focus on relevant range
    ax[1,1].set_xlabel('[mrad]'); ax[1,2].set_xlabel('[mrad]') 
    ax[1,1].set_ylabel(r'r [$\mu$m]'); ax[1,2].set_ylabel(r'r [$\mu$m]')
    ax[1,1].set_title('mean phase shift (projection)'); ax[1,2].set_title('mean phase shift (projection)')
    ax[1,2].yaxis.tick_right(); ax[1,2].yaxis.set_label_position("right")
    ax[1,1].legend(); ax[1,2].legend()

    # abel inversion + two-color
    #abel-transform the centered and rotated 2D ROIs (using the pyAbelTransform function defined above); ONLY the raw phase maps are inverted here, without converting to delta-eta yet
    abel_transformed_phase_R = pyAbelTransform(PhaseMap=rolled_ROI_R, plumeOrientation='horizontal') #2D Abel inversion
    abel_transformed_phase_G = pyAbelTransform(PhaseMap=rolled_ROI_G, plumeOrientation='horizontal')

    # plot abel-inverted phase maps (again, the raw phase maps, not delta-eta yet)
    ax[2,0].imshow(abel_transformed_phase_R, cmap='RdBu',
                   vmin=np.amin(abel_transformed_phase_R), vmax=-np.amin(abel_transformed_phase_R))# set limits symmetric around zero
    ax[2,3].imshow(abel_transformed_phase_G, cmap='RdBu',
                   vmin=np.amin(abel_transformed_phase_R), vmax=-np.amin(abel_transformed_phase_R))
    ax[2,0].set_title('Abel-inverted phase shift (RED)')
    ax[2,3].set_title('Abel-inverted phase shift (GREEN)')
    change_ticks(data=abel_transformed_phase_R,pixsize=pixsize,axis=ax[2,0],change_X=True,change_Y=True) # custom function to change ticks to microns
    change_ticks(data=abel_transformed_phase_R,pixsize=pixsize,axis=ax[2,3],change_X=True,change_Y=True)
    ax[2,3].yaxis.tick_right(); ax[2,3].yaxis.set_label_position("right") # move y-axis to the right for symmetry

    # create radial axes and radial profiles from the abel-inverted data (symmetrized around r=0 via splicing function defined above)
    N_size = len(np.mean(abel_transformed_phase_R,axis=1)) #size of the abel-inverted arrays along axis=0 (radial direction)
    xabel_R = splice_middle_elements(np.linspace(-N_size/2,N_size/2,N_size)*pixsize, N) # radial axis in microns for the Abel-inverted data (RED), via splicing function defined above
    xabel_G = xabel_R # same for GREEN (same size after inversion); important to use the same N here for both colors for the two-color analysis later

    # radial profiles of the Abel-inverted phase maps (mean and stddev along axis=1, then spliced to be symmetric around r=0)
    # these are already converted to delta-eta inside the pyAbelTransform function
    # Note the conversion from phase to delta-eta (refractive index perturbation) inside the pyAbelTransform function: PhaseMap*lambda/(2*pi*pixsize*1e-6)
    yabel_R = splice_middle_elements(np.mean(pyAbelTransform(PhaseMap=rolled_ROI_R*lambda_red/(2*np.pi*pixsize*1e-6), plumeOrientation='horizontal'),axis=1), N)
    yabel_G = splice_middle_elements(np.mean(pyAbelTransform(PhaseMap=rolled_ROI_G*lambda_green/(2*np.pi*pixsize*1e-6), plumeOrientation='horizontal'),axis=1), N)
    yabel_R_stdev = splice_middle_elements(np.std(pyAbelTransform(PhaseMap=rolled_ROI_R*lambda_red/(2*np.pi*pixsize*1e-6), plumeOrientation='horizontal'),axis=1), N)
    yabel_G_stdev = splice_middle_elements(np.std(pyAbelTransform(PhaseMap=rolled_ROI_G*lambda_green/(2*np.pi*pixsize*1e-6), plumeOrientation='horizontal'),axis=1), N)

    # plot abel-inverted delta-eta profiles
    ax[2,1].plot(xabel_R,1000*yabel_R,color='red',label='1030 nm')
    ax[2,1].plot(xabel_G,1000*yabel_G,color='green',label='515 nm')

    #plot shaded error regions using smoothed stddev (half width above, half width below):
    ax[2,1].fill_between(xabel_R,1000*(yabel_R+0.5*savgol_filter(yabel_R_stdev,15,2)),
                                   1000*(yabel_R-0.5*savgol_filter(yabel_R_stdev,15,2)),
                                   color='red',alpha=0.5,linewidth=0)
    ax[2,1].fill_between(xabel_G,1000*(yabel_G+0.5*savgol_filter(yabel_G_stdev,15,2)),
                                   1000*(yabel_G-0.5*savgol_filter(yabel_G_stdev,15,2)),
                                   color='green',alpha=0.5,linewidth=0)
    ax[2,1].set_ylabel(r'$\Delta \eta$(r) [10$^{-3}$]',labelpad=-45)
    ax[2,1].set_xlabel(r'r [$\mu$m]')
    ax[2,1].legend(loc='lower right')
    ax[2,1].set_title('Abel-inverted refractive\nindex perturbation')
    ax[2,1].set_xlim(-rlim_um,rlim_um) #focus on relevant range (defined manually in the constants at the start of the function)
    ax[2,1].set_ylim(1.15*1000*min(yabel_R),0.05) #focus on relevant range (shifted a bit to avoid cutting off the top of the curves)
    ax[2,1].tick_params(axis='y', direction='in', pad=-25) #move y-axis ticks inward to save space
    ax[2,2].set_title('Density profile') 

    #################################################################
    ## now perform the two-color analysis to extract ne(r) and dn0(r)
    ##
    ## the system of equations has the following form:
    ## delta_eta1(r) = A*ne(r) + B*dn0(r)
    ## delta_eta2(r) = C*ne(r) + D*dn0(r)
    ##
    ## where: delta_eta1,2(r) are the measured refractive index perturbations at the two wavelengths, specifically:
    ## delta_eta1(r[m]) = Abel^{-1} [ PhaseMap_RED(y[m])[rad]   * lambda_red[m]   / (2*pi*pixel_size[m]) ]
    ## delta_eta2(r[m]) = Abel^{-1} [ PhaseMap_GREEN(y[m])[rad] * lambda_green[m] / (2*pi*pixel_size[m]) ]
    ##
    ## and the constants A,B,C,D are defined as:
    ## A = -e^2/(8*pi^2*epsilon_0*m_e*c^2) * lambda_red^2
    ## B = (n0_mbar/n_STP_mbar) * delta_eta_H2(lambda_red)
    ## C = -e^2/(8*pi^2*epsilon_0*m_e*c^2) * lambda_green^2
    ## D = (n0_mbar/n_STP_mbar) * delta_eta_H2(lambda_green)

    ## here, the delta_eta_H2(lambda) function gives the refractive index change per unit fractional density change for molecular hydrogen at wavelength lambda (in microns).
    ## n_STP_mbar is the standard temperature and pressure number density in mbar (1000 mbar), and n0_mbar is the initial ambient pressure in mbar (input to this function).
    ## This gives the scaling of the neutral density contribution to delta_eta due to the initial ambient pressure
    ## The two_color function solves the above system of equations for ne(r) and dn0(r) given delta_eta1(r) and delta_eta2(r), which are found from the phase maps via Abel inversion as shown above.

    ## watch out for a few important things:
    ## 1) the delta_eta_H2(lambda) function expects wavelength in microns, so I multiply lambda_red and lambda_green by 1e6 when calling it
    ## 2) the output ne(r) will be in units of m^-3, so I convert it to cm^-3 by multiplying by 1e-6
    ## 3) the output dn0(r) is dimensionless (fractional change in neutral density)
    ##################################################################

    # two-color constants, as discussed right above:
    A = -sc.e**2/(8*np.pi**2*sc.epsilon_0*sc.m_e*sc.c**2)*lambda_red**2
    C = -sc.e**2/(8*np.pi**2*sc.epsilon_0*sc.m_e*sc.c**2)*lambda_green**2

    # delta-eta due to neutrals
    n_STP_mbar = 1000
    B = n0_mbar/n_STP_mbar*get_delta_eta_H2(lambda_red*1e6) # note the lambda in microns here (see the function definition above)
    D = n0_mbar/n_STP_mbar*get_delta_eta_H2(lambda_green*1e6)

    # now again compute radial profiles of delta-eta (mean and stddev along axis=1, then spliced to be symmetric around r=0)
    # 2D for now 
    delta_eta1_2D = pyAbelTransform(PhaseMap=rolled_ROI_R*lambda_red/(2*np.pi*pixsize*1e-6), plumeOrientation='horizontal')
    delta_eta2_2D = pyAbelTransform(PhaseMap=rolled_ROI_G*lambda_green/(2*np.pi*pixsize*1e-6), plumeOrientation='horizontal')

    # turn the 2D abel-inverted delta_eta into 1D radial profiles and their stddevs
    delta_eta1 = splice_middle_elements(np.mean(delta_eta1_2D,axis=1), N) #project the 2D delta-eta maps to 1D radial profiles, and splice to be symmetric around r=0
    sigma_delta_eta1 = splice_middle_elements(np.std(delta_eta1_2D,axis=1), N) #stddev profile of the 2D delta-eta map
    sigma_delta_eta1 = np.array([err_to_peak_R*max(sigma_delta_eta1) if i < err_to_peak_R*max(sigma_delta_eta1) else i for i in sigma_delta_eta1]) #enforce a minimum error based on error-to-peak metric

    # do the same for the GREEN channel:
    delta_eta2 = splice_middle_elements(np.mean(delta_eta2_2D,axis=1), N)
    sigma_delta_eta2 = splice_middle_elements(np.std(delta_eta2_2D,axis=1), N)
    sigma_delta_eta2 = np.array([err_to_peak_G*max(sigma_delta_eta2) if i < err_to_peak_G*max(sigma_delta_eta2) else i for i in sigma_delta_eta2])

    # solve the two-color equations to get ne(r) and dn0(r) using the function defined above (just a matrix inversion)
    soln = two_color(delta_eta1, delta_eta2, A, B, C, D)

    ne_r = soln[:,0]*1e-6 # convert the electron radial density profile from m^-3 to cm^-3, as noted above
    dn0_r = soln[:,1] # fractional change in neutral density profile (dimensionless)

    # The Abel-inversion likes to produce funky outliers (obvious spikes) around the center of the profiles, so I will smooth it using a Savitzky-Golay filter if the repair_abel flag is set to True
    # mostly set to False anyways though, as it can sometimes mess up real features (so use with caution!)
    if repair_abel:
        dn0_r = savgol_filter(dn0_r,15,2) #smoothing neutral density profile, form: (array, window size, polynomial order)
        ne_r = savgol_filter(ne_r,15,2) #smoothing electron density profile

    # plot densities
    ax[2,2].plot(xabel_R, ne_r, color='tab:orange', label=r'$n_e(r)$') #electron density profile
    ax[2,2].set_xlim(-rlim_um, rlim_um)
    ax[2,2].set_ylabel(r'$n_e(r)$ [cm$^{-3}$]', labelpad=-40, color='tab:orange') #
    ax[2,2].tick_params(axis='y', direction='in', pad=-22, colors='tab:orange') #move y-axis ticks inward to save space
    ax[2,2].spines['left'].set_color('tab:orange')
    ax[2,2].yaxis.label.set_color('tab:orange')
    ax[2,2].set_xlabel(r'r [$\mu$m]')

    ax22 = ax[2,2].twinx() #fractional change in neutral density profile on the same plot but with a different y-axis
    ax22.plot(xabel_R, dn0_r, color='black', label=r'$\delta n_0(r)$') #neutral density profile
    ax22.set_ylabel(r'$\Delta n_0/n_0$', labelpad=-45, color='black')
    ax22.set_ylim(-1.08, 1.1) #set y-limits to focus on relevant range (relative change in neutral gas density --> should be around +/- 1)
    ax22.tick_params(axis='y', direction='in', pad=-30, colors='black') #move y-axis ticks inward to save space
    ax22.spines['right'].set_color('black') #right y-axis spine color
    ax22.yaxis.label.set_color('black')

    plt.figtext(0.5, 0.91, plot_title, ha='center', va='center', size=16) #overall title for the figure

    # Savingthe radial profiles data to a text file:
    saveloc_txt = r"C:/Users/sann7609/Documents/Oxford/ChannelAnalysis_CALA_Sept25_minisforum/HOFI_profiles/CALA_Sept25_channels_txt_files"
    if not os.path.exists(saveloc_txt): os.makedirs(saveloc_txt) #create directory if it doesn't exist
    data_out = np.column_stack((xabel_R, ne_r, dn0_r)) #stack the data columns together
    # create a header (first line) for the text file:
    headers=('r [microns];',
             'electron density [cm^-3];',
             'fractional change in density of neutrals [cm^-3];')
    # save to text file with appropriate formatting (using tab as delimiter, and including the header):
    np.savetxt(os.path.join(saveloc_txt, f"{plot_name}.txt"),data_out,header='\t'.join(headers), comments='')

    results = { #return relevant results as a dictionary for further analysis in this notebook
        "sigma": sigma_R2, #RED gaussian fit width (microns)
        "xabel": xabel_R, #radial axis (microns)
        "ne_r":  ne_r, #electron radial density profile (cm^-3)
        "dn0_r":  dn0_r, #fractional change in neutral density profile (dimensionless)
    }
    return results #return the dictionary


# Now using the functions to process and plot the phase maps + extract the radial density profiles

In [ ]:
# define common paths for data folders (change these as needed; hardcoded for convenience here)
common_path = r"C:/Users/sann7609/Documents/Oxford/ChannelAnalysis_CALA_Sept25_minisforum/channel_analysis_250917_timescan" #common path for the timescan data
common_path_E = r"C:/Users/sann7609/Documents/Oxford/ChannelAnalysis_CALA_Sept25_minisforum/channel_analysis_250910_largeFFT" #common path for the energy scan data

In [ ]:
tscan_250917_Bdel560_100mbar_t0p0ns_red   = common_path_E + r"/phase_maps/100mbar_Bdel560_t0/Interferometry2"
tscan_250917_Bdel560_100mbar_t0p0ns_green = common_path_E + r"/phase_maps/100mbar_Bdel560_t0/Interferometry1"

res_tscan_250917_Bdel560_100mbar_t0p0ns=plot_2d_data(n0_mbar=100,
                        RED_filepath=tscan_250917_Bdel560_100mbar_t0p0ns_red,
                        GREEN_filepath=tscan_250917_Bdel560_100mbar_t0p0ns_green,
                        plot_name=r"tscan_250917_Bdel560_100mbar_t0p0ns",
                        plot_title=r"tscan_250917_Bdel560_100mbar_t0p0ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=True,
                        repair_abel=True,
                        xlims_um_R=[700,1200],
                        ylims_um_R=[100,600],
                        xlims_um_G=[300,1300],
                        ylims_um_G=[300,650],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
tscan_250917_Bdel560_100mbar_t1p0ns_red   = common_path + r"/phase_maps100mbar_Bdel560_t1.0ns/Interferometry2"
tscan_250917_Bdel560_100mbar_t1p0ns_green = common_path + r"/phase_maps100mbar_Bdel560_t1.0ns/Interferometry1"

res_tscan_250917_Bdel560_100mbar_t1p0ns=plot_2d_data(n0_mbar=100,
                        RED_filepath=tscan_250917_Bdel560_100mbar_t1p0ns_red,
                        GREEN_filepath=tscan_250917_Bdel560_100mbar_t1p0ns_green,
                        plot_name=r"tscan_250917_Bdel560_100mbar_t1p0ns",
                        plot_title=r"tscan_250917_Bdel560_100mbar_t1p0ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=True,
                        repair_abel=True,
                        xlims_um_R=[100,1300],
                        ylims_um_R=[150,600],
                        xlims_um_G=[300,1300],
                        ylims_um_G=[300,650],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
tscan_250917_Bdel560_100mbar_t2p0ns_red   = common_path + r"/phase_maps100mbar_Bdel560_t2.0ns/Interferometry2"
tscan_250917_Bdel560_100mbar_t2p0ns_green = common_path + r"/phase_maps100mbar_Bdel560_t2.0ns/Interferometry1"

res_tscan_250917_Bdel560_100mbar_t2p0ns=plot_2d_data(n0_mbar=100,
                        RED_filepath=tscan_250917_Bdel560_100mbar_t2p0ns_red,
                        GREEN_filepath=tscan_250917_Bdel560_100mbar_t2p0ns_green,
                        plot_name=r"tscan_250917_Bdel560_100mbar_t2p0ns",
                        plot_title=r"tscan_250917_Bdel560_100mbar_t2p0ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=True,
                        repair_abel=True,
                        xlims_um_R=[100,1300],
                        ylims_um_R=[150,600],
                        xlims_um_G=[300,1300],
                        ylims_um_G=[300,650],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
tscan_250917_Bdel560_100mbar_t2p5ns_red   = common_path + r"/phase_maps100mbar_Bdel560_t2.5ns/Interferometry2"
tscan_250917_Bdel560_100mbar_t2p5ns_green = common_path + r"/phase_maps100mbar_Bdel560_t2.5ns/Interferometry1"

res_tscan_250917_Bdel560_100mbar_t2p5ns=plot_2d_data(n0_mbar=100,
                        RED_filepath=tscan_250917_Bdel560_100mbar_t2p5ns_red,
                        GREEN_filepath=tscan_250917_Bdel560_100mbar_t2p5ns_green,
                        plot_name=r"tscan_250917_Bdel560_100mbar_t2p5ns",
                        plot_title=r"tscan_250917_Bdel560_100mbar_t2p5ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=False,
                        repair_abel=True,
                        xlims_um_R=[300,1300],
                        ylims_um_R=[150,600],
                        xlims_um_G=[500,1300],
                        ylims_um_G=[700,1050],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
tscan_250917_Bdel560_100mbar_t3p0ns_red   = common_path + r"/phase_maps100mbar_Bdel560_t3.0ns/Interferometry2"
tscan_250917_Bdel560_100mbar_t3p0ns_green = common_path + r"/phase_maps100mbar_Bdel560_t3.0ns/Interferometry1"

res_tscan_250917_Bdel560_100mbar_t3p0ns=plot_2d_data(n0_mbar=100,
                        RED_filepath=tscan_250917_Bdel560_100mbar_t3p0ns_red,
                        GREEN_filepath=tscan_250917_Bdel560_100mbar_t3p0ns_green,
                        plot_name=r"tscan_250917_Bdel560_100mbar_t3p0ns",
                        plot_title=r"tscan_250917_Bdel560_100mbar_t3p0ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=False,
                        repair_abel=True,
                        xlims_um_R=[200,1300],
                        ylims_um_R=[150,600],
                        xlims_um_G=[500,1300],
                        ylims_um_G=[700,1050],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
tscan_250917_Bdel560_100mbar_t3p5ns_red   = common_path + r"/phase_maps100mbar_Bdel560_t3.5ns/Interferometry2"
tscan_250917_Bdel560_100mbar_t3p5ns_green = common_path + r"/phase_maps100mbar_Bdel560_t3.5ns/Interferometry1"

res_tscan_250917_Bdel560_100mbar_t3p5ns=plot_2d_data(n0_mbar=100,
                        RED_filepath=tscan_250917_Bdel560_100mbar_t3p5ns_red,
                        GREEN_filepath=tscan_250917_Bdel560_100mbar_t3p5ns_green,
                        plot_name=r"tscan_250917_Bdel560_100mbar_t3p5ns",
                        plot_title=r"tscan_250917_Bdel560_100mbar_t3p5ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=False,
                        repair_abel=True,
                        xlims_um_R=[200,1000],
                        ylims_um_R=[150,600],
                        xlims_um_G=[500,1300],
                        ylims_um_G=[700,1050],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
tscan_250917_Bdel560_100mbar_t4p0ns_red   = common_path + r"/phase_maps100mbar_Bdel560_t4.0ns/Interferometry2"
tscan_250917_Bdel560_100mbar_t4p0ns_green = common_path + r"/phase_maps100mbar_Bdel560_t4.0ns/Interferometry1"

res_tscan_250917_Bdel560_100mbar_t4p0ns=plot_2d_data(n0_mbar=100,
                        RED_filepath=tscan_250917_Bdel560_100mbar_t4p0ns_red,
                        GREEN_filepath=tscan_250917_Bdel560_100mbar_t4p0ns_green,
                        plot_name=r"tscan_250917_Bdel560_100mbar_t4p0ns",
                        plot_title=r"tscan_250917_Bdel560_100mbar_t4p0ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=False,
                        repair_abel=True,
                        xlims_um_R=[200,1300],
                        ylims_um_R=[150,600],
                        xlims_um_G=[500,1300],
                        ylims_um_G=[700,1050],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
res_250917_100mbar_Bdel560 = [res_tscan_250917_Bdel560_100mbar_t0p0ns,
                       res_tscan_250917_Bdel560_100mbar_t1p0ns,
                       res_tscan_250917_Bdel560_100mbar_t2p0ns,
                       res_tscan_250917_Bdel560_100mbar_t2p5ns,
                       res_tscan_250917_Bdel560_100mbar_t3p0ns,
                       res_tscan_250917_Bdel560_100mbar_t3p5ns,
                       res_tscan_250917_Bdel560_100mbar_t4p0ns]

fig, ax = plt.subplot_mosaic([['00','01','02','03','04','05','06'],
                              ['10','11','12','13','14','15','16']], 
                              figsize=(17,4),
                              gridspec_kw={'width_ratios': [1,1,1,1,1,1,1], 'height_ratios':[1,1]})

top_plots = ['00', '01', '02', '03', '04', '05', '06']
bottom_plots = ['10', '11', '12', '13', '14', '15', '16']

xlim=120

dicts = res_250917_100mbar_Bdel560
times=[0,1,2,2.5,3,3.5,4]

ne_max = 1.2*np.max(dicts[0]['ne_r'])
dn0_min = -1.5

T_stand = 293.15 #K, standard temperature
Nb_100mbar = 100*(1013.25*100/1000)/(sc.k*T_stand)*1e-6 *2

# -------- user-settable: EXCLUDE the center; fit only where |r| in [r_in, r_out] (µm) --------
r_in, r_out = 45.0, 55.0   # example values; tweak as needed

def fit_parabola_exterior(x, y, r_in, r_out, y_sigma=None):
    """
    Fit y ≈ -1 + beta * r^2 using only |r| in [r_in, r_out].
    Optionally provide y_sigma for weights (1/sigma^2). Returns (R, f(x), mask).
    """
    x = np.asarray(x, dtype=float)
    y = np.asarray(y, dtype=float)

    # Annulus mask: use both wings, exclude the unreliable core
    mask = (np.abs(x) >= r_in) & (np.abs(x) <= r_out)

    # Need enough points to fit robustly
    if mask.sum() < 5:
        return np.nan, np.full_like(x, np.nan, dtype=float), mask

    xr = x[mask]
    yr = y[mask]

    X2 = xr**2
    rhs = (yr + 1.0)

    if y_sigma is not None:
        w = 1.0 / np.clip(np.asarray(y_sigma)[mask], 1e-12, np.inf)**2
    else:
        w = np.ones_like(xr)

    denom = np.sum(w * (X2**2))  # Σ w r^4
    if denom <= 0:
        return np.nan, np.full_like(x, np.nan, dtype=float), mask

    beta = np.sum(w * (X2 * rhs)) / denom

    R = np.nan if beta <= 0 else 1.0 / np.sqrt(beta)
    f = -1.0 + beta * (x**2)
    return R, f, mask


linestyle='dotted'
linewidth=1
line_color='black'

dx=1.07
alpha=0.3

for i,top_plot_name,bottom_plot_name,dict,time in zip(range(len(dicts)),top_plots,bottom_plots,dicts,times):
    axs0 = ax[top_plot_name]
    axs1 = ax[bottom_plot_name]

    axs0.set_title(str(time))

    if dict['xabel'] is not None:
        axs0.plot(dict['xabel'],dict['ne_r'],color='tab:orange')
        #axs0.fill_between(dict['xabel'],dict['ne_r']+1/2*dict['Delta_ne_r'],dict['ne_r']-1/2*dict['Delta_ne_r'],color='tab:orange',alpha=alpha,linewidth=0)
        axs1.plot(dict['xabel'],dict['dn0_r'],color='black')
            #axs1.fill_between(dict['xabel'],(dict['dn0_r']+1/2*dict['Delta_dn0_r']),(dict['dn0_r']-1/2*dict['Delta_dn0_r']),color='black',alpha=alpha,linewidth=0)

for plot_name in top_plots[:-1]:
    ax[plot_name].set_yticks([])
    ax[plot_name].set_xticks([])

for plot_name in bottom_plots[:-1]:
    ax[plot_name].set_yticks([])
    #ax[plot_name].set_xticks([])

for plot_name in bottom_plots:
    for y in [50,100]:
        ax[plot_name].axhline(y=y,color=line_color,linewidth=linewidth,linestyle=linestyle,alpha=0.2)
    for x in [-50,0,50]:
        ax[plot_name].axvline(x=x,color=line_color,linewidth=linewidth,linestyle=linestyle,alpha=0.2)

linestyle='dotted'
linewidth=1
line_color='black'



for i,plot_name,time in zip(range(len(top_plots)),top_plots,times):
    ax[plot_name].set_ylim(-0.1*ne_max,6e18)
    ax[plot_name].set_xlim(-xlim,xlim)
    ax[plot_name].axhline(y=0,color=line_color,linewidth=linewidth,linestyle=linestyle)
    ax[plot_name].axhline(y=Nb_100mbar,color='red',ls='dotted') #plot what 100% ionization level is!!

    for y in [2e18,4e18]:
        ax[plot_name].axhline(y=y,color=line_color,linewidth=linewidth,linestyle=linestyle,alpha=0.2)
    for x in [-50,0,50]:
        ax[plot_name].axvline(x=x,color=line_color,linewidth=linewidth,linestyle=linestyle,alpha=0.2)

    ax[plot_name].set_title(str(time)+' ns')  # Remove default title
    # Manually add two lines of text with different styles



for plot_name in bottom_plots[:-1]:
    ax[plot_name].set_yticks([])

for plot_name in bottom_plots:
    ax[plot_name].set_ylim(-1.5,2)
    ax[plot_name].set_xlim(-xlim,xlim)
    ax[plot_name].axhline(y=Nb_100mbar,color=line_color,linewidth=linewidth,linestyle=linestyle)
    for y in [0,-1,-0.5]:
        ax[plot_name].axhline(y=y,color=line_color,linewidth=linewidth,linestyle=linestyle,alpha=0.2)
    for x in [-50,0,50]:
        ax[plot_name].axvline(x=x,color=line_color,linewidth=linewidth,linestyle=linestyle,alpha=0.2)

ax['00'].set_ylabel(r'$n_e$ [cm$^{-3}$]',color='tab:orange',size=13)
ax['10'].set_ylabel(r'$\Delta n/n_0$',color='black',size=13)

ax['10'].set_xlabel(r'r [$\mu$m]')
ax['11'].set_xticks([])
ax['12'].set_xlabel(r'r [$\mu$m]')
ax['13'].set_xticks([])
ax['14'].set_xlabel(r'r [$\mu$m]')
ax['15'].set_xticks([])
ax['16'].set_xlabel(r'r [$\mu$m]')


ax['06'].yaxis.tick_right()
ax['06'].yaxis.set_label_position("right")

ax['16'].yaxis.tick_right()
ax['16'].yaxis.set_label_position("right")

ax['00'].text(x=-30,y=1.05*Nb_100mbar,s='100%',color='red')
ax['00'].text(x=-50,y=0.85*Nb_100mbar,s='inoization',color='red')

plt.subplots_adjust(wspace=0, hspace=0)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.constants as sc  # for sc.k

# ------------------------------ Data lists ------------------------------
res_250917_100mbar_Bdel560 = [
    res_tscan_250917_Bdel560_100mbar_t0p0ns,
    res_tscan_250917_Bdel560_100mbar_t1p0ns,
    res_tscan_250917_Bdel560_100mbar_t2p0ns,
    res_tscan_250917_Bdel560_100mbar_t2p5ns,
    res_tscan_250917_Bdel560_100mbar_t3p0ns,
    res_tscan_250917_Bdel560_100mbar_t3p5ns,
    res_tscan_250917_Bdel560_100mbar_t4p0ns
]
dicts = res_250917_100mbar_Bdel560
times = [0.0, 1.0, 2.0, 2.5, 3.0, 3.5, 4.0]  # ns

# ------------------------------ Figure scaffold ------------------------------
fig, ax = plt.subplot_mosaic(
    [['00','01','02','03','04','05','06'],
     ['10','11','12','13','14','15','16']],
    figsize=(17, 4),
    gridspec_kw={'width_ratios': [1,1,1,1,1,1,1], 'height_ratios':[1,1]}
)
top_plots    = ['00', '01', '02', '03', '04', '05', '06']
bottom_plots = ['10', '11', '12', '13', '14', '15', '16']

xlim       = 120
linestyle  = 'dotted'
linewidth  = 1
line_color = 'black'
alpha      = 0.3

# ------------------------------ Physics helpers ------------------------------
T_stand = 293.15  # K
Nb_100mbar = 100*(1013.25*100/1000)/(sc.k*T_stand)*1e-6 * 2  # cm^-3 (100% ionization ref for top row)

# ------------------------------ Convex quartic fit ------------------------------
def _weighted_sums(r2, r4, z, w):
    S22 = np.sum(w * (r2 * r2))    # Σ w r^4
    S24 = np.sum(w * (r2 * r4))    # Σ w r^6
    S44 = np.sum(w * (r4 * r4))    # Σ w r^8
    t2  = np.sum(w * (r2 * z))     # Σ w r^2 z
    t4  = np.sum(w * (r4 * z))     # Σ w r^4 z
    return S22, S24, S44, t2, t4

def fit_quartic_convex_from_annulus(x, y, r_in, r_out, y_sigma=None, ridge=0.0, force_pure_quartic=False):
    """
    Fit convex f(r) = -1 + a r^2 + b r^4 using only |r| in [r_in, r_out].
    Enforces convexity by requiring a >= 0, b >= 0 via a simple NNLS-by-cases:
      - If unconstrained solution has a>=0 and b>=0, use it.
      - Else, try boundary fits: (a>=0,b=0) and (a=0,b>=0). Pick smallest weighted SSE.
    If force_pure_quartic=True, we fit a=0,b>=0 only (widest convex center).
    Returns: a, b, f(x), mask, R_eff
    """
    x = np.asarray(x, dtype=float)
    y = np.asarray(y, dtype=float)
    mask = (np.abs(x) >= r_in) & (np.abs(x) <= r_out)
    if mask.sum() < 5:
        return np.nan, np.nan, np.full_like(x, np.nan), mask, np.nan

    xr  = x[mask]
    yr  = y[mask]
    z   = yr + 1.0
    r2  = xr**2
    r4  = r2**2

    if y_sigma is not None:
        w = 1.0 / np.clip(np.asarray(y_sigma)[mask], 1e-12, np.inf)**2
    else:
        w = np.ones_like(xr)

    # helper to compute SSE for given (a,b)
    def sse(a, b):
        res = z - (a*r2 + b*r4)
        return np.sum(w * res**2)

    S22, S24, S44, t2, t4 = _weighted_sums(r2, r4, z, w)

    # Boundary fits
    # (1) a>=0, b=0  -> solve for a
    a_lin = max(0.0, (t2) / (S22 + ridge)) if (S22 + ridge) > 0 else 0.0
    sse_a_only = sse(a_lin, 0.0)

    # (2) a=0, b>=0  -> solve for b  (pure quartic; "wider than parabola")
    b_lin = max(0.0, (t4) / (S44 + ridge)) if (S44 + ridge) > 0 else 0.0
    sse_b_only = sse(0.0, b_lin)

    if force_pure_quartic:
        a, b = 0.0, b_lin
    else:
        # Unconstrained 2x2 solve
        A = np.array([[S22 + ridge, S24],
                      [S24,         S44 + ridge]], dtype=float)
        bvec = np.array([t2, t4], dtype=float)
        try:
            a0, b0 = np.linalg.solve(A, bvec)
        except np.linalg.LinAlgError:
            a0, b0 = np.nan, np.nan

        if np.isfinite(a0) and np.isfinite(b0) and (a0 >= 0.0) and (b0 >= 0.0):
            a, b = a0, b0
        else:
            # pick best boundary
            if sse_a_only <= sse_b_only:
                a, b = a_lin, 0.0
            else:
                a, b = 0.0, b_lin

    # Fitted curve across all x
    f = -1.0 + a*(x**2) + b*(x**4)

    # Effective rim (smallest positive root of a r^2 + b r^4 = 1), if it exists
    R_eff = np.nan
    if b > 0 or a > 0:
        if b > 0:
            disc = a*a + 4.0*b
            s_list = []
            if disc >= 0:
                s1 = (-a - np.sqrt(disc)) / (2.0*b)
                s2 = (-a + np.sqrt(disc)) / (2.0*b)
                for s in (s1, s2):
                    if s > 0:
                        s_list.append(s)
            if s_list:
                R_eff = np.sqrt(min(s_list))
        elif a > 0:
            R_eff = 1.0 / np.sqrt(a)

    return a, b, f, mask, R_eff

# ------------------------------ Per-panel annulus bounds ------------------------------
default_bounds = (45.0, 55.0)  # µm, used if a panel isn't specified below
fit_bounds_by_key = {
    '10': (45.0, 55.0),
    '11': (40.0, 47.0),
    '12': (35.0, 50.0),
    '13': (45.0, 60.0),
    '14': (10.0, 60.0),
    '15': (10.0, 68.0),
    '16': (50.0, 70.0),
}
def bounds_for(key):
    return fit_bounds_by_key.get(key, default_bounds)

# ------------------------------ Plot data ------------------------------
ne_max = 1.2 * np.max(dicts[0]['ne_r'])

for i, top_key, bot_key, d, t in zip(range(len(dicts)), top_plots, bottom_plots, dicts, times):
    axs0 = ax[top_key]
    axs1 = ax[bot_key]

    # Top row: n_e(r)
    if d['xabel'] is not None:
        axs0.plot(d['xabel'], d['ne_r'], color='tab:orange', lw=1.5)

    axs0.set_xlim(-xlim, xlim)
    axs0.set_ylim(-0.1*ne_max, 6e18)
    axs0.axhline(0, color=line_color, linewidth=linewidth, linestyle=linestyle)
    axs0.axhline(Nb_100mbar, color='red', ls='dotted')
    for y in [2e18, 4e18]:
        axs0.axhline(y, color=line_color, linewidth=linewidth, linestyle=linestyle, alpha=0.2)
    for xv in [-50, 0, 50]:
        axs0.axvline(xv, color=line_color, linewidth=linewidth, linestyle=linestyle, alpha=0.2)
    axs0.set_title(f'{t:g} ns', fontsize=10)

    # Bottom row: Δn/n0
    if d['xabel'] is not None:
        axs1.plot(d['xabel'], d['dn0_r'], color='black', lw=1.2)

    axs1.set_xlim(-xlim, xlim)
    axs1.set_ylim(-1.5, 2.0)
    for y in [0.0, -1.0, -0.5, 0.5, 1.0]:
        axs1.axhline(y, color=line_color, linewidth=linewidth, linestyle=linestyle, alpha=0.2)
    for xv in [-50, 0, 50]:
        axs1.axvline(xv, color=line_color, linewidth=linewidth, linestyle=linestyle, alpha=0.2)

# Tidy ticks and labels
for key in top_plots[:-1]:
    ax[key].set_yticks([]); ax[key].set_xticks([])
ax[top_plots[-1]].yaxis.tick_right()
ax[top_plots[-1]].yaxis.set_label_position("right")

for key in bottom_plots[:-1]:
    ax[key].set_yticks([])
ax[bottom_plots[-1]].yaxis.tick_right()
ax[bottom_plots[-1]].yaxis.set_label_position("right")

ax['00'].set_ylabel(r'$n_e$ [cm$^{-3}$]', color='tab:orange', size=13)
ax['10'].set_ylabel(r'$\Delta n/n_0$', color='black', size=13)

ax['10'].set_xlabel(r'r [$\mu$m]')
ax['12'].set_xlabel(r'r [$\mu$m]')
ax['14'].set_xlabel(r'r [$\mu$m]')
ax['16'].set_xlabel(r'r [$\mu$m]')

# Small annotations on top-left panel
ax['00'].text(x=-30, y=1.05*Nb_100mbar, s='100%', color='red')
ax['00'].text(x=-65, y=0.85*Nb_100mbar, s='ionization', color='red')

# ------------------------------ Run fits & overlay ------------------------------
fit_summaries = []  # collect (key, a, b, R_eff)

# Set to True if you want a strictly "wider than parabola" shape (a=0, b>=0)
force_pure_quartic = False

for bot_key, d in zip(bottom_plots, dicts):
    axs1 = ax[bot_key]
    if d['xabel'] is None:
        fit_summaries.append((bot_key, np.nan, np.nan, np.nan))
        continue

    x = np.asarray(d['xabel'])
    y = np.asarray(d['dn0_r'])
    y_sigma = d.get('Delta_dn0_r', None)

    r_in, r_out = bounds_for(bot_key)
    a, b, f, mask, R_eff = fit_quartic_convex_from_annulus(
        x, y, r_in, r_out, y_sigma=y_sigma, ridge=0.0, force_pure_quartic=force_pure_quartic
    )
    fit_summaries.append((bot_key, a, b, R_eff))

    # Draw the convex quartic across the full range (solid)
    line_fit, = axs1.plot(x, f, lw=2.0, label='Convex quartic fit')
    c = line_fit.get_color()

    # Shade the used annulus and excluded core
    axs1.axvspan(-r_in,  r_in,  color='gray', alpha=0.10, lw=0)
    axs1.axvspan(-r_out, -r_in, color=c,     alpha=0.06, lw=0)
    axs1.axvspan( r_in,   r_out, color=c,     alpha=0.06, lw=0)

    # Mark the rims with guide lines
    for xv in (-r_out, -r_in, r_in, r_out):
        axs1.axvline(xv, color=c, lw=1.2, ls='-')

    # Annotate effective rim if real/positive (where f crosses 0)
    if np.isfinite(R_eff):
        axs1.axvline( R_eff,  color=c, lw=1.0, ls='--', alpha=0.8)
        axs1.axvline(-R_eff,  color=c, lw=1.0, ls='--', alpha=0.8)
        axs1.text(0.02, 0.07, f'$R_{{\\rm eff}}$ = {R_eff:.1f} µm',
                  transform=axs1.transAxes, fontsize=10, color=c)
    else:
        axs1.text(0.02, 0.07, f'a={a:.3e}, b={b:.3e}',
                  transform=axs1.transAxes, fontsize=9, color=c)

# Single legend
ax['10'].legend(loc='upper right', frameon=False)

plt.subplots_adjust(wspace=0, hspace=0)

# Console summary
for key, a, b, R_eff in fit_summaries:
    if np.isfinite(a) and np.isfinite(b):
        s = f"{key}: a={a:.6e}, b={b:.6e}"
        if np.isfinite(R_eff):
            s += f", R_eff={R_eff:.2f} µm"
        print(s)
    else:
        print(f"{key}: fit failed")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.constants as sc  # for sc.k

saveloc_txt = r"/Users/sebastiankalos/Documents/OXFORD/FBPIC_102/HOFI_profiles/CALA_Sept25_channels_txt_files"
os.makedirs(saveloc_txt, exist_ok=True)

# ------------------------------ Data lists ------------------------------
res_250917_100mbar_Bdel560 = [
    res_tscan_250917_Bdel560_100mbar_t0p0ns,
    res_tscan_250917_Bdel560_100mbar_t1p0ns,
    res_tscan_250917_Bdel560_100mbar_t2p0ns,
    res_tscan_250917_Bdel560_100mbar_t2p5ns,
    res_tscan_250917_Bdel560_100mbar_t3p0ns,
    res_tscan_250917_Bdel560_100mbar_t3p5ns,
    res_tscan_250917_Bdel560_100mbar_t4p0ns
]
dicts = res_250917_100mbar_Bdel560
times = [0.0, 1.0, 2.0, 2.5, 3.0, 3.5, 4.0]  # ns
# --- filename meta (minimal changes) ---
DATE_TAG = "250917"       # <-- set this once
PRESSURE_MBAR = 100        # <-- set this once
EXPERIMENT_TAG = "Bdel560"  # optional; set "" if you don't want it

def time_tag(t):
    """e.g. 0.0 -> t0ns, 1.0 -> t1ns, 2.5 -> t2p5ns"""
    s = f"{t:g}"
    return "t" + s.replace(".", "p") + "ns"


# ------------------------------ Figure scaffold ------------------------------
fig, ax = plt.subplot_mosaic(
    [['00','01','02','03','04','05','06'],
     ['10','11','12','13','14','15','16']],
    figsize=(14, 4),
    gridspec_kw={'width_ratios': [1,1,1,1,1,1,1], 'height_ratios':[1,1]}
)
top_plots    = ['00', '01', '02', '03', '04', '05', '06']
bottom_plots = ['10', '11', '12', '13', '14', '15', '16']

xlim       = 140
linestyle  = 'dotted'
linewidth  = 1
line_color = 'black'

# ------------------------------ Physics helpers ------------------------------
T_stand = 293.15  # K
Nb_100mbar = 100*(1013.25*100/1000)/(sc.k*T_stand)*1e-6 * 2  # cm^-3 (100% ionization ref for top row)

# ------------------------------ Convex quartic fit ------------------------------
def _weighted_sums(r2, r4, z, w):
    S22 = np.sum(w * (r2 * r2))    # Σ w r^4
    S24 = np.sum(w * (r2 * r4))    # Σ w r^6
    S44 = np.sum(w * (r4 * r4))    # Σ w r^8
    t2  = np.sum(w * (r2 * z))     # Σ w r^2 z
    t4  = np.sum(w * (r4 * z))     # Σ w r^4 z
    return S22, S24, S44, t2, t4

def fit_quartic_convex_from_annulus(x, y, r_in, r_out, y_sigma=None, ridge=0.0, force_pure_quartic=False):
    """
    Fit convex f(r) = -1 + a r^2 + b r^4 using only |r| in [r_in, r_out].
    Enforces convexity via a>=0, b>=0. If force_pure_quartic=True, set a=0 and fit b>=0 only.
    Returns: a, b, f(x over all x), mask_used, R_eff
    """
    x = np.asarray(x, dtype=float)
    y = np.asarray(y, dtype=float)
    mask = (np.abs(x) >= r_in) & (np.abs(x) <= r_out)
    if mask.sum() < 5:
        return np.nan, np.nan, np.full_like(x, np.nan), mask, np.nan

    xr  = x[mask]
    yr  = y[mask]
    z   = yr + 1.0
    r2  = xr**2
    r4  = r2**2

    if y_sigma is not None:
        w = 1.0 / np.clip(np.asarray(y_sigma)[mask], 1e-12, np.inf)**2
    else:
        w = np.ones_like(xr)

    def sse(a, b):
        res = z - (a*r2 + b*r4)
        return np.sum(w * res**2)

    S22, S24, S44, t2, t4 = _weighted_sums(r2, r4, z, w)

    # Boundary fits (convexity-safe)
    a_lin = max(0.0, (t2) / (S22 + ridge)) if (S22 + ridge) > 0 else 0.0
    b_lin = max(0.0, (t4) / (S44 + ridge)) if (S44 + ridge) > 0 else 0.0
    sse_a_only = sse(a_lin, 0.0)
    sse_b_only = sse(0.0, b_lin)

    if force_pure_quartic:
        a, b = 0.0, b_lin
    else:
        # Unconstrained 2x2
        A = np.array([[S22 + ridge, S24],
                      [S24,         S44 + ridge]], dtype=float)
        bvec = np.array([t2, t4], dtype=float)
        try:
            a0, b0 = np.linalg.solve(A, bvec)
        except np.linalg.LinAlgError:
            a0, b0 = np.nan, np.nan

        if np.isfinite(a0) and np.isfinite(b0) and (a0 >= 0.0) and (b0 >= 0.0):
            a, b = a0, b0
        else:
            a, b = (a_lin, 0.0) if sse_a_only <= sse_b_only else (0.0, b_lin)

    f = -1.0 + a*(x**2) + b*(x**4)

    # Effective radius where f(r)=0 (if exists)
    R_eff = np.nan
    if b > 0:
        disc = a*a + 4.0*b
        if disc >= 0:
            s1 = (-a - np.sqrt(disc)) / (2.0*b)
            s2 = (-a + np.sqrt(disc)) / (2.0*b)
            s_pos = [s for s in (s1, s2) if s > 0]
            if s_pos:
                R_eff = np.sqrt(min(s_pos))
    elif a > 0:
        R_eff = 1.0 / np.sqrt(a)

    return a, b, f, mask, R_eff

# ------------------------------ Per-panel annulus bounds ------------------------------
default_bounds = (45.0, 55.0)  # µm
fit_bounds_by_key = {
    '10': (45.0, 55.0),
    '11': (30.0, 33.0),
    '12': (35.0, 40.0),
    '13': (45.0, 50.0),
    '14': (45.0, 51.0),
    '15': (50.0, 57.0),
    '16': (55.0, 60.0),
}
def bounds_for(key):
    return fit_bounds_by_key.get(key, default_bounds)

# ------------------------------ Plot data (top row) ------------------------------
ne_max = 1.2 * np.max(dicts[0]['ne_r'])

for i, top_key, bot_key, d, t in zip(range(len(dicts)), top_plots, bottom_plots, dicts, times):
    axs0 = ax[top_key]
    if d['xabel'] is not None:
        axs0.plot(d['xabel'], d['ne_r']/1e18, color='tab:orange', lw=1.5)

    axs0.set_xlim(-xlim, xlim)
    axs0.set_ylim(0, 6)
    axs0.axhline(0, color=line_color, linewidth=linewidth, linestyle=linestyle)
    for yline in [2, 4]:
        axs0.axhline(yline, color=line_color, linewidth=linewidth, linestyle=linestyle, alpha=0.1)
    for xv in [-100,-50, 0, 50,100]:
        axs0.axvline(xv, color=line_color, linewidth=linewidth, linestyle=linestyle, alpha=0.1)
    axs0.set_title(f'{t:g} ns', fontsize=14)

# ------------------------------ Axes cosmetics (bottom row too) ------------------------------
for key in top_plots[1:-1]:
    ax[key].set_yticks([]); ax[key].set_xticks([])
ax[top_plots[-1]].yaxis.tick_right()
ax[top_plots[-1]].yaxis.set_label_position("right")

for key in bottom_plots[1:-1]:
    ax[key].set_yticks([])
ax[bottom_plots[-1]].yaxis.tick_right()
ax[bottom_plots[-1]].yaxis.set_label_position("right")

for key in bottom_plots:
    ax[key].set_xlim(-xlim, xlim)
    ax[key].set_ylim(-1.0, 1.9)
    for yline in [0.0, -1.0]:
        ax[key].axhline(yline, color=line_color, linewidth=linewidth, linestyle=linestyle, alpha=0.2)
    for xv in [-100,-50, 0, 50,100]:
        ax[key].axvline(xv, color=line_color, linewidth=linewidth, linestyle=linestyle, alpha=0.1)

ax['00'].axhline(Nb_100mbar/1e18, color='red', ls='dashed')
ax['00'].set_ylabel(r'$n_e$ [10$^{18}$ cm$^{-3}$]', color='tab:orange', size=14)
ax['10'].set_ylabel(r'$\Delta n/n_0$', color='black', size=14)
ax['10'].set_xlabel(r'r [$\mu$m]')
ax['12'].set_xlabel(r'r [$\mu$m]')
ax['14'].set_xlabel(r'r [$\mu$m]')
ax['16'].set_xlabel(r'r [$\mu$m]')

ax['00'].text(x=-90, y=1.05*Nb_100mbar/1e18, s='100% ionization', color='red')

# ------------------------------ Fits & patched display (bottom row) ------------------------------
fit_summaries = []
force_pure_quartic = False  # True -> enforce a=0, b>=0 (extra-wide convex core)

for bot_key, d, t in zip(bottom_plots, dicts, times):
    axs1 = ax[bot_key]

        # --- Special case: 0 ns bottom plot -> save file with neutrals = -ne/ne_full_ion ([-1,0]) and draw nothing ---
    if t == 0.0:
        # x and electrons
        x = np.asarray(d.get('xabel', []), dtype=float)
        if x.size == 0:
            x = np.array([np.nan])
        ne_r = np.asarray(d.get('ne_r', np.full_like(x, np.nan)), dtype=float)
        ne_r_1e18 = ne_r / 1e18

        # full-ion density at this pressure (cm^-3)
        T_stand = 293.15
        ne_full_ion = PRESSURE_MBAR * (1013.25 * 100 / 1000) / (sc.k * T_stand) * 1e-6 * 2

        # neutrals at t=0 as relative change: Δn/n0 = - ne / ne_full_ion  (clip to [-1, 0])
        with np.errstate(divide='ignore', invalid='ignore'):
            dn0_rel_t0 = - ne_r / ne_full_ion
        dn0_rel_t0 = np.clip(dn0_rel_t0, -1.0, 0.0)

        # In the saved file we keep: patched, original, fit. For t=0 use the same dn0_rel_t0 in all 3.
        dn0_patched = dn0_rel_t0.copy()
        y_original  = dn0_rel_t0.copy()
        f_fit       = dn0_rel_t0.copy()

        # one file with r, electrons, and neutrals (patched/original/fit)
        data_out = np.column_stack((x, ne_r, ne_r_1e18, dn0_patched, y_original, f_fit))
        headers = (
            'r [microns];'
            '\tn_e [cm^-3];'
            '\tn_e [1e18 cm^-3];'
            '\tΔn/n0 (patched, quartic fit inside r_out);'
            '\tΔn/n0 (original);'
            '\tΔn/n0 (fit over all r);'
        )

        fname = f"{DATE_TAG}_{PRESSURE_MBAR}mbar"
        if EXPERIMENT_TAG:
            fname += f"_{EXPERIMENT_TAG}"
        fname += f"_{time_tag(t)}.txt"

        np.savetxt(os.path.join(saveloc_txt, fname), data_out, header=headers, comments='')

        # keep plotting behavior: nothing drawn at t=0
        fit_summaries.append((bot_key, np.nan, np.nan, np.nan))
        continue


    if d['xabel'] is None:
        fit_summaries.append((bot_key, np.nan, np.nan, np.nan))
        continue

    x = np.asarray(d['xabel'])
    y = np.asarray(d['dn0_r'])
    y_sigma = d.get('Delta_dn0_r', None)

    r_in, r_out = bounds_for(bot_key)

    a, b, f, mask_used, R_eff = fit_quartic_convex_from_annulus(
        x, y, r_in, r_out, y_sigma=y_sigma, ridge=0.0, force_pure_quartic=force_pure_quartic
    )
    fit_summaries.append((bot_key, a, b, R_eff))

    # ---------- Patch logic with small overlap (no gap/bridge) ----------
    if len(x) > 1:
        dx_med = np.median(np.diff(np.sort(x)))
        eps = 0.5 * dx_med if np.isfinite(dx_med) and dx_med > 0 else 0.5
    else:
        eps = 0.5

    mask_inside  = (np.abs(x) <= (r_out + eps))   # dotted region (a hair wider)
    # Solid outside measured data
    y_outside = y.copy()
    y_outside[mask_inside] = np.nan
    axs1.plot(x, y_outside, color='black', lw=1.2, ls='-',label='data')

    # Dotted fitted curve inside (replacement, slight overlap)
    f_inside = np.full_like(f, np.nan, dtype=float)
    f_inside[mask_inside] = f[mask_inside]
    axs1.plot(x, f_inside, color='black', lw=1.2, ls='dotted',label='fit')

    # ---------- Vertical dotted seam at x = ±r_out ----------
    # Take y-values from the nearest samples to ±r_out
    def nearest_idx(val):
        return int(np.nanargmin(np.abs(x - val)))

    for x_seam in (r_out, -r_out):
        i_seam = nearest_idx(x_seam)
        # Measured (outside) value and fitted (inside) value at the seam
        y_meas = y[i_seam]
        y_fit  = f[i_seam]
        # Only draw if both are finite
        if np.isfinite(y_meas) and np.isfinite(y_fit):
            y0, y1 = (y_meas, y_fit) if y_meas <= y_fit else (y_fit, y_meas)
            axs1.plot([x_seam, x_seam], [y0, y1], color='black', lw=1.2, ls='dotted')

    # ---------- Build "patched" neutrals (replace |r| <= r_out with fit) ----------
    mask_patch   = (np.abs(x) <= r_out)
    dn0_patched  = y.copy()
    dn0_patched[mask_patch] = f[mask_patch]

    # ---------- Electrons ----------
    ne_r      = np.asarray(d['ne_r'])       # [cm^-3]
    ne_r_1e18 = ne_r / 1e18                 # [10^18 cm^-3] convenience column

    # ---------- Combine into one table ----------
    # Columns: r, electrons (cm^-3), electrons (1e18 cm^-3),
    #          neutrals patched, neutrals original, neutrals fit
    data_out = np.column_stack((x, ne_r, ne_r_1e18, dn0_patched, y, f))
    headers = (
        'r [microns];'
        '\tn_e [cm^-3];'
        '\tn_e [1e18 cm^-3];'
        '\tΔn/n0 (patched, quartic fit inside r_out);'
        '\tΔn/n0 (original);'
        '\tΔn/n0 (fit over all r);'
    )

    # ---------- Save (date + pressure + tag + time) ----------
    fname = f"{DATE_TAG}_{PRESSURE_MBAR}mbar"
    if EXPERIMENT_TAG:
        fname += f"_{EXPERIMENT_TAG}"
    fname += f"_{time_tag(t)}.txt"

    np.savetxt(os.path.join(saveloc_txt, fname), data_out,
            header=headers, comments='')


ax['12'].legend()
ax['01'].text(s='electrons',x=-80,y=2.4,color='tab:orange',size=14)
ax['11'].text(s='neutrals',x=-80,y=0.8,color='black',size=14)
ax['10'].text(s='100 mbar\n ambient\npressure',x=-80,y=0.3,color='black',size=14)
ax['00'].yaxis.set_label_position("left")
plt.subplots_adjust(wspace=0, hspace=0)



# 100 mbar, B delay 582:

In [ ]:
tscan_250917_Bdel582_100mbar_t0p0ns_red   = common_path_E + r"/phase_maps/100mbar_Bdel582_t0/Interferometry2"
tscan_250917_Bdel582_100mbar_t0p0ns_green = common_path_E + r"/phase_maps/100mbar_Bdel582_t0/Interferometry1"

res_tscan_250917_Bdel582_100mbar_t0p0ns=plot_2d_data(n0_mbar=100,
                        RED_filepath=tscan_250917_Bdel582_100mbar_t0p0ns_red,
                        GREEN_filepath=tscan_250917_Bdel582_100mbar_t0p0ns_green,
                        plot_name=r"tscan_250917_Bdel582_100mbar_t0p0ns",
                        plot_title=r"tscan_250917_Bdel582_100mbar_t0p0ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=True,
                        repair_abel=True,
                        xlims_um_R=[800,1200],
                        ylims_um_R=[150,600],
                        xlims_um_G=[900,1300],
                        ylims_um_G=[300,650],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
tscan_250917_Bdel582_100mbar_t1p0ns_red   = common_path + r"/phase_maps100mbar_Bdel582_t1.0ns/Interferometry2"
tscan_250917_Bdel582_100mbar_t1p0ns_green = common_path + r"/phase_maps100mbar_Bdel582_t1.0ns/Interferometry1"

res_tscan_250917_Bdel582_100mbar_t1p0ns=plot_2d_data(n0_mbar=100,
                        RED_filepath=tscan_250917_Bdel582_100mbar_t1p0ns_red,
                        GREEN_filepath=tscan_250917_Bdel582_100mbar_t1p0ns_green,
                        plot_name=r"tscan_250917_Bdel582_100mbar_t1p0ns",
                        plot_title=r"tscan_250917_Bdel582_100mbar_t1p0ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=False,
                        repair_abel=True,
                        xlims_um_R=[200,1300],
                        ylims_um_R=[150,600],
                        xlims_um_G=[200,1300],
                        ylims_um_G=[700,1050],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
tscan_250917_Bdel582_100mbar_t2p0ns_red   = common_path + r"/phase_maps100mbar_Bdel582_t2.0ns/Interferometry2"
tscan_250917_Bdel582_100mbar_t2p0ns_green = common_path + r"/phase_maps100mbar_Bdel582_t2.0ns/Interferometry1"

res_tscan_250917_Bdel582_100mbar_t2p0ns=plot_2d_data(n0_mbar=100,
                        RED_filepath=tscan_250917_Bdel582_100mbar_t2p0ns_red,
                        GREEN_filepath=tscan_250917_Bdel582_100mbar_t2p0ns_green,
                        plot_name=r"tscan_250917_Bdel582_100mbar_t2p0ns",
                        plot_title=r"tscan_250917_Bdel582_100mbar_t2p0ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=False,
                        repair_abel=True,
                        xlims_um_R=[200,1300],
                        ylims_um_R=[150,600],
                        xlims_um_G=[200,1300],
                        ylims_um_G=[700,1050],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
tscan_250917_Bdel582_100mbar_t2p5ns_red   = common_path + r"/phase_maps100mbar_Bdel582_t2.5ns/Interferometry2"
tscan_250917_Bdel582_100mbar_t2p5ns_green = common_path + r"/phase_maps100mbar_Bdel582_t2.5ns/Interferometry1"

res_tscan_250917_Bdel582_100mbar_t2p5ns=plot_2d_data(n0_mbar=100,
                        RED_filepath=tscan_250917_Bdel582_100mbar_t2p5ns_red,
                        GREEN_filepath=tscan_250917_Bdel582_100mbar_t2p5ns_green,
                        plot_name=r"tscan_250917_Bdel582_100mbar_t2p5ns",
                        plot_title=r"tscan_250917_Bdel582_100mbar_t2p5ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=False,
                        repair_abel=True,
                        xlims_um_R=[200,1300],
                        ylims_um_R=[150,600],
                        xlims_um_G=[500,1300],
                        ylims_um_G=[700,1050],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
tscan_250917_Bdel582_100mbar_t3p0ns_red   = common_path + r"/phase_maps100mbar_Bdel582_t3.0ns/Interferometry2"
tscan_250917_Bdel582_100mbar_t3p0ns_green = common_path + r"/phase_maps100mbar_Bdel582_t3.0ns/Interferometry1"

res_tscan_250917_Bdel582_100mbar_t3p0ns=plot_2d_data(n0_mbar=100,
                        RED_filepath=tscan_250917_Bdel582_100mbar_t3p0ns_red,
                        GREEN_filepath=tscan_250917_Bdel582_100mbar_t3p0ns_green,
                        plot_name=r"tscan_250917_Bdel582_100mbar_t3p0ns",
                        plot_title=r"tscan_250917_Bdel582_100mbar_t3p0ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=True,
                        repair_abel=True,
                        xlims_um_R=[200,1300],
                        ylims_um_R=[150,600],
                        xlims_um_G=[700,900],
                        ylims_um_G=[250,600],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
tscan_250917_Bdel582_100mbar_t3p5ns_red   = common_path + r"/phase_maps100mbar_Bdel582_t3.5ns/Interferometry2"
tscan_250917_Bdel582_100mbar_t3p5ns_green = common_path + r"/phase_maps100mbar_Bdel582_t3.5ns/Interferometry1"

res_tscan_250917_Bdel582_100mbar_t3p5ns=plot_2d_data(n0_mbar=100,
                        RED_filepath=tscan_250917_Bdel582_100mbar_t3p5ns_red,
                        GREEN_filepath=tscan_250917_Bdel582_100mbar_t3p5ns_green,
                        plot_name=r"tscan_250917_Bdel582_100mbar_t3p5ns",
                        plot_title=r"tscan_250917_Bdel582_100mbar_t3p5ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=False,
                        repair_abel=True,
                        xlims_um_R=[200,1300],
                        ylims_um_R=[150,600],
                        xlims_um_G=[700,1300],
                        ylims_um_G=[700,1050],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
tscan_250917_Bdel582_100mbar_t4p0ns_red   = common_path + r"/phase_maps100mbar_Bdel582_t4.0ns/Interferometry2"
tscan_250917_Bdel582_100mbar_t4p0ns_green = common_path + r"/phase_maps100mbar_Bdel582_t4.0ns/Interferometry1"

res_tscan_250917_Bdel582_100mbar_t4p0ns=plot_2d_data(n0_mbar=100,
                        RED_filepath=tscan_250917_Bdel582_100mbar_t4p0ns_red,
                        GREEN_filepath=tscan_250917_Bdel582_100mbar_t4p0ns_green,
                        plot_name=r"tscan_250917_Bdel582_100mbar_t4p0ns",
                        plot_title=r"tscan_250917_Bdel582_100mbar_t4p0ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=False,
                        repair_abel=True,
                        xlims_um_R=[200,1300],
                        ylims_um_R=[150,600],
                        xlims_um_G=[300,1300],
                        ylims_um_G=[700,1050],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.constants as sc  # for sc.k

saveloc_txt = r"/Users/sebastiankalos/Documents/OXFORD/FBPIC_102/HOFI_profiles/CALA_Sept25_channels_txt_files"
os.makedirs(saveloc_txt, exist_ok=True)

# ------------------------------ Data lists ------------------------------
res_250917_100mbar_Bdel582 = [
    res_tscan_250917_Bdel582_100mbar_t0p0ns,
    res_tscan_250917_Bdel582_100mbar_t1p0ns,
    res_tscan_250917_Bdel582_100mbar_t2p0ns,
    res_tscan_250917_Bdel582_100mbar_t2p5ns,
    res_tscan_250917_Bdel582_100mbar_t3p0ns,
    res_tscan_250917_Bdel582_100mbar_t3p5ns,
    res_tscan_250917_Bdel582_100mbar_t4p0ns
]
dicts = res_250917_100mbar_Bdel582
times = [0.0, 1.0, 2.0, 2.5, 3.0, 3.5, 4.0]  # ns
# --- filename meta (minimal changes) ---
DATE_TAG = "250917"       # <-- set this once
PRESSURE_MBAR = 100        # <-- set this once
EXPERIMENT_TAG = "Bdel582"  # optional; set "" if you don't want it

def time_tag(t):
    """e.g. 0.0 -> t0ns, 1.0 -> t1ns, 2.5 -> t2p5ns"""
    s = f"{t:g}"
    return "t" + s.replace(".", "p") + "ns"


# ------------------------------ Figure scaffold ------------------------------
fig, ax = plt.subplot_mosaic(
    [['00','01','02','03','04','05','06'],
     ['10','11','12','13','14','15','16']],
    figsize=(14, 4),
    gridspec_kw={'width_ratios': [1,1,1,1,1,1,1], 'height_ratios':[1,1]}
)
top_plots    = ['00', '01', '02', '03', '04', '05', '06']
bottom_plots = ['10', '11', '12', '13', '14', '15', '16']

xlim       = 140
linestyle  = 'dotted'
linewidth  = 1
line_color = 'black'

# ------------------------------ Physics helpers ------------------------------
T_stand = 293.15  # K
Nb_100mbar = 100*(1013.25*100/1000)/(sc.k*T_stand)*1e-6 * 2  # cm^-3 (100% ionization ref for top row)

# ------------------------------ Convex quartic fit ------------------------------
def _weighted_sums(r2, r4, z, w):
    S22 = np.sum(w * (r2 * r2))    # Σ w r^4
    S24 = np.sum(w * (r2 * r4))    # Σ w r^6
    S44 = np.sum(w * (r4 * r4))    # Σ w r^8
    t2  = np.sum(w * (r2 * z))     # Σ w r^2 z
    t4  = np.sum(w * (r4 * z))     # Σ w r^4 z
    return S22, S24, S44, t2, t4

def fit_quartic_convex_from_annulus(x, y, r_in, r_out, y_sigma=None, ridge=0.0, force_pure_quartic=False):
    """
    Fit convex f(r) = -1 + a r^2 + b r^4 using only |r| in [r_in, r_out].
    Enforces convexity via a>=0, b>=0. If force_pure_quartic=True, set a=0 and fit b>=0 only.
    Returns: a, b, f(x over all x), mask_used, R_eff
    """
    x = np.asarray(x, dtype=float)
    y = np.asarray(y, dtype=float)
    mask = (np.abs(x) >= r_in) & (np.abs(x) <= r_out)
    if mask.sum() < 5:
        return np.nan, np.nan, np.full_like(x, np.nan), mask, np.nan

    xr  = x[mask]
    yr  = y[mask]
    z   = yr + 1.0
    r2  = xr**2
    r4  = r2**2

    if y_sigma is not None:
        w = 1.0 / np.clip(np.asarray(y_sigma)[mask], 1e-12, np.inf)**2
    else:
        w = np.ones_like(xr)

    def sse(a, b):
        res = z - (a*r2 + b*r4)
        return np.sum(w * res**2)

    S22, S24, S44, t2, t4 = _weighted_sums(r2, r4, z, w)

    # Boundary fits (convexity-safe)
    a_lin = max(0.0, (t2) / (S22 + ridge)) if (S22 + ridge) > 0 else 0.0
    b_lin = max(0.0, (t4) / (S44 + ridge)) if (S44 + ridge) > 0 else 0.0
    sse_a_only = sse(a_lin, 0.0)
    sse_b_only = sse(0.0, b_lin)

    if force_pure_quartic:
        a, b = 0.0, b_lin
    else:
        # Unconstrained 2x2
        A = np.array([[S22 + ridge, S24],
                      [S24,         S44 + ridge]], dtype=float)
        bvec = np.array([t2, t4], dtype=float)
        try:
            a0, b0 = np.linalg.solve(A, bvec)
        except np.linalg.LinAlgError:
            a0, b0 = np.nan, np.nan

        if np.isfinite(a0) and np.isfinite(b0) and (a0 >= 0.0) and (b0 >= 0.0):
            a, b = a0, b0
        else:
            a, b = (a_lin, 0.0) if sse_a_only <= sse_b_only else (0.0, b_lin)

    f = -1.0 + a*(x**2) + b*(x**4)

    # Effective radius where f(r)=0 (if exists)
    R_eff = np.nan
    if b > 0:
        disc = a*a + 4.0*b
        if disc >= 0:
            s1 = (-a - np.sqrt(disc)) / (2.0*b)
            s2 = (-a + np.sqrt(disc)) / (2.0*b)
            s_pos = [s for s in (s1, s2) if s > 0]
            if s_pos:
                R_eff = np.sqrt(min(s_pos))
    elif a > 0:
        R_eff = 1.0 / np.sqrt(a)

    return a, b, f, mask, R_eff

# ------------------------------ Per-panel annulus bounds ------------------------------
default_bounds = (45.0, 55.0)  # µm
fit_bounds_by_key = {
    '10': (45.0, 55.0),
    '11': (20.0, 23.0),
    '12': (25.0, 30.0),
    '13': (30.0, 35.0),
    '14': (35.0, 41.0),
    '15': (37.0, 42.0),
    '16': (45.0, 50.0),
}
def bounds_for(key):
    return fit_bounds_by_key.get(key, default_bounds)

# ------------------------------ Plot data (top row) ------------------------------
ne_max = 1.2 * np.max(dicts[0]['ne_r'])

for i, top_key, bot_key, d, t in zip(range(len(dicts)), top_plots, bottom_plots, dicts, times):
    axs0 = ax[top_key]
    if d['xabel'] is not None:
        axs0.plot(d['xabel'], d['ne_r']/1e18, color='tab:orange', lw=1.5)

    axs0.set_xlim(-xlim, xlim)
    axs0.set_ylim(0, 6)
    axs0.axhline(0, color=line_color, linewidth=linewidth, linestyle=linestyle)
    for yline in [2, 4]:
        axs0.axhline(yline, color=line_color, linewidth=linewidth, linestyle=linestyle, alpha=0.1)
    for xv in [-100,-50, 0, 50,100]:
        axs0.axvline(xv, color=line_color, linewidth=linewidth, linestyle=linestyle, alpha=0.1)
    axs0.set_title(f'{t:g} ns', fontsize=14)

# ------------------------------ Axes cosmetics (bottom row too) ------------------------------
for key in top_plots[1:-1]:
    ax[key].set_yticks([]); ax[key].set_xticks([])
ax[top_plots[-1]].yaxis.tick_right()
ax[top_plots[-1]].yaxis.set_label_position("right")

for key in bottom_plots[1:-1]:
    ax[key].set_yticks([])
ax[bottom_plots[-1]].yaxis.tick_right()
ax[bottom_plots[-1]].yaxis.set_label_position("right")

for key in bottom_plots:
    ax[key].set_xlim(-xlim, xlim)
    ax[key].set_ylim(-1.0, 1.9)
    for yline in [0.0, -1.0]:
        ax[key].axhline(yline, color=line_color, linewidth=linewidth, linestyle=linestyle, alpha=0.2)
    for xv in [-100,-50, 0, 50,100]:
        ax[key].axvline(xv, color=line_color, linewidth=linewidth, linestyle=linestyle, alpha=0.1)

ax['00'].axhline(Nb_100mbar/1e18, color='red', ls='dashed')
ax['00'].set_ylabel(r'$n_e$ [10$^{18}$ cm$^{-3}$]', color='tab:orange', size=14)
ax['10'].set_ylabel(r'$\Delta n/n_0$', color='black', size=14)
ax['10'].set_xlabel(r'r [$\mu$m]')
ax['12'].set_xlabel(r'r [$\mu$m]')
ax['14'].set_xlabel(r'r [$\mu$m]')
ax['16'].set_xlabel(r'r [$\mu$m]')

ax['00'].text(x=-90, y=1.05*Nb_100mbar/1e18, s='100% ionization', color='red')

# ------------------------------ Fits & patched display (bottom row) ------------------------------
fit_summaries = []
force_pure_quartic = False  # True -> enforce a=0, b>=0 (extra-wide convex core)

for bot_key, d, t in zip(bottom_plots, dicts, times):
    axs1 = ax[bot_key]

        # --- Special case: 0 ns bottom plot -> save file with neutrals = -ne/ne_full_ion ([-1,0]) and draw nothing ---
    if t == 0.0:
        # x and electrons
        x = np.asarray(d.get('xabel', []), dtype=float)
        if x.size == 0:
            x = np.array([np.nan])
        ne_r = np.asarray(d.get('ne_r', np.full_like(x, np.nan)), dtype=float)
        ne_r_1e18 = ne_r / 1e18

        # full-ion density at this pressure (cm^-3)
        T_stand = 293.15
        ne_full_ion = PRESSURE_MBAR * (1013.25 * 100 / 1000) / (sc.k * T_stand) * 1e-6 * 2

        # neutrals at t=0 as relative change: Δn/n0 = - ne / ne_full_ion  (clip to [-1, 0])
        with np.errstate(divide='ignore', invalid='ignore'):
            dn0_rel_t0 = - ne_r / ne_full_ion
        dn0_rel_t0 = np.clip(dn0_rel_t0, -1.0, 0.0)

        # In the saved file we keep: patched, original, fit. For t=0 use the same dn0_rel_t0 in all 3.
        dn0_patched = dn0_rel_t0.copy()
        y_original  = dn0_rel_t0.copy()
        f_fit       = dn0_rel_t0.copy()

        # one file with r, electrons, and neutrals (patched/original/fit)
        data_out = np.column_stack((x, ne_r, ne_r_1e18, dn0_patched, y_original, f_fit))
        headers = (
            'r [microns];'
            '\tn_e [cm^-3];'
            '\tn_e [1e18 cm^-3];'
            '\tΔn/n0 (patched, quartic fit inside r_out);'
            '\tΔn/n0 (original);'
            '\tΔn/n0 (fit over all r);'
        )

        fname = f"{DATE_TAG}_{PRESSURE_MBAR}mbar"
        if EXPERIMENT_TAG:
            fname += f"_{EXPERIMENT_TAG}"
        fname += f"_{time_tag(t)}.txt"

        np.savetxt(os.path.join(saveloc_txt, fname), data_out, header=headers, comments='')

        # keep plotting behavior: nothing drawn at t=0
        fit_summaries.append((bot_key, np.nan, np.nan, np.nan))
        continue


    if d['xabel'] is None:
        fit_summaries.append((bot_key, np.nan, np.nan, np.nan))
        continue

    x = np.asarray(d['xabel'])
    y = np.asarray(d['dn0_r'])
    y_sigma = d.get('Delta_dn0_r', None)

    r_in, r_out = bounds_for(bot_key)

    a, b, f, mask_used, R_eff = fit_quartic_convex_from_annulus(
        x, y, r_in, r_out, y_sigma=y_sigma, ridge=0.0, force_pure_quartic=force_pure_quartic
    )
    fit_summaries.append((bot_key, a, b, R_eff))

    # ---------- Patch logic with small overlap (no gap/bridge) ----------
    if len(x) > 1:
        dx_med = np.median(np.diff(np.sort(x)))
        eps = 0.5 * dx_med if np.isfinite(dx_med) and dx_med > 0 else 0.5
    else:
        eps = 0.5

    mask_inside  = (np.abs(x) <= (r_out + eps))   # dotted region (a hair wider)
    # Solid outside measured data
    y_outside = y.copy()
    y_outside[mask_inside] = np.nan
    axs1.plot(x, y_outside, color='black', lw=1.2, ls='-',label='data')

    # Dotted fitted curve inside (replacement, slight overlap)
    f_inside = np.full_like(f, np.nan, dtype=float)
    f_inside[mask_inside] = f[mask_inside]
    axs1.plot(x, f_inside, color='black', lw=1.2, ls='dotted',label='fit')

    # ---------- Vertical dotted seam at x = ±r_out ----------
    # Take y-values from the nearest samples to ±r_out
    def nearest_idx(val):
        return int(np.nanargmin(np.abs(x - val)))

    for x_seam in (r_out, -r_out):
        i_seam = nearest_idx(x_seam)
        # Measured (outside) value and fitted (inside) value at the seam
        y_meas = y[i_seam]
        y_fit  = f[i_seam]
        # Only draw if both are finite
        if np.isfinite(y_meas) and np.isfinite(y_fit):
            y0, y1 = (y_meas, y_fit) if y_meas <= y_fit else (y_fit, y_meas)
            axs1.plot([x_seam, x_seam], [y0, y1], color='black', lw=1.2, ls='dotted')

    # ---------- Build "patched" neutrals (replace |r| <= r_out with fit) ----------
    mask_patch   = (np.abs(x) <= r_out)
    dn0_patched  = y.copy()
    dn0_patched[mask_patch] = f[mask_patch]

    # ---------- Electrons ----------
    ne_r      = np.asarray(d['ne_r'])       # [cm^-3]
    ne_r_1e18 = ne_r / 1e18                 # [10^18 cm^-3] convenience column

    # ---------- Combine into one table ----------
    # Columns: r, electrons (cm^-3), electrons (1e18 cm^-3),
    #          neutrals patched, neutrals original, neutrals fit
    data_out = np.column_stack((x, ne_r, ne_r_1e18, dn0_patched, y, f))
    headers = (
        'r [microns];'
        '\tn_e [cm^-3];'
        '\tn_e [1e18 cm^-3];'
        '\tΔn/n0 (patched, quartic fit inside r_out);'
        '\tΔn/n0 (original);'
        '\tΔn/n0 (fit over all r);'
    )

    # ---------- Save (date + pressure + tag + time) ----------
    fname = f"{DATE_TAG}_{PRESSURE_MBAR}mbar"
    if EXPERIMENT_TAG:
        fname += f"_{EXPERIMENT_TAG}"
    fname += f"_{time_tag(t)}.txt"

    np.savetxt(os.path.join(saveloc_txt, fname), data_out,
            header=headers, comments='')


ax['12'].legend()
ax['01'].text(s='electrons',x=-80,y=2.4,color='tab:orange',size=14)
ax['11'].text(s='neutrals',x=-80,y=0.8,color='black',size=14)
ax['10'].text(s='100 mbar\n ambient\npressure',x=-80,y=0.3,color='black',size=14)
ax['00'].yaxis.set_label_position("left")
plt.subplots_adjust(wspace=0, hspace=0)



# 80 mbar, BM 582:

In [ ]:
tscan_250917_Bdel582_80mbar_t0p0ns_red   = common_path_E + r"/phase_maps/80mbar_Bdel582_t0/Interferometry2"
tscan_250917_Bdel582_80mbar_t0p0ns_green = common_path_E + r"/phase_maps/80mbar_Bdel582_t0/Interferometry1"

res_tscan_250917_Bdel582_80mbar_t0p0ns=plot_2d_data(n0_mbar=80,
                        RED_filepath=tscan_250917_Bdel582_80mbar_t0p0ns_red,
                        GREEN_filepath=tscan_250917_Bdel582_80mbar_t0p0ns_green,
                        plot_name=r"tscan_250917_Bdel582_80mbar_t0p0ns",
                        plot_title=r"tscan_250917_Bdel582_80mbar_t0p0ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=True,
                        repair_abel=True,
                        xlims_um_R=[800,1200],
                        ylims_um_R=[150,600],
                        xlims_um_G=[900,1300],
                        ylims_um_G=[300,650],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
tscan_250917_Bdel582_80mbar_t1p0ns_red   = common_path + r"/phase_maps80mbar_Bdel582_t1.0ns/Interferometry2"
tscan_250917_Bdel582_80mbar_t1p0ns_green = common_path + r"/phase_maps80mbar_Bdel582_t1.0ns/Interferometry1"

res_tscan_250917_Bdel582_80mbar_t1p0ns=plot_2d_data(n0_mbar=80,
                        RED_filepath=tscan_250917_Bdel582_80mbar_t1p0ns_red,
                        GREEN_filepath=tscan_250917_Bdel582_80mbar_t1p0ns_green,
                        plot_name=r"tscan_250917_Bdel582_80mbar_t1p0ns",
                        plot_title=r"tscan_250917_Bdel582_80mbar_t1p0ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=False,
                        repair_abel=True,
                        xlims_um_R=[200,1300],
                        ylims_um_R=[150,600],
                        xlims_um_G=[200,1300],
                        ylims_um_G=[700,1050],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
tscan_250917_Bdel582_80mbar_t2p0ns_red   = common_path + r"/phase_maps80mbar_Bdel582_t2.0ns/Interferometry2"
tscan_250917_Bdel582_80mbar_t2p0ns_green = common_path + r"/phase_maps80mbar_Bdel582_t2.0ns/Interferometry1"

res_tscan_250917_Bdel582_80mbar_t2p0ns=plot_2d_data(n0_mbar=80,
                        RED_filepath=tscan_250917_Bdel582_80mbar_t2p0ns_red,
                        GREEN_filepath=tscan_250917_Bdel582_80mbar_t2p0ns_green,
                        plot_name=r"tscan_250917_Bdel582_80mbar_t2p0ns",
                        plot_title=r"tscan_250917_Bdel582_80mbar_t2p0ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=False,
                        repair_abel=True,
                        xlims_um_R=[200,1300],
                        ylims_um_R=[150,600],
                        xlims_um_G=[200,1300],
                        ylims_um_G=[700,1050],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
tscan_250917_Bdel582_80mbar_t2p5ns_red   = common_path + r"/phase_maps80mbar_Bdel582_t2.5ns/Interferometry2"
tscan_250917_Bdel582_80mbar_t2p5ns_green = common_path + r"/phase_maps80mbar_Bdel582_t2.5ns/Interferometry1"

res_tscan_250917_Bdel582_80mbar_t2p5ns=plot_2d_data(n0_mbar=80,
                        RED_filepath=tscan_250917_Bdel582_80mbar_t2p5ns_red,
                        GREEN_filepath=tscan_250917_Bdel582_80mbar_t2p5ns_green,
                        plot_name=r"tscan_250917_Bdel582_80mbar_t2p5ns",
                        plot_title=r"tscan_250917_Bdel582_80mbar_t2p5ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=False,
                        repair_abel=True,
                        xlims_um_R=[200,1300],
                        ylims_um_R=[150,600],
                        xlims_um_G=[600,900],
                        ylims_um_G=[700,1050],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
tscan_250917_Bdel582_80mbar_t3p0ns_red   = common_path + r"/phase_maps80mbar_Bdel582_t3.0ns/Interferometry2"
tscan_250917_Bdel582_80mbar_t3p0ns_green = common_path + r"/phase_maps80mbar_Bdel582_t3.0ns/Interferometry1"

res_tscan_250917_Bdel582_80mbar_t3p0ns=plot_2d_data(n0_mbar=80,
                        RED_filepath=tscan_250917_Bdel582_80mbar_t3p0ns_red,
                        GREEN_filepath=tscan_250917_Bdel582_80mbar_t3p0ns_green,
                        plot_name=r"tscan_250917_Bdel582_80mbar_t3p0ns",
                        plot_title=r"tscan_250917_Bdel582_80mbar_t3p0ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=False,
                        repair_abel=True,
                        xlims_um_R=[200,1300],
                        ylims_um_R=[150,600],
                        xlims_um_G=[200,1300],
                        ylims_um_G=[700,1050],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
tscan_250917_Bdel582_80mbar_t3p5ns_red   = common_path + r"/phase_maps80mbar_Bdel582_t3.5ns/Interferometry2"
tscan_250917_Bdel582_80mbar_t3p5ns_green = common_path + r"/phase_maps80mbar_Bdel582_t3.5ns/Interferometry1"

res_tscan_250917_Bdel582_80mbar_t3p5ns=plot_2d_data(n0_mbar=80,
                        RED_filepath=tscan_250917_Bdel582_80mbar_t3p5ns_red,
                        GREEN_filepath=tscan_250917_Bdel582_80mbar_t3p5ns_green,
                        plot_name=r"tscan_250917_Bdel582_80mbar_t3p5ns",
                        plot_title=r"tscan_250917_Bdel582_80mbar_t3p5ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=False,
                        repair_abel=True,
                        xlims_um_R=[200,1300],
                        ylims_um_R=[150,600],
                        xlims_um_G=[500,1300],
                        ylims_um_G=[700,1050],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
tscan_250917_Bdel582_80mbar_t4p0ns_red   = common_path + r"/phase_maps80mbar_Bdel582_t4.0ns/Interferometry2"
tscan_250917_Bdel582_80mbar_t4p0ns_green = common_path + r"/phase_maps80mbar_Bdel582_t4.0ns/Interferometry1"

res_tscan_250917_Bdel582_80mbar_t4p0ns=plot_2d_data(n0_mbar=80,
                        RED_filepath=tscan_250917_Bdel582_80mbar_t4p0ns_red,
                        GREEN_filepath=tscan_250917_Bdel582_80mbar_t4p0ns_green,
                        plot_name=r"tscan_250917_Bdel582_80mbar_t4p0ns",
                        plot_title=r"tscan_250917_Bdel582_80mbar_t4p0ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=False,
                        repair_abel=True,
                        xlims_um_R=[200,1300],
                        ylims_um_R=[150,600],
                        xlims_um_G=[600,1300],
                        ylims_um_G=[700,1050],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.constants as sc  # for sc.k

saveloc_txt = r"/Users/sebastiankalos/Documents/OXFORD/FBPIC_102/HOFI_profiles/CALA_Sept25_channels_txt_files"
os.makedirs(saveloc_txt, exist_ok=True)

# ------------------------------ Data lists ------------------------------
res_250917_80mbar_Bdel582 = [
    res_tscan_250917_Bdel582_80mbar_t0p0ns,
    res_tscan_250917_Bdel582_80mbar_t1p0ns,
    res_tscan_250917_Bdel582_80mbar_t2p0ns,
    res_tscan_250917_Bdel582_80mbar_t2p5ns,
    res_tscan_250917_Bdel582_80mbar_t3p0ns,
    res_tscan_250917_Bdel582_80mbar_t3p5ns,
    res_tscan_250917_Bdel582_80mbar_t4p0ns
]
dicts = res_250917_80mbar_Bdel582
times = [0.0, 1.0, 2.0, 2.5, 3.0, 3.5, 4.0]  # ns
# --- filename meta (minimal changes) ---
DATE_TAG = "250917"       # <-- set this once
PRESSURE_MBAR = 80        # <-- set this once
EXPERIMENT_TAG = "Bdel582"  # optional; set "" if you don't want it

def time_tag(t):
    """e.g. 0.0 -> t0ns, 1.0 -> t1ns, 2.5 -> t2p5ns"""
    s = f"{t:g}"
    return "t" + s.replace(".", "p") + "ns"


# ------------------------------ Figure scaffold ------------------------------
fig, ax = plt.subplot_mosaic(
    [['00','01','02','03','04','05','06'],
     ['10','11','12','13','14','15','16']],
    figsize=(14, 4),
    gridspec_kw={'width_ratios': [1,1,1,1,1,1,1], 'height_ratios':[1,1]}
)
top_plots    = ['00', '01', '02', '03', '04', '05', '06']
bottom_plots = ['10', '11', '12', '13', '14', '15', '16']

xlim       = 140
linestyle  = 'dotted'
linewidth  = 1
line_color = 'black'

# ------------------------------ Physics helpers ------------------------------
T_stand = 293.15  # K
Nb_80mbar = 80*(1013.25*100/1000)/(sc.k*T_stand)*1e-6 * 2  # cm^-3 (100% ionization ref for top row)

# ------------------------------ Convex quartic fit ------------------------------
def _weighted_sums(r2, r4, z, w):
    S22 = np.sum(w * (r2 * r2))    # Σ w r^4
    S24 = np.sum(w * (r2 * r4))    # Σ w r^6
    S44 = np.sum(w * (r4 * r4))    # Σ w r^8
    t2  = np.sum(w * (r2 * z))     # Σ w r^2 z
    t4  = np.sum(w * (r4 * z))     # Σ w r^4 z
    return S22, S24, S44, t2, t4

def fit_quartic_convex_from_annulus(x, y, r_in, r_out, y_sigma=None, ridge=0.0, force_pure_quartic=False):
    """
    Fit convex f(r) = -1 + a r^2 + b r^4 using only |r| in [r_in, r_out].
    Enforces convexity via a>=0, b>=0. If force_pure_quartic=True, set a=0 and fit b>=0 only.
    Returns: a, b, f(x over all x), mask_used, R_eff
    """
    x = np.asarray(x, dtype=float)
    y = np.asarray(y, dtype=float)
    mask = (np.abs(x) >= r_in) & (np.abs(x) <= r_out)
    if mask.sum() < 5:
        return np.nan, np.nan, np.full_like(x, np.nan), mask, np.nan

    xr  = x[mask]
    yr  = y[mask]
    z   = yr + 1.0
    r2  = xr**2
    r4  = r2**2

    if y_sigma is not None:
        w = 1.0 / np.clip(np.asarray(y_sigma)[mask], 1e-12, np.inf)**2
    else:
        w = np.ones_like(xr)

    def sse(a, b):
        res = z - (a*r2 + b*r4)
        return np.sum(w * res**2)

    S22, S24, S44, t2, t4 = _weighted_sums(r2, r4, z, w)

    # Boundary fits (convexity-safe)
    a_lin = max(0.0, (t2) / (S22 + ridge)) if (S22 + ridge) > 0 else 0.0
    b_lin = max(0.0, (t4) / (S44 + ridge)) if (S44 + ridge) > 0 else 0.0
    sse_a_only = sse(a_lin, 0.0)
    sse_b_only = sse(0.0, b_lin)

    if force_pure_quartic:
        a, b = 0.0, b_lin
    else:
        # Unconstrained 2x2
        A = np.array([[S22 + ridge, S24],
                      [S24,         S44 + ridge]], dtype=float)
        bvec = np.array([t2, t4], dtype=float)
        try:
            a0, b0 = np.linalg.solve(A, bvec)
        except np.linalg.LinAlgError:
            a0, b0 = np.nan, np.nan

        if np.isfinite(a0) and np.isfinite(b0) and (a0 >= 0.0) and (b0 >= 0.0):
            a, b = a0, b0
        else:
            a, b = (a_lin, 0.0) if sse_a_only <= sse_b_only else (0.0, b_lin)

    f = -1.0 + a*(x**2) + b*(x**4)

    # Effective radius where f(r)=0 (if exists)
    R_eff = np.nan
    if b > 0:
        disc = a*a + 4.0*b
        if disc >= 0:
            s1 = (-a - np.sqrt(disc)) / (2.0*b)
            s2 = (-a + np.sqrt(disc)) / (2.0*b)
            s_pos = [s for s in (s1, s2) if s > 0]
            if s_pos:
                R_eff = np.sqrt(min(s_pos))
    elif a > 0:
        R_eff = 1.0 / np.sqrt(a)

    return a, b, f, mask, R_eff

# ------------------------------ Per-panel annulus bounds ------------------------------
default_bounds = (45.0, 55.0)  # µm
fit_bounds_by_key = {
    '10': (45.0, 55.0),
    '11': (20.0, 23.0),
    '12': (25.0, 30.0),
    '13': (30.0, 35.0),
    '14': (35.0, 41.0),
    '15': (37.0, 42.0),
    '16': (45.0, 50.0),
}
def bounds_for(key):
    return fit_bounds_by_key.get(key, default_bounds)

# ------------------------------ Plot data (top row) ------------------------------
ne_max = 1.2 * np.max(dicts[0]['ne_r'])

for i, top_key, bot_key, d, t in zip(range(len(dicts)), top_plots, bottom_plots, dicts, times):
    axs0 = ax[top_key]
    if d['xabel'] is not None:
        axs0.plot(d['xabel'], d['ne_r']/1e18, color='tab:orange', lw=1.5)

    axs0.set_xlim(-xlim, xlim)
    axs0.set_ylim(0, 6)
    axs0.axhline(0, color=line_color, linewidth=linewidth, linestyle=linestyle)
    for yline in [2, 4]:
        axs0.axhline(yline, color=line_color, linewidth=linewidth, linestyle=linestyle, alpha=0.1)
    for xv in [-100,-50, 0, 50,100]:
        axs0.axvline(xv, color=line_color, linewidth=linewidth, linestyle=linestyle, alpha=0.1)
    axs0.set_title(f'{t:g} ns', fontsize=14)

# ------------------------------ Axes cosmetics (bottom row too) ------------------------------
for key in top_plots[1:-1]:
    ax[key].set_yticks([]); ax[key].set_xticks([])
ax[top_plots[-1]].yaxis.tick_right()
ax[top_plots[-1]].yaxis.set_label_position("right")

for key in bottom_plots[1:-1]:
    ax[key].set_yticks([])
ax[bottom_plots[-1]].yaxis.tick_right()
ax[bottom_plots[-1]].yaxis.set_label_position("right")

for key in bottom_plots:
    ax[key].set_xlim(-xlim, xlim)
    ax[key].set_ylim(-1.0, 1.9)
    for yline in [0.0, -1.0]:
        ax[key].axhline(yline, color=line_color, linewidth=linewidth, linestyle=linestyle, alpha=0.2)
    for xv in [-100,-50, 0, 50,100]:
        ax[key].axvline(xv, color=line_color, linewidth=linewidth, linestyle=linestyle, alpha=0.1)

ax['00'].axhline(Nb_80mbar/1e18, color='red', ls='dashed')
ax['00'].set_ylabel(r'$n_e$ [10$^{18}$ cm$^{-3}$]', color='tab:orange', size=14)
ax['10'].set_ylabel(r'$\Delta n/n_0$', color='black', size=14)
ax['10'].set_xlabel(r'r [$\mu$m]')
ax['12'].set_xlabel(r'r [$\mu$m]')
ax['14'].set_xlabel(r'r [$\mu$m]')
ax['16'].set_xlabel(r'r [$\mu$m]')

ax['00'].text(x=-90, y=1.05*Nb_80mbar/1e18, s='100% ionization', color='red')

# ------------------------------ Fits & patched display (bottom row) ------------------------------
fit_summaries = []
force_pure_quartic = False  # True -> enforce a=0, b>=0 (extra-wide convex core)

for bot_key, d, t in zip(bottom_plots, dicts, times):
    axs1 = ax[bot_key]

        # --- Special case: 0 ns bottom plot -> save file with neutrals = -ne/ne_full_ion ([-1,0]) and draw nothing ---
    if t == 0.0:
        # x and electrons
        x = np.asarray(d.get('xabel', []), dtype=float)
        if x.size == 0:
            x = np.array([np.nan])
        ne_r = np.asarray(d.get('ne_r', np.full_like(x, np.nan)), dtype=float)
        ne_r_1e18 = ne_r / 1e18

        # full-ion density at this pressure (cm^-3)
        T_stand = 293.15
        ne_full_ion = PRESSURE_MBAR * (1013.25 * 100 / 1000) / (sc.k * T_stand) * 1e-6 * 2

        # neutrals at t=0 as relative change: Δn/n0 = - ne / ne_full_ion  (clip to [-1, 0])
        with np.errstate(divide='ignore', invalid='ignore'):
            dn0_rel_t0 = - ne_r / ne_full_ion
        dn0_rel_t0 = np.clip(dn0_rel_t0, -1.0, 0.0)

        # In the saved file we keep: patched, original, fit. For t=0 use the same dn0_rel_t0 in all 3.
        dn0_patched = dn0_rel_t0.copy()
        y_original  = dn0_rel_t0.copy()
        f_fit       = dn0_rel_t0.copy()

        # one file with r, electrons, and neutrals (patched/original/fit)
        data_out = np.column_stack((x, ne_r, ne_r_1e18, dn0_patched, y_original, f_fit))
        headers = (
            'r [microns];'
            '\tn_e [cm^-3];'
            '\tn_e [1e18 cm^-3];'
            '\tΔn/n0 (patched, quartic fit inside r_out);'
            '\tΔn/n0 (original);'
            '\tΔn/n0 (fit over all r);'
        )

        fname = f"{DATE_TAG}_{PRESSURE_MBAR}mbar"
        if EXPERIMENT_TAG:
            fname += f"_{EXPERIMENT_TAG}"
        fname += f"_{time_tag(t)}.txt"

        np.savetxt(os.path.join(saveloc_txt, fname), data_out, header=headers, comments='')

        # keep plotting behavior: nothing drawn at t=0
        fit_summaries.append((bot_key, np.nan, np.nan, np.nan))
        continue


    if d['xabel'] is None:
        fit_summaries.append((bot_key, np.nan, np.nan, np.nan))
        continue

    x = np.asarray(d['xabel'])
    y = np.asarray(d['dn0_r'])
    y_sigma = d.get('Delta_dn0_r', None)

    r_in, r_out = bounds_for(bot_key)

    a, b, f, mask_used, R_eff = fit_quartic_convex_from_annulus(
        x, y, r_in, r_out, y_sigma=y_sigma, ridge=0.0, force_pure_quartic=force_pure_quartic
    )
    fit_summaries.append((bot_key, a, b, R_eff))

    # ---------- Patch logic with small overlap (no gap/bridge) ----------
    if len(x) > 1:
        dx_med = np.median(np.diff(np.sort(x)))
        eps = 0.5 * dx_med if np.isfinite(dx_med) and dx_med > 0 else 0.5
    else:
        eps = 0.5

    mask_inside  = (np.abs(x) <= (r_out + eps))   # dotted region (a hair wider)
    # Solid outside measured data
    y_outside = y.copy()
    y_outside[mask_inside] = np.nan
    axs1.plot(x, y_outside, color='black', lw=1.2, ls='-',label='data')

    # Dotted fitted curve inside (replacement, slight overlap)
    f_inside = np.full_like(f, np.nan, dtype=float)
    f_inside[mask_inside] = f[mask_inside]
    axs1.plot(x, f_inside, color='black', lw=1.2, ls='dotted',label='fit')

    # ---------- Vertical dotted seam at x = ±r_out ----------
    # Take y-values from the nearest samples to ±r_out
    def nearest_idx(val):
        return int(np.nanargmin(np.abs(x - val)))

    for x_seam in (r_out, -r_out):
        i_seam = nearest_idx(x_seam)
        # Measured (outside) value and fitted (inside) value at the seam
        y_meas = y[i_seam]
        y_fit  = f[i_seam]
        # Only draw if both are finite
        if np.isfinite(y_meas) and np.isfinite(y_fit):
            y0, y1 = (y_meas, y_fit) if y_meas <= y_fit else (y_fit, y_meas)
            axs1.plot([x_seam, x_seam], [y0, y1], color='black', lw=1.2, ls='dotted')

    # ---------- Build "patched" neutrals (replace |r| <= r_out with fit) ----------
    mask_patch   = (np.abs(x) <= r_out)
    dn0_patched  = y.copy()
    dn0_patched[mask_patch] = f[mask_patch]

    # ---------- Electrons ----------
    ne_r      = np.asarray(d['ne_r'])       # [cm^-3]
    ne_r_1e18 = ne_r / 1e18                 # [10^18 cm^-3] convenience column

    # ---------- Combine into one table ----------
    # Columns: r, electrons (cm^-3), electrons (1e18 cm^-3),
    #          neutrals patched, neutrals original, neutrals fit
    data_out = np.column_stack((x, ne_r, ne_r_1e18, dn0_patched, y, f))
    headers = (
        'r [microns];'
        '\tn_e [cm^-3];'
        '\tn_e [1e18 cm^-3];'
        '\tΔn/n0 (patched, quartic fit inside r_out);'
        '\tΔn/n0 (original);'
        '\tΔn/n0 (fit over all r);'
    )

    # ---------- Save (date + pressure + tag + time) ----------
    fname = f"{DATE_TAG}_{PRESSURE_MBAR}mbar"
    if EXPERIMENT_TAG:
        fname += f"_{EXPERIMENT_TAG}"
    fname += f"_{time_tag(t)}.txt"

    np.savetxt(os.path.join(saveloc_txt, fname), data_out,
            header=headers, comments='')


ax['12'].legend()
ax['01'].text(s='electrons',x=-80,y=2.4,color='tab:orange',size=14)
ax['11'].text(s='neutrals',x=-80,y=0.8,color='black',size=14)
ax['10'].text(s='80 mbar\n ambient\npressure',x=-80,y=0.3,color='black',size=14)
ax['00'].yaxis.set_label_position("left")
plt.subplots_adjust(wspace=0, hspace=0)



# 80 bar, BM 560:

In [ ]:
tscan_250917_Bdel560_80mbar_t0p0ns_red   = common_path_E + r"/phase_maps/80mbar_Bdel560_t0/Interferometry2"
tscan_250917_Bdel560_80mbar_t0p0ns_green = common_path_E + r"/phase_maps/80mbar_Bdel560_t0/Interferometry1"

res_tscan_250917_Bdel560_80mbar_t0p0ns=plot_2d_data(n0_mbar=80,
                        RED_filepath=tscan_250917_Bdel560_80mbar_t0p0ns_red,
                        GREEN_filepath=tscan_250917_Bdel560_80mbar_t0p0ns_green,
                        plot_name=r"tscan_250917_Bdel560_80mbar_t0p0ns",
                        plot_title=r"tscan_250917_Bdel560_80mbar_t0p0ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=True,
                        repair_abel=True,
                        xlims_um_R=[900,1300],
                        ylims_um_R=[150,550],
                        xlims_um_G=[300,1300],
                        ylims_um_G=[300,650],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
tscan_250917_Bdel560_80mbar_t1p0ns_red   = common_path + r"/phase_maps80mbar_Bdel560_t1.0ns/Interferometry2"
tscan_250917_Bdel560_80mbar_t1p0ns_green = common_path + r"/phase_maps80mbar_Bdel560_t1.0ns/Interferometry1"

res_tscan_250917_Bdel560_80mbar_t1p0ns=plot_2d_data(n0_mbar=80,
                        RED_filepath=tscan_250917_Bdel560_80mbar_t1p0ns_red,
                        GREEN_filepath=tscan_250917_Bdel560_80mbar_t1p0ns_green,
                        plot_name=r"tscan_250917_Bdel560_80mbar_t1p0ns",
                        plot_title=r"tscan_250917_Bdel560_80mbar_t1p0ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=False,
                        repair_abel=True,
                        xlims_um_R=[200,1300],
                        ylims_um_R=[150,600],
                        xlims_um_G=[200,1300],
                        ylims_um_G=[700,1050],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
tscan_250917_Bdel560_80mbar_t2p0ns_red   = common_path + r"/phase_maps80mbar_Bdel560_t2.0ns/Interferometry2"
tscan_250917_Bdel560_80mbar_t2p0ns_green = common_path + r"/phase_maps80mbar_Bdel560_t2.0ns/Interferometry1"

res_tscan_250917_Bdel560_80mbar_t2p0ns=plot_2d_data(n0_mbar=80,
                        RED_filepath=tscan_250917_Bdel560_80mbar_t2p0ns_red,
                        GREEN_filepath=tscan_250917_Bdel560_80mbar_t2p0ns_green,
                        plot_name=r"tscan_250917_Bdel560_80mbar_t2p0ns",
                        plot_title=r"tscan_250917_Bdel560_80mbar_t2p0ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=False,
                        repair_abel=True,
                        xlims_um_R=[200,1300],
                        ylims_um_R=[150,600],
                        xlims_um_G=[200,1300],
                        ylims_um_G=[700,1050],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
tscan_250917_Bdel560_80mbar_t2p5ns_red   = common_path + r"/phase_maps80mbar_Bdel560_t2.5ns/Interferometry2"
tscan_250917_Bdel560_80mbar_t2p5ns_green = common_path + r"/phase_maps80mbar_Bdel560_t2.5ns/Interferometry1"

res_tscan_250917_Bdel560_80mbar_t2p5ns=plot_2d_data(n0_mbar=80,
                        RED_filepath=tscan_250917_Bdel560_80mbar_t2p5ns_red,
                        GREEN_filepath=tscan_250917_Bdel560_80mbar_t2p5ns_green,
                        plot_name=r"tscan_250917_Bdel560_80mbar_t2p5ns",
                        plot_title=r"tscan_250917_Bdel560_80mbar_t2p5ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=False,
                        repair_abel=True,
                        xlims_um_R=[200,1300],
                        ylims_um_R=[150,600],
                        xlims_um_G=[700,1300],
                        ylims_um_G=[700,1050],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
tscan_250917_Bdel560_80mbar_t3p0ns_red   = common_path + r"/phase_maps80mbar_Bdel560_t3.0ns/Interferometry2"
tscan_250917_Bdel560_80mbar_t3p0ns_green = common_path + r"/phase_maps80mbar_Bdel560_t3.0ns/Interferometry1"

res_tscan_250917_Bdel560_80mbar_t3p0ns=plot_2d_data(n0_mbar=80,
                        RED_filepath=tscan_250917_Bdel560_80mbar_t3p0ns_red,
                        GREEN_filepath=tscan_250917_Bdel560_80mbar_t3p0ns_green,
                        plot_name=r"tscan_250917_Bdel560_80mbar_t3p0ns",
                        plot_title=r"tscan_250917_Bdel560_80mbar_t3p0ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=False,
                        repair_abel=True,
                        xlims_um_R=[200,1300],
                        ylims_um_R=[150,600],
                        xlims_um_G=[700,1300],
                        ylims_um_G=[700,1050],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
tscan_250917_Bdel560_80mbar_t3p5ns_red   = common_path + r"/phase_maps80mbar_Bdel560_t3.5ns/Interferometry2"
tscan_250917_Bdel560_80mbar_t3p5ns_green = common_path + r"/phase_maps80mbar_Bdel560_t3.5ns/Interferometry1"

res_tscan_250917_Bdel560_80mbar_t3p5ns=plot_2d_data(n0_mbar=80,
                        RED_filepath=tscan_250917_Bdel560_80mbar_t3p5ns_red,
                        GREEN_filepath=tscan_250917_Bdel560_80mbar_t3p5ns_green,
                        plot_name=r"tscan_250917_Bdel560_80mbar_t3p5ns",
                        plot_title=r"tscan_250917_Bdel560_80mbar_t3p5ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=False,
                        repair_abel=True,
                        xlims_um_R=[200,1300],
                        ylims_um_R=[150,600],
                        xlims_um_G=[700,1300],
                        ylims_um_G=[700,1050],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
tscan_250917_Bdel560_80mbar_t4p0ns_red   = common_path + r"/phase_maps80mbar_Bdel560_t4.0ns/Interferometry2"
tscan_250917_Bdel560_80mbar_t4p0ns_green = common_path + r"/phase_maps80mbar_Bdel560_t4.0ns/Interferometry1"

res_tscan_250917_Bdel560_80mbar_t4p0ns=plot_2d_data(n0_mbar=80,
                        RED_filepath=tscan_250917_Bdel560_80mbar_t4p0ns_red,
                        GREEN_filepath=tscan_250917_Bdel560_80mbar_t4p0ns_green,
                        plot_name=r"tscan_250917_Bdel560_80mbar_t4p0ns",
                        plot_title=r"tscan_250917_Bdel560_80mbar_t4p0ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=False,
                        repair_abel=True,
                        xlims_um_R=[200,1300],
                        ylims_um_R=[150,600],
                        xlims_um_G=[700,1300],
                        ylims_um_G=[700,1050],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.constants as sc  # for sc.k

# ------------------------------ Data lists ------------------------------
res_250917_80mbar_Bdel560 = [
    res_tscan_250917_Bdel560_80mbar_t0p0ns,
    res_tscan_250917_Bdel560_80mbar_t1p0ns,
    res_tscan_250917_Bdel560_80mbar_t2p0ns,
    res_tscan_250917_Bdel560_80mbar_t2p5ns,
    res_tscan_250917_Bdel560_80mbar_t3p0ns,
    res_tscan_250917_Bdel560_80mbar_t3p5ns,
    res_tscan_250917_Bdel560_80mbar_t4p0ns
]
dicts = res_250917_80mbar_Bdel560
times = [0.0, 1.0, 2.0, 2.5, 3.0, 3.5, 4.0]  # ns

# ------------------------------ Figure scaffold ------------------------------
fig, ax = plt.subplot_mosaic(
    [['00','01','02','03','04','05','06'],
     ['10','11','12','13','14','15','16']],
    figsize=(14, 4),
    gridspec_kw={'width_ratios': [1,1,1,1,1,1,1], 'height_ratios':[1,1]}
)
top_plots    = ['00', '01', '02', '03', '04', '05', '06']
bottom_plots = ['10', '11', '12', '13', '14', '15', '16']

xlim       = 140
linestyle  = 'dotted'
linewidth  = 1
line_color = 'black'

# ------------------------------ Physics helpers ------------------------------
T_stand = 293.15  # K
Nb_80mbar = 80*(1013.25*100/1000)/(sc.k*T_stand)*1e-6 * 2  # cm^-3 (100% ionization ref for top row)

# ------------------------------ Convex quartic fit ------------------------------
def _weighted_sums(r2, r4, z, w):
    S22 = np.sum(w * (r2 * r2))    # Σ w r^4
    S24 = np.sum(w * (r2 * r4))    # Σ w r^6
    S44 = np.sum(w * (r4 * r4))    # Σ w r^8
    t2  = np.sum(w * (r2 * z))     # Σ w r^2 z
    t4  = np.sum(w * (r4 * z))     # Σ w r^4 z
    return S22, S24, S44, t2, t4

def fit_quartic_convex_from_annulus(x, y, r_in, r_out, y_sigma=None, ridge=0.0, force_pure_quartic=False):
    """
    Fit convex f(r) = -1 + a r^2 + b r^4 using only |r| in [r_in, r_out].
    Enforces convexity via a>=0, b>=0. If force_pure_quartic=True, set a=0 and fit b>=0 only.
    Returns: a, b, f(x over all x), mask_used, R_eff
    """
    x = np.asarray(x, dtype=float)
    y = np.asarray(y, dtype=float)
    mask = (np.abs(x) >= r_in) & (np.abs(x) <= r_out)
    if mask.sum() < 5:
        return np.nan, np.nan, np.full_like(x, np.nan), mask, np.nan

    xr  = x[mask]
    yr  = y[mask]
    z   = yr + 1.0
    r2  = xr**2
    r4  = r2**2

    if y_sigma is not None:
        w = 1.0 / np.clip(np.asarray(y_sigma)[mask], 1e-12, np.inf)**2
    else:
        w = np.ones_like(xr)

    def sse(a, b):
        res = z - (a*r2 + b*r4)
        return np.sum(w * res**2)

    S22, S24, S44, t2, t4 = _weighted_sums(r2, r4, z, w)

    # Boundary fits (convexity-safe)
    a_lin = max(0.0, (t2) / (S22 + ridge)) if (S22 + ridge) > 0 else 0.0
    b_lin = max(0.0, (t4) / (S44 + ridge)) if (S44 + ridge) > 0 else 0.0
    sse_a_only = sse(a_lin, 0.0)
    sse_b_only = sse(0.0, b_lin)

    if force_pure_quartic:
        a, b = 0.0, b_lin
    else:
        # Unconstrained 2x2
        A = np.array([[S22 + ridge, S24],
                      [S24,         S44 + ridge]], dtype=float)
        bvec = np.array([t2, t4], dtype=float)
        try:
            a0, b0 = np.linalg.solve(A, bvec)
        except np.linalg.LinAlgError:
            a0, b0 = np.nan, np.nan

        if np.isfinite(a0) and np.isfinite(b0) and (a0 >= 0.0) and (b0 >= 0.0):
            a, b = a0, b0
        else:
            a, b = (a_lin, 0.0) if sse_a_only <= sse_b_only else (0.0, b_lin)

    f = -1.0 + a*(x**2) + b*(x**4)

    # Effective radius where f(r)=0 (if exists)
    R_eff = np.nan
    if b > 0:
        disc = a*a + 4.0*b
        if disc >= 0:
            s1 = (-a - np.sqrt(disc)) / (2.0*b)
            s2 = (-a + np.sqrt(disc)) / (2.0*b)
            s_pos = [s for s in (s1, s2) if s > 0]
            if s_pos:
                R_eff = np.sqrt(min(s_pos))
    elif a > 0:
        R_eff = 1.0 / np.sqrt(a)

    return a, b, f, mask, R_eff

# ------------------------------ Per-panel annulus bounds ------------------------------
default_bounds = (45.0, 55.0)  # µm
fit_bounds_by_key = {
    '10': (45.0, 55.0),
    '11': (27.0, 31.0),
    '12': (31.0, 36.0),
    '13': (44.0, 48.0),
    '14': (46.0, 50.0),
    '15': (53.0, 56.0),
    '16': (54.0, 57.0),
}
def bounds_for(key):
    return fit_bounds_by_key.get(key, default_bounds)

# ------------------------------ Plot data (top row) ------------------------------
ne_max = 1.2 * np.max(dicts[0]['ne_r'])

for i, top_key, bot_key, d, t in zip(range(len(dicts)), top_plots, bottom_plots, dicts, times):
    axs0 = ax[top_key]
    if d['xabel'] is not None:
        axs0.plot(d['xabel'], d['ne_r']/1e18, color='tab:orange', lw=1.5)

    axs0.set_xlim(-xlim, xlim)
    axs0.set_ylim(0, 4.8)
    axs0.axhline(0, color=line_color, linewidth=linewidth, linestyle=linestyle)
    for yline in [2, 4]:
        axs0.axhline(yline, color=line_color, linewidth=linewidth, linestyle=linestyle, alpha=0.2)
    for xv in [-100,-50, 0, 50,100]:
        axs0.axvline(xv, color=line_color, linewidth=linewidth, linestyle=linestyle, alpha=0.2)
    axs0.set_title(f'{t:g} ns', fontsize=14)

# ------------------------------ Axes cosmetics (bottom row too) ------------------------------
for key in top_plots[1:-1]:
    ax[key].set_yticks([]); ax[key].set_xticks([])
ax[top_plots[-1]].yaxis.tick_right()
ax[top_plots[-1]].yaxis.set_label_position("right")

for key in bottom_plots[1:-1]:
    ax[key].set_yticks([])
ax[bottom_plots[-1]].yaxis.tick_right()
ax[bottom_plots[-1]].yaxis.set_label_position("right")

for key in bottom_plots:
    ax[key].set_xlim(-xlim, xlim)
    ax[key].set_ylim(-1.0, 1.9)
    for yline in [0.0, -1.0]:
        ax[key].axhline(yline, color=line_color, linewidth=linewidth, linestyle=linestyle, alpha=0.2)
    for xv in [-100,-50, 0, 50,100]:
        ax[key].axvline(xv, color=line_color, linewidth=linewidth, linestyle=linestyle, alpha=0.2)

ax['00'].axhline(Nb_80mbar/1e18, color='red', ls='dashed')
ax['00'].set_ylabel(r'$n_e$ [10$^{18}$ cm$^{-3}$]', color='tab:orange', size=14)
ax['10'].set_ylabel(r'$\Delta n/n_0$', color='black', size=14)
ax['10'].set_xlabel(r'r [$\mu$m]')
ax['12'].set_xlabel(r'r [$\mu$m]')
ax['14'].set_xlabel(r'r [$\mu$m]')
ax['16'].set_xlabel(r'r [$\mu$m]')

ax['00'].text(x=-90, y=1.05*Nb_80mbar/1e18, s='100% ionization', color='red')

# ------------------------------ Fits & patched display (bottom row) ------------------------------
fit_summaries = []
force_pure_quartic = False  # True -> enforce a=0, b>=0 (extra-wide convex core)

for bot_key, d, t in zip(bottom_plots, dicts, times):
    axs1 = ax[bot_key]

    # --- Special case: 0 ns bottom plot -> keep axes but draw nothing ---
    if t == 0.0:
        fit_summaries.append((bot_key, np.nan, np.nan, np.nan))
        continue

    if d['xabel'] is None:
        fit_summaries.append((bot_key, np.nan, np.nan, np.nan))
        continue

    x = np.asarray(d['xabel'])
    y = np.asarray(d['dn0_r'])
    y_sigma = d.get('Delta_dn0_r', None)

    r_in, r_out = bounds_for(bot_key)

    a, b, f, mask_used, R_eff = fit_quartic_convex_from_annulus(
        x, y, r_in, r_out, y_sigma=y_sigma, ridge=0.0, force_pure_quartic=force_pure_quartic
    )
    fit_summaries.append((bot_key, a, b, R_eff))

    # ---------- Patch logic with small overlap (no gap/bridge) ----------
    if len(x) > 1:
        dx_med = np.median(np.diff(np.sort(x)))
        eps = 0.5 * dx_med if np.isfinite(dx_med) and dx_med > 0 else 0.5
    else:
        eps = 0.5

    mask_inside  = (np.abs(x) <= (r_out + eps))   # dotted region (a hair wider)
    # Solid outside measured data
    y_outside = y.copy()
    y_outside[mask_inside] = np.nan
    axs1.plot(x, y_outside, color='black', lw=1.2, ls='-',label='data')

    # Dotted fitted curve inside (replacement, slight overlap)
    f_inside = np.full_like(f, np.nan, dtype=float)
    f_inside[mask_inside] = f[mask_inside]
    axs1.plot(x, f_inside, color='black', lw=1.2, ls='dotted',label='fit')

    # ---------- Vertical dotted seam at x = ±r_out ----------
    # Take y-values from the nearest samples to ±r_out
    def nearest_idx(val):
        return int(np.nanargmin(np.abs(x - val)))

    for x_seam in (r_out, -r_out):
        i_seam = nearest_idx(x_seam)
        # Measured (outside) value and fitted (inside) value at the seam
        y_meas = y[i_seam]
        y_fit  = f[i_seam]
        # Only draw if both are finite
        if np.isfinite(y_meas) and np.isfinite(y_fit):
            y0, y1 = (y_meas, y_fit) if y_meas <= y_fit else (y_fit, y_meas)
            axs1.plot([x_seam, x_seam], [y0, y1], color='black', lw=1.2, ls='dotted')


ax['12'].legend()
ax['01'].text(s='electrons',x=-80,y=2.4*0.9,color='tab:orange',size=14)
ax['11'].text(s='neutrals',x=-80,y=0.4,color='black',size=14)
ax['10'].text(s='80 mbar\n ambient\npressure',x=-80,y=0.24,color='black',size=14)
ax['00'].yaxis.set_label_position("left")
plt.subplots_adjust(wspace=0, hspace=0)


# 60 mbar, BM 582:

In [ ]:
tscan_250917_Bdel582_60mbar_t0p0ns_red   = common_path_E + r"/phase_maps/60mbar_Bdel582_t0/Interferometry2"
tscan_250917_Bdel582_60mbar_t0p0ns_green = common_path_E + r"/phase_maps/60mbar_Bdel582_t0/Interferometry1"

res_tscan_250917_Bdel582_60mbar_t0p0ns=plot_2d_data(n0_mbar=60,
                        RED_filepath=tscan_250917_Bdel582_60mbar_t0p0ns_red,
                        GREEN_filepath=tscan_250917_Bdel582_60mbar_t0p0ns_green,
                        plot_name=r"tscan_250917_Bdel582_60mbar_t0p0ns",
                        plot_title=r"tscan_250917_Bdel582_60mbar_t0p0ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=True,
                        repair_abel=True,
                        xlims_um_R=[800,1200],
                        ylims_um_R=[150,600],
                        xlims_um_G=[900,1300],
                        ylims_um_G=[300,650],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
tscan_250917_Bdel582_60mbar_t1p0ns_red   = common_path + r"/phase_maps60mbar_Bdel582_t1.0ns/Interferometry2"
tscan_250917_Bdel582_60mbar_t1p0ns_green = common_path + r"/phase_maps60mbar_Bdel582_t1.0ns/Interferometry1"

res_tscan_250917_Bdel582_60mbar_t1p0ns=plot_2d_data(n0_mbar=60,
                        RED_filepath=tscan_250917_Bdel582_60mbar_t1p0ns_red,
                        GREEN_filepath=tscan_250917_Bdel582_60mbar_t1p0ns_green,
                        plot_name=r"tscan_250917_Bdel582_60mbar_t1p0ns",
                        plot_title=r"tscan_250917_Bdel582_60mbar_t1p0ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=False,
                        repair_abel=True,
                        xlims_um_R=[200,1300],
                        ylims_um_R=[150,600],
                        xlims_um_G=[200,1300],
                        ylims_um_G=[700,1050],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
tscan_250917_Bdel582_60mbar_t2p0ns_red   = common_path + r"/phase_maps60mbar_Bdel582_t2.0ns/Interferometry2"
tscan_250917_Bdel582_60mbar_t2p0ns_green = common_path + r"/phase_maps60mbar_Bdel582_t2.0ns/Interferometry1"

res_tscan_250917_Bdel582_60mbar_t2p0ns=plot_2d_data(n0_mbar=60,
                        RED_filepath=tscan_250917_Bdel582_60mbar_t2p0ns_red,
                        GREEN_filepath=tscan_250917_Bdel582_60mbar_t2p0ns_green,
                        plot_name=r"tscan_250917_Bdel582_60mbar_t2p0ns",
                        plot_title=r"tscan_250917_Bdel582_60mbar_t2p0ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=False,
                        repair_abel=True,
                        xlims_um_R=[200,1300],
                        ylims_um_R=[150,600],
                        xlims_um_G=[200,1300],
                        ylims_um_G=[700,1050],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
tscan_250917_Bdel582_60mbar_t2p5ns_red   = common_path + r"/phase_maps60mbar_Bdel582_t2.5ns/Interferometry2"
tscan_250917_Bdel582_60mbar_t2p5ns_green = common_path + r"/phase_maps60mbar_Bdel582_t2.5ns/Interferometry1"

res_tscan_250917_Bdel582_60mbar_t2p5ns=plot_2d_data(n0_mbar=60,
                        RED_filepath=tscan_250917_Bdel582_60mbar_t2p5ns_red,
                        GREEN_filepath=tscan_250917_Bdel582_60mbar_t2p5ns_green,
                        plot_name=r"tscan_250917_Bdel582_60mbar_t2p5ns",
                        plot_title=r"tscan_250917_Bdel582_60mbar_t2p5ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=False,
                        repair_abel=True,
                        xlims_um_R=[200,1300],
                        ylims_um_R=[150,600],
                        xlims_um_G=[500,1300],
                        ylims_um_G=[700,1050],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
tscan_250917_Bdel582_60mbar_t3p0ns_red   = common_path + r"/phase_maps60mbar_Bdel582_t3.0ns/Interferometry2"
tscan_250917_Bdel582_60mbar_t3p0ns_green = common_path + r"/phase_maps60mbar_Bdel582_t3.0ns/Interferometry1"

res_tscan_250917_Bdel582_60mbar_t3p0ns=plot_2d_data(n0_mbar=60,
                        RED_filepath=tscan_250917_Bdel582_60mbar_t3p0ns_red,
                        GREEN_filepath=tscan_250917_Bdel582_60mbar_t3p0ns_green,
                        plot_name=r"tscan_250917_Bdel582_60mbar_t3p0ns",
                        plot_title=r"tscan_250917_Bdel582_60mbar_t3p0ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=False,
                        repair_abel=True,
                        xlims_um_R=[200,1300],
                        ylims_um_R=[150,600],
                        xlims_um_G=[500,1300],
                        ylims_um_G=[700,1050],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
tscan_250917_Bdel582_60mbar_t3p5ns_red   = common_path + r"/phase_maps60mbar_Bdel582_t3.5ns/Interferometry2"
tscan_250917_Bdel582_60mbar_t3p5ns_green = common_path + r"/phase_maps60mbar_Bdel582_t3.5ns/Interferometry1"

res_tscan_250917_Bdel582_60mbar_t3p5ns=plot_2d_data(n0_mbar=60,
                        RED_filepath=tscan_250917_Bdel582_60mbar_t3p5ns_red,
                        GREEN_filepath=tscan_250917_Bdel582_60mbar_t3p5ns_green,
                        plot_name=r"tscan_250917_Bdel582_60mbar_t3p5ns",
                        plot_title=r"tscan_250917_Bdel582_60mbar_t3p5ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=False,
                        repair_abel=True,
                        xlims_um_R=[200,1300],
                        ylims_um_R=[150,600],
                        xlims_um_G=[500,1300],
                        ylims_um_G=[700,1050],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
tscan_250917_Bdel582_60mbar_t4p0ns_red   = common_path + r"/phase_maps60mbar_Bdel582_t4.0ns/Interferometry2"
tscan_250917_Bdel582_60mbar_t4p0ns_green = common_path + r"/phase_maps60mbar_Bdel582_t4.0ns/Interferometry1"

res_tscan_250917_Bdel582_60mbar_t4p0ns=plot_2d_data(n0_mbar=60,
                        RED_filepath=tscan_250917_Bdel582_60mbar_t4p0ns_red,
                        GREEN_filepath=tscan_250917_Bdel582_60mbar_t4p0ns_green,
                        plot_name=r"tscan_250917_Bdel582_60mbar_t4p0ns",
                        plot_title=r"tscan_250917_Bdel582_60mbar_t4p0ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=False,
                        repair_abel=True,
                        xlims_um_R=[200,1300],
                        ylims_um_R=[150,600],
                        xlims_um_G=[500,1300],
                        ylims_um_G=[700,1050],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.constants as sc  # for sc.k

# ------------------------------ Data lists ------------------------------
res_250917_60mbar_Bdel582 = [
    res_tscan_250917_Bdel582_60mbar_t0p0ns,
    res_tscan_250917_Bdel582_60mbar_t1p0ns,
    res_tscan_250917_Bdel582_60mbar_t2p0ns,
    res_tscan_250917_Bdel582_60mbar_t2p5ns,
    res_tscan_250917_Bdel582_60mbar_t3p0ns,
    res_tscan_250917_Bdel582_60mbar_t3p5ns,
    res_tscan_250917_Bdel582_60mbar_t4p0ns
]
dicts = res_250917_60mbar_Bdel582
times = [0.0, 1.0, 2.0, 2.5, 3.0, 3.5, 4.0]  # ns

# ------------------------------ Figure scaffold ------------------------------
fig, ax = plt.subplot_mosaic(
    [['00','01','02','03','04','05','06'],
     ['10','11','12','13','14','15','16']],
    figsize=(14, 4),
    gridspec_kw={'width_ratios': [1,1,1,1,1,1,1], 'height_ratios':[1,1]}
)
top_plots    = ['00', '01', '02', '03', '04', '05', '06']
bottom_plots = ['10', '11', '12', '13', '14', '15', '16']

xlim       = 140
linestyle  = 'dotted'
linewidth  = 1
line_color = 'black'

# ------------------------------ Physics helpers ------------------------------
T_stand = 293.15  # K
Nb_60mbar = 60*(1013.25*100/1000)/(sc.k*T_stand)*1e-6 * 2  # cm^-3 (100% ionization ref for top row)

# ------------------------------ Convex quartic fit ------------------------------
def _weighted_sums(r2, r4, z, w):
    S22 = np.sum(w * (r2 * r2))    # Σ w r^4
    S24 = np.sum(w * (r2 * r4))    # Σ w r^6
    S44 = np.sum(w * (r4 * r4))    # Σ w r^8
    t2  = np.sum(w * (r2 * z))     # Σ w r^2 z
    t4  = np.sum(w * (r4 * z))     # Σ w r^4 z
    return S22, S24, S44, t2, t4

def fit_quartic_convex_from_annulus(x, y, r_in, r_out, y_sigma=None, ridge=0.0, force_pure_quartic=False):
    """
    Fit convex f(r) = -1 + a r^2 + b r^4 using only |r| in [r_in, r_out].
    Enforces convexity via a>=0, b>=0. If force_pure_quartic=True, set a=0 and fit b>=0 only.
    Returns: a, b, f(x over all x), mask_used, R_eff
    """
    x = np.asarray(x, dtype=float)
    y = np.asarray(y, dtype=float)
    mask = (np.abs(x) >= r_in) & (np.abs(x) <= r_out)
    if mask.sum() < 5:
        return np.nan, np.nan, np.full_like(x, np.nan), mask, np.nan

    xr  = x[mask]
    yr  = y[mask]
    z   = yr + 1.0
    r2  = xr**2
    r4  = r2**2

    if y_sigma is not None:
        w = 1.0 / np.clip(np.asarray(y_sigma)[mask], 1e-12, np.inf)**2
    else:
        w = np.ones_like(xr)

    def sse(a, b):
        res = z - (a*r2 + b*r4)
        return np.sum(w * res**2)

    S22, S24, S44, t2, t4 = _weighted_sums(r2, r4, z, w)

    # Boundary fits (convexity-safe)
    a_lin = max(0.0, (t2) / (S22 + ridge)) if (S22 + ridge) > 0 else 0.0
    b_lin = max(0.0, (t4) / (S44 + ridge)) if (S44 + ridge) > 0 else 0.0
    sse_a_only = sse(a_lin, 0.0)
    sse_b_only = sse(0.0, b_lin)

    if force_pure_quartic:
        a, b = 0.0, b_lin
    else:
        # Unconstrained 2x2
        A = np.array([[S22 + ridge, S24],
                      [S24,         S44 + ridge]], dtype=float)
        bvec = np.array([t2, t4], dtype=float)
        try:
            a0, b0 = np.linalg.solve(A, bvec)
        except np.linalg.LinAlgError:
            a0, b0 = np.nan, np.nan

        if np.isfinite(a0) and np.isfinite(b0) and (a0 >= 0.0) and (b0 >= 0.0):
            a, b = a0, b0
        else:
            a, b = (a_lin, 0.0) if sse_a_only <= sse_b_only else (0.0, b_lin)

    f = -1.0 + a*(x**2) + b*(x**4)

    # Effective radius where f(r)=0 (if exists)
    R_eff = np.nan
    if b > 0:
        disc = a*a + 4.0*b
        if disc >= 0:
            s1 = (-a - np.sqrt(disc)) / (2.0*b)
            s2 = (-a + np.sqrt(disc)) / (2.0*b)
            s_pos = [s for s in (s1, s2) if s > 0]
            if s_pos:
                R_eff = np.sqrt(min(s_pos))
    elif a > 0:
        R_eff = 1.0 / np.sqrt(a)

    return a, b, f, mask, R_eff

# ------------------------------ Per-panel annulus bounds ------------------------------
default_bounds = (45.0, 55.0)  # µm
fit_bounds_by_key = {
    '10': (45.0, 55.0),
    '11': (27.0, 31.0),
    '12': (31.0, 36.0),
    '13': (44.0, 48.0),
    '14': (46.0, 50.0),
    '15': (53.0, 56.0),
    '16': (54.0, 57.0),
}
def bounds_for(key):
    return fit_bounds_by_key.get(key, default_bounds)

# ------------------------------ Plot data (top row) ------------------------------
ne_max = 1.2 * np.max(dicts[0]['ne_r'])

for i, top_key, bot_key, d, t in zip(range(len(dicts)), top_plots, bottom_plots, dicts, times):
    axs0 = ax[top_key]
    if d['xabel'] is not None:
        axs0.plot(d['xabel'], d['ne_r']/1e18, color='tab:orange', lw=1.5)

    axs0.set_xlim(-xlim, xlim)
    axs0.set_ylim(0, 4.8)
    axs0.axhline(0, color=line_color, linewidth=linewidth, linestyle=linestyle)
    for yline in [2, 4]:
        axs0.axhline(yline, color=line_color, linewidth=linewidth, linestyle=linestyle, alpha=0.2)
    for xv in [-100,-50, 0, 50,100]:
        axs0.axvline(xv, color=line_color, linewidth=linewidth, linestyle=linestyle, alpha=0.2)
    axs0.set_title(f'{t:g} ns', fontsize=14)

# ------------------------------ Axes cosmetics (bottom row too) ------------------------------
for key in top_plots[1:-1]:
    ax[key].set_yticks([]); ax[key].set_xticks([])
ax[top_plots[-1]].yaxis.tick_right()
ax[top_plots[-1]].yaxis.set_label_position("right")

for key in bottom_plots[1:-1]:
    ax[key].set_yticks([])
ax[bottom_plots[-1]].yaxis.tick_right()
ax[bottom_plots[-1]].yaxis.set_label_position("right")

for key in bottom_plots:
    ax[key].set_xlim(-xlim, xlim)
    ax[key].set_ylim(-1.0, 1.9)
    for yline in [0.0, -1.0]:
        ax[key].axhline(yline, color=line_color, linewidth=linewidth, linestyle=linestyle, alpha=0.2)
    for xv in [-100,-50, 0, 50,100]:
        ax[key].axvline(xv, color=line_color, linewidth=linewidth, linestyle=linestyle, alpha=0.2)

ax['00'].axhline(Nb_60mbar/1e18, color='red', ls='dashed')
ax['00'].set_ylabel(r'$n_e$ [10$^{18}$ cm$^{-3}$]', color='tab:orange', size=14)
ax['10'].set_ylabel(r'$\Delta n/n_0$', color='black', size=14)
ax['10'].set_xlabel(r'r [$\mu$m]')
ax['12'].set_xlabel(r'r [$\mu$m]')
ax['14'].set_xlabel(r'r [$\mu$m]')
ax['16'].set_xlabel(r'r [$\mu$m]')

ax['00'].text(x=-90, y=1.05*Nb_60mbar/1e18, s='100% ionization', color='red')

# ------------------------------ Fits & patched display (bottom row) ------------------------------
fit_summaries = []
force_pure_quartic = False  # True -> enforce a=0, b>=0 (extra-wide convex core)

for bot_key, d, t in zip(bottom_plots, dicts, times):
    axs1 = ax[bot_key]

    # --- Special case: 0 ns bottom plot -> keep axes but draw nothing ---
    if t == 0.0:
        fit_summaries.append((bot_key, np.nan, np.nan, np.nan))
        continue

    if d['xabel'] is None:
        fit_summaries.append((bot_key, np.nan, np.nan, np.nan))
        continue

    x = np.asarray(d['xabel'])
    y = np.asarray(d['dn0_r'])
    y_sigma = d.get('Delta_dn0_r', None)

    r_in, r_out = bounds_for(bot_key)

    a, b, f, mask_used, R_eff = fit_quartic_convex_from_annulus(
        x, y, r_in, r_out, y_sigma=y_sigma, ridge=0.0, force_pure_quartic=force_pure_quartic
    )
    fit_summaries.append((bot_key, a, b, R_eff))

    # ---------- Patch logic with small overlap (no gap/bridge) ----------
    if len(x) > 1:
        dx_med = np.median(np.diff(np.sort(x)))
        eps = 0.5 * dx_med if np.isfinite(dx_med) and dx_med > 0 else 0.5
    else:
        eps = 0.5

    mask_inside  = (np.abs(x) <= (r_out + eps))   # dotted region (a hair wider)
    # Solid outside measured data
    y_outside = y.copy()
    y_outside[mask_inside] = np.nan
    axs1.plot(x, y_outside, color='black', lw=1.2, ls='-',label='data')

    # Dotted fitted curve inside (replacement, slight overlap)
    f_inside = np.full_like(f, np.nan, dtype=float)
    f_inside[mask_inside] = f[mask_inside]
    axs1.plot(x, f_inside, color='black', lw=1.2, ls='dotted',label='fit')

    # ---------- Vertical dotted seam at x = ±r_out ----------
    # Take y-values from the nearest samples to ±r_out
    def nearest_idx(val):
        return int(np.nanargmin(np.abs(x - val)))

    for x_seam in (r_out, -r_out):
        i_seam = nearest_idx(x_seam)
        # Measured (outside) value and fitted (inside) value at the seam
        y_meas = y[i_seam]
        y_fit  = f[i_seam]
        # Only draw if both are finite
        if np.isfinite(y_meas) and np.isfinite(y_fit):
            y0, y1 = (y_meas, y_fit) if y_meas <= y_fit else (y_fit, y_meas)
            axs1.plot([x_seam, x_seam], [y0, y1], color='black', lw=1.2, ls='dotted')


ax['12'].legend()
ax['01'].text(s='electrons',x=-80,y=2.4*0.9,color='tab:orange',size=14)
ax['11'].text(s='neutrals',x=-80,y=0.4,color='black',size=14)
ax['10'].text(s='80 mbar\n ambient\npressure',x=-80,y=0.24,color='black',size=14)
ax['00'].yaxis.set_label_position("left")
plt.subplots_adjust(wspace=0, hspace=0)


# 60 mbar, BM 560::

In [ ]:
tscan_250917_Bdel560_60mbar_t0p0ns_red   = common_path_E + r"/phase_maps/60mbar_Bdel560_t0/Interferometry2"
tscan_250917_Bdel560_60mbar_t0p0ns_green = common_path_E + r"/phase_maps/60mbar_Bdel560_t0/Interferometry1"

res_tscan_250917_Bdel560_60mbar_t0p0ns=plot_2d_data(n0_mbar=60,
                        RED_filepath=tscan_250917_Bdel560_60mbar_t0p0ns_red,
                        GREEN_filepath=tscan_250917_Bdel560_60mbar_t0p0ns_green,
                        plot_name=r"tscan_250917_Bdel560_60mbar_t0p0ns",
                        plot_title=r"tscan_250917_Bdel560_60mbar_t0p0ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=False,
                        flip_sign_GREEN=True,
                        repair_abel=True,
                        xlims_um_R=[500,1300],
                        ylims_um_R=[600,1000],
                        xlims_um_G=[300,1300],
                        ylims_um_G=[300,650],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
tscan_250917_Bdel560_60mbar_t1p0ns_red   = common_path + r"/phase_maps60mbar_Bdel560_t1.0ns/Interferometry2"
tscan_250917_Bdel560_60mbar_t1p0ns_green = common_path + r"/phase_maps60mbar_Bdel560_t1.0ns/Interferometry1"

res_tscan_250917_Bdel560_60mbar_t1p0ns=plot_2d_data(n0_mbar=100,
                        RED_filepath=tscan_250917_Bdel560_60mbar_t1p0ns_red,
                        GREEN_filepath=tscan_250917_Bdel560_60mbar_t1p0ns_green,
                        plot_name=r"tscan_250917_Bdel560_60mbar_t1p0ns",
                        plot_title=r"tscan_250917_Bdel560_60mbar_t1p0ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=True,
                        repair_abel=True,
                        xlims_um_R=[100,1300],
                        ylims_um_R=[150,600],
                        xlims_um_G=[300,1300],
                        ylims_um_G=[300,650],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
tscan_250917_Bdel560_60mbar_t2p0ns_red   = common_path + r"/phase_maps60mbar_Bdel560_t2.0ns/Interferometry2"
tscan_250917_Bdel560_60mbar_t2p0ns_green = common_path + r"/phase_maps60mbar_Bdel560_t2.0ns/Interferometry1"

res_tscan_250917_Bdel560_60mbar_t2p0ns=plot_2d_data(n0_mbar=60,
                        RED_filepath=tscan_250917_Bdel560_60mbar_t2p0ns_red,
                        GREEN_filepath=tscan_250917_Bdel560_60mbar_t2p0ns_green,
                        plot_name=r"tscan_250917_Bdel560_60mbar_t2p0ns",
                        plot_title=r"tscan_250917_Bdel560_60mbar_t2p0ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=True,
                        repair_abel=True,
                        xlims_um_R=[100,1300],
                        ylims_um_R=[150,600],
                        xlims_um_G=[300,1300],
                        ylims_um_G=[300,650],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
tscan_250917_Bdel560_60mbar_t2p5ns_red   = common_path + r"/phase_maps60mbar_Bdel560_t2.5ns/Interferometry2"
tscan_250917_Bdel560_60mbar_t2p5ns_green = common_path + r"/phase_maps60mbar_Bdel560_t2.5ns/Interferometry1"

res_tscan_250917_Bdel560_60mbar_t2p5ns=plot_2d_data(n0_mbar=60,
                        RED_filepath=tscan_250917_Bdel560_60mbar_t2p5ns_red,
                        GREEN_filepath=tscan_250917_Bdel560_60mbar_t2p5ns_green,
                        plot_name=r"tscan_250917_Bdel560_60mbar_t2p5ns",
                        plot_title=r"tscan_250917_Bdel560_60mbar_t2p5ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=True,
                        repair_abel=True,
                        xlims_um_R=[100,1300],
                        ylims_um_R=[150,600],
                        xlims_um_G=[300,1300],
                        ylims_um_G=[300,650],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
tscan_250917_Bdel560_60mbar_t3p0ns_red   = common_path + r"/phase_maps60mbar_Bdel560_t3.0ns/Interferometry2"
tscan_250917_Bdel560_60mbar_t3p0ns_green = common_path + r"/phase_maps60mbar_Bdel560_t3.0ns/Interferometry1"

res_tscan_250917_Bdel560_60mbar_t3p0ns=plot_2d_data(n0_mbar=60,
                        RED_filepath=tscan_250917_Bdel560_60mbar_t3p0ns_red,
                        GREEN_filepath=tscan_250917_Bdel560_60mbar_t3p0ns_green,
                        plot_name=r"tscan_250917_Bdel560_60mbar_t3p0ns",
                        plot_title=r"tscan_250917_Bdel560_60mbar_t3p0ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=False,
                        repair_abel=True,
                        xlims_um_R=[500,1300],
                        ylims_um_R=[150,600],
                        xlims_um_G=[700,1300],
                        ylims_um_G=[650,1050],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
tscan_250917_Bdel560_60mbar_t3p5ns_red   = common_path + r"/phase_maps60mbar_Bdel560_t3.5ns/Interferometry2"
tscan_250917_Bdel560_60mbar_t3p5ns_green = common_path + r"/phase_maps60mbar_Bdel560_t3.5ns/Interferometry1"

res_tscan_250917_Bdel560_60mbar_t3p5ns=plot_2d_data(n0_mbar=60,
                        RED_filepath=tscan_250917_Bdel560_60mbar_t3p5ns_red,
                        GREEN_filepath=tscan_250917_Bdel560_60mbar_t3p5ns_green,
                        plot_name=r"tscan_250917_Bdel560_60mbar_t3p5ns",
                        plot_title=r"tscan_250917_Bdel560_60mbar_t3p5ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=False,
                        repair_abel=True,
                        xlims_um_R=[100,1300],
                        ylims_um_R=[150,600],
                        xlims_um_G=[500,1300],
                        ylims_um_G=[650,1050],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
tscan_250917_Bdel560_60mbar_t4p0ns_red   = common_path + r"/phase_maps60mbar_Bdel560_t4.0ns/Interferometry2"
tscan_250917_Bdel560_60mbar_t4p0ns_green = common_path + r"/phase_maps60mbar_Bdel560_t4.0ns/Interferometry1"

res_tscan_250917_Bdel560_60mbar_t4p0ns=plot_2d_data(n0_mbar=60,
                        RED_filepath=tscan_250917_Bdel560_60mbar_t4p0ns_red,
                        GREEN_filepath=tscan_250917_Bdel560_60mbar_t4p0ns_green,
                        plot_name=r"tscan_250917_Bdel560_60mbar_t4p0ns",
                        plot_title=r"tscan_250917_Bdel560_60mbar_t4p0ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=False,
                        repair_abel=True,
                        xlims_um_R=[100,1300],
                        ylims_um_R=[150,600],
                        xlims_um_G=[500,1300],
                        ylims_um_G=[650,1050],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.constants as sc  # for sc.k

# ------------------------------ Data lists ------------------------------
res_250917_60mbar_Bdel560 = [
    res_tscan_250917_Bdel560_60mbar_t0p0ns,
    res_tscan_250917_Bdel560_60mbar_t1p0ns,
    res_tscan_250917_Bdel560_60mbar_t2p0ns,
    res_tscan_250917_Bdel560_60mbar_t2p5ns,
    res_tscan_250917_Bdel560_60mbar_t3p0ns,
    res_tscan_250917_Bdel560_60mbar_t3p5ns,
    res_tscan_250917_Bdel560_60mbar_t4p0ns
]
dicts = res_250917_60mbar_Bdel560
times = [0.0, 1.0, 2.0, 2.5, 3.0, 3.5, 4.0]  # ns

# ------------------------------ Figure scaffold ------------------------------
fig, ax = plt.subplot_mosaic(
    [['00','01','02','03','04','05','06'],
     ['10','11','12','13','14','15','16']],
    figsize=(14, 4),
    gridspec_kw={'width_ratios': [1,1,1,1,1,1,1], 'height_ratios':[1,1]}
)
top_plots    = ['00', '01', '02', '03', '04', '05', '06']
bottom_plots = ['10', '11', '12', '13', '14', '15', '16']

xlim       = 140
linestyle  = 'dotted'
linewidth  = 1
line_color = 'black'

# ------------------------------ Physics helpers ------------------------------
T_stand = 293.15  # K
Nb_60mbar = 60*(1013.25*100/1000)/(sc.k*T_stand)*1e-6 * 2  # cm^-3 (100% ionization ref for top row)

# ------------------------------ Convex quartic fit ------------------------------
def _weighted_sums(r2, r4, z, w):
    S22 = np.sum(w * (r2 * r2))    # Σ w r^4
    S24 = np.sum(w * (r2 * r4))    # Σ w r^6
    S44 = np.sum(w * (r4 * r4))    # Σ w r^8
    t2  = np.sum(w * (r2 * z))     # Σ w r^2 z
    t4  = np.sum(w * (r4 * z))     # Σ w r^4 z
    return S22, S24, S44, t2, t4

def fit_quartic_convex_from_annulus(x, y, r_in, r_out, y_sigma=None, ridge=0.0, force_pure_quartic=False):
    """
    Fit convex f(r) = -1 + a r^2 + b r^4 using only |r| in [r_in, r_out].
    Enforces convexity via a>=0, b>=0. If force_pure_quartic=True, set a=0 and fit b>=0 only.
    Returns: a, b, f(x over all x), mask_used, R_eff
    """
    x = np.asarray(x, dtype=float)
    y = np.asarray(y, dtype=float)
    mask = (np.abs(x) >= r_in) & (np.abs(x) <= r_out)
    if mask.sum() < 5:
        return np.nan, np.nan, np.full_like(x, np.nan), mask, np.nan

    xr  = x[mask]
    yr  = y[mask]
    z   = yr + 1.0
    r2  = xr**2
    r4  = r2**2

    if y_sigma is not None:
        w = 1.0 / np.clip(np.asarray(y_sigma)[mask], 1e-12, np.inf)**2
    else:
        w = np.ones_like(xr)

    def sse(a, b):
        res = z - (a*r2 + b*r4)
        return np.sum(w * res**2)

    S22, S24, S44, t2, t4 = _weighted_sums(r2, r4, z, w)

    # Boundary fits (convexity-safe)
    a_lin = max(0.0, (t2) / (S22 + ridge)) if (S22 + ridge) > 0 else 0.0
    b_lin = max(0.0, (t4) / (S44 + ridge)) if (S44 + ridge) > 0 else 0.0
    sse_a_only = sse(a_lin, 0.0)
    sse_b_only = sse(0.0, b_lin)

    if force_pure_quartic:
        a, b = 0.0, b_lin
    else:
        # Unconstrained 2x2
        A = np.array([[S22 + ridge, S24],
                      [S24,         S44 + ridge]], dtype=float)
        bvec = np.array([t2, t4], dtype=float)
        try:
            a0, b0 = np.linalg.solve(A, bvec)
        except np.linalg.LinAlgError:
            a0, b0 = np.nan, np.nan

        if np.isfinite(a0) and np.isfinite(b0) and (a0 >= 0.0) and (b0 >= 0.0):
            a, b = a0, b0
        else:
            a, b = (a_lin, 0.0) if sse_a_only <= sse_b_only else (0.0, b_lin)

    f = -1.0 + a*(x**2) + b*(x**4)

    # Effective radius where f(r)=0 (if exists)
    R_eff = np.nan
    if b > 0:
        disc = a*a + 4.0*b
        if disc >= 0:
            s1 = (-a - np.sqrt(disc)) / (2.0*b)
            s2 = (-a + np.sqrt(disc)) / (2.0*b)
            s_pos = [s for s in (s1, s2) if s > 0]
            if s_pos:
                R_eff = np.sqrt(min(s_pos))
    elif a > 0:
        R_eff = 1.0 / np.sqrt(a)

    return a, b, f, mask, R_eff

# ------------------------------ Per-panel annulus bounds ------------------------------
default_bounds = (45.0, 55.0)  # µm
fit_bounds_by_key = {
    '10': (45.0, 55.0),
    '11': (27.0, 31.0),
    '12': (31.0, 36.0),
    '13': (44.0, 48.0),
    '14': (46.0, 50.0),
    '15': (48.0, 53.0),
    '16': (49.0, 53.0),
}
def bounds_for(key):
    return fit_bounds_by_key.get(key, default_bounds)

# ------------------------------ Plot data (top row) ------------------------------
ne_max = 1.2 * np.max(dicts[0]['ne_r'])

for i, top_key, bot_key, d, t in zip(range(len(dicts)), top_plots, bottom_plots, dicts, times):
    axs0 = ax[top_key]
    if d['xabel'] is not None:
        axs0.plot(d['xabel'], d['ne_r']/1e18, color='tab:orange', lw=1.5)

    axs0.set_xlim(-xlim, xlim)
    axs0.set_ylim(0, 3.8)
    axs0.axhline(0, color=line_color, linewidth=linewidth, linestyle=linestyle)
    for yline in [2, 4]:
        axs0.axhline(yline, color=line_color, linewidth=linewidth, linestyle=linestyle, alpha=0.2)
    for xv in [-100,-50, 0, 50,100]:
        axs0.axvline(xv, color=line_color, linewidth=linewidth, linestyle=linestyle, alpha=0.2)
    axs0.set_title(f'{t:g} ns', fontsize=14)

# ------------------------------ Axes cosmetics (bottom row too) ------------------------------
for key in top_plots[1:-1]:
    ax[key].set_yticks([]); ax[key].set_xticks([])
ax[top_plots[-1]].yaxis.tick_right()
ax[top_plots[-1]].yaxis.set_label_position("right")

for key in bottom_plots[1:-1]:
    ax[key].set_yticks([])
ax[bottom_plots[-1]].yaxis.tick_right()
ax[bottom_plots[-1]].yaxis.set_label_position("right")

for key in bottom_plots:
    ax[key].set_xlim(-xlim, xlim)
    ax[key].set_ylim(-1.0, 1.9)
    for yline in [0.0, -1.0]:
        ax[key].axhline(yline, color=line_color, linewidth=linewidth, linestyle=linestyle, alpha=0.2)
    for xv in [-100,-50, 0, 50,100]:
        ax[key].axvline(xv, color=line_color, linewidth=linewidth, linestyle=linestyle, alpha=0.2)

ax['00'].axhline(Nb_60mbar/1e18, color='red', ls='dashed')
ax['00'].set_ylabel(r'$n_e$ [10$^{18}$ cm$^{-3}$]', color='tab:orange', size=14)
ax['10'].set_ylabel(r'$\Delta n/n_0$', color='black', size=14)
ax['10'].set_xlabel(r'r [$\mu$m]')
ax['12'].set_xlabel(r'r [$\mu$m]')
ax['14'].set_xlabel(r'r [$\mu$m]')
ax['16'].set_xlabel(r'r [$\mu$m]')

ax['00'].text(x=-90, y=1.05*Nb_60mbar/1e18, s='100% ionization', color='red')

# ------------------------------ Fits & patched display (bottom row) ------------------------------
fit_summaries = []
force_pure_quartic = False  # True -> enforce a=0, b>=0 (extra-wide convex core)

for bot_key, d, t in zip(bottom_plots, dicts, times):
    axs1 = ax[bot_key]

    # --- Special case: 0 ns bottom plot -> keep axes but draw nothing ---
    if t == 0.0:
        fit_summaries.append((bot_key, np.nan, np.nan, np.nan))
        continue

    if d['xabel'] is None:
        fit_summaries.append((bot_key, np.nan, np.nan, np.nan))
        continue

    x = np.asarray(d['xabel'])
    y = np.asarray(d['dn0_r'])
    y_sigma = d.get('Delta_dn0_r', None)

    r_in, r_out = bounds_for(bot_key)

    a, b, f, mask_used, R_eff = fit_quartic_convex_from_annulus(
        x, y, r_in, r_out, y_sigma=y_sigma, ridge=0.0, force_pure_quartic=force_pure_quartic
    )
    fit_summaries.append((bot_key, a, b, R_eff))

    # ---------- Patch logic with small overlap (no gap/bridge) ----------
    if len(x) > 1:
        dx_med = np.median(np.diff(np.sort(x)))
        eps = 0.5 * dx_med if np.isfinite(dx_med) and dx_med > 0 else 0.5
    else:
        eps = 0.5

    mask_inside  = (np.abs(x) <= (r_out + eps))   # dotted region (a hair wider)
    # Solid outside measured data
    y_outside = y.copy()
    y_outside[mask_inside] = np.nan
    axs1.plot(x, y_outside, color='black', lw=1.2, ls='-',label='data')

    # Dotted fitted curve inside (replacement, slight overlap)
    f_inside = np.full_like(f, np.nan, dtype=float)
    f_inside[mask_inside] = f[mask_inside]
    axs1.plot(x, f_inside, color='black', lw=1.2, ls='dotted',label='fit')

    # ---------- Vertical dotted seam at x = ±r_out ----------
    # Take y-values from the nearest samples to ±r_out
    def nearest_idx(val):
        return int(np.nanargmin(np.abs(x - val)))

    for x_seam in (r_out, -r_out):
        i_seam = nearest_idx(x_seam)
        # Measured (outside) value and fitted (inside) value at the seam
        y_meas = y[i_seam]
        y_fit  = f[i_seam]
        # Only draw if both are finite
        if np.isfinite(y_meas) and np.isfinite(y_fit):
            y0, y1 = (y_meas, y_fit) if y_meas <= y_fit else (y_fit, y_meas)
            axs1.plot([x_seam, x_seam], [y0, y1], color='black', lw=1.2, ls='dotted')


ax['12'].legend()
ax['01'].text(s='electrons',x=-80,y=2.4*0.6,color='tab:orange',size=14)
ax['11'].text(s='neutrals',x=-80,y=0.4,color='black',size=14)
ax['10'].text(s='60 mbar\n ambient\npressure',x=-80,y=0.24,color='black',size=14)
ax['00'].yaxis.set_label_position("left")
plt.subplots_adjust(wspace=0, hspace=0)


# 40 mbar, BM 560:

In [ ]:
tscan_250917_Bdel560_40mbar_t0p0ns_red   = common_path_E + r"/phase_maps/40mbar_Bdel560_t0/Interferometry2"
tscan_250917_Bdel560_40mbar_t0p0ns_green = common_path_E + r"/phase_maps/40mbar_Bdel560_t0/Interferometry1"

res_tscan_250917_Bdel560_40mbar_t0p0ns=plot_2d_data(n0_mbar=40,
                        RED_filepath=tscan_250917_Bdel560_40mbar_t0p0ns_red,
                        GREEN_filepath=tscan_250917_Bdel560_40mbar_t0p0ns_green,
                        plot_name=r"tscan_250917_Bdel560_40mbar_t0p0ns",
                        plot_title=r"tscan_250917_Bdel560_40mbar_t0p0ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=True,
                        repair_abel=True,
                        xlims_um_R=[900,1300],
                        ylims_um_R=[150,550],
                        xlims_um_G=[300,1300],
                        ylims_um_G=[300,650],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
tscan_250917_Bdel560_40mbar_t1p0ns_red   = common_path + r"/phase_maps40mbar_Bdel560_t1.0ns/Interferometry2"
tscan_250917_Bdel560_40mbar_t1p0ns_green = common_path + r"/phase_maps40mbar_Bdel560_t1.0ns/Interferometry1"

res_tscan_250917_Bdel560_40mbar_t1p0ns=plot_2d_data(n0_mbar=40,
                        RED_filepath=tscan_250917_Bdel560_40mbar_t1p0ns_red,
                        GREEN_filepath=tscan_250917_Bdel560_40mbar_t1p0ns_green,
                        plot_name=r"tscan_250917_Bdel560_40mbar_t1p0ns",
                        plot_title=r"tscan_250917_Bdel560_40mbar_t1p0ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=True,
                        repair_abel=True,
                        xlims_um_R=[100,1300],
                        ylims_um_R=[150,600],
                        xlims_um_G=[300,1300],
                        ylims_um_G=[300,650],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
tscan_250917_Bdel560_40mbar_t2p0ns_red   = common_path + r"/phase_maps40mbar_Bdel560_t2.0ns/Interferometry2"
tscan_250917_Bdel560_40mbar_t2p0ns_green = common_path + r"/phase_maps40mbar_Bdel560_t2.0ns/Interferometry1"

res_tscan_250917_Bdel560_40mbar_t2p0ns=plot_2d_data(n0_mbar=40,
                        RED_filepath=tscan_250917_Bdel560_40mbar_t2p0ns_red,
                        GREEN_filepath=tscan_250917_Bdel560_40mbar_t2p0ns_green,
                        plot_name=r"tscan_250917_Bdel560_40mbar_t2p0ns",
                        plot_title=r"tscan_250917_Bdel560_40mbar_t2p0ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=True,
                        repair_abel=True,
                        xlims_um_R=[100,1300],
                        ylims_um_R=[150,600],
                        xlims_um_G=[300,1300],
                        ylims_um_G=[300,650],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
tscan_250917_Bdel560_40mbar_t2p5ns_red   = common_path + r"/phase_maps40mbar_Bdel560_t2.5ns/Interferometry2"
tscan_250917_Bdel560_40mbar_t2p5ns_green = common_path + r"/phase_maps40mbar_Bdel560_t2.5ns/Interferometry1"

res_tscan_250917_Bdel560_40mbar_t2p5ns=plot_2d_data(n0_mbar=40,
                        RED_filepath=tscan_250917_Bdel560_40mbar_t2p5ns_red,
                        GREEN_filepath=tscan_250917_Bdel560_40mbar_t2p5ns_green,
                        plot_name=r"tscan_250917_Bdel560_40mbar_t2p5ns",
                        plot_title=r"tscan_250917_Bdel560_40mbar_t2p5ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=True,
                        repair_abel=True,
                        xlims_um_R=[100,1300],
                        ylims_um_R=[150,600],
                        xlims_um_G=[300,1300],
                        ylims_um_G=[300,650],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
tscan_250917_Bdel560_40mbar_t3p0ns_red   = common_path + r"/phase_maps40mbar_Bdel560_t3.0ns/Interferometry2"
tscan_250917_Bdel560_40mbar_t3p0ns_green = common_path + r"/phase_maps40mbar_Bdel560_t3.0ns/Interferometry1"

res_tscan_250917_Bdel560_40mbar_t3p0ns=plot_2d_data(n0_mbar=40,
                        RED_filepath=tscan_250917_Bdel560_40mbar_t3p0ns_red,
                        GREEN_filepath=tscan_250917_Bdel560_40mbar_t3p0ns_green,
                        plot_name=r"tscan_250917_Bdel560_40mbar_t3p0ns",
                        plot_title=r"tscan_250917_Bdel560_40mbar_t3p0ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=True,
                        repair_abel=True,
                        xlims_um_R=[100,1300],
                        ylims_um_R=[150,600],
                        xlims_um_G=[300,1300],
                        ylims_um_G=[300,650],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
tscan_250917_Bdel560_40mbar_t3p5ns_red   = common_path + r"/phase_maps40mbar_Bdel560_t3.5ns/Interferometry2"
tscan_250917_Bdel560_40mbar_t3p5ns_green = common_path + r"/phase_maps40mbar_Bdel560_t3.5ns/Interferometry1"

res_tscan_250917_Bdel560_40mbar_t3p5ns=plot_2d_data(n0_mbar=40,
                        RED_filepath=tscan_250917_Bdel560_40mbar_t3p5ns_red,
                        GREEN_filepath=tscan_250917_Bdel560_40mbar_t3p5ns_green,
                        plot_name=r"tscan_250917_Bdel560_40mbar_t3p5ns",
                        plot_title=r"tscan_250917_Bdel560_40mbar_t3p5ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=True,
                        repair_abel=True,
                        xlims_um_R=[100,1300],
                        ylims_um_R=[150,600],
                        xlims_um_G=[300,1300],
                        ylims_um_G=[300,650],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
tscan_250917_Bdel560_40mbar_t4p0ns_red   = common_path + r"/phase_maps40mbar_Bdel560_t4.0ns/Interferometry2"
tscan_250917_Bdel560_40mbar_t4p0ns_green = common_path + r"/phase_maps40mbar_Bdel560_t4.0ns/Interferometry1"

res_tscan_250917_Bdel560_40mbar_t4p0ns=plot_2d_data(n0_mbar=40,
                        RED_filepath=tscan_250917_Bdel560_40mbar_t4p0ns_red,
                        GREEN_filepath=tscan_250917_Bdel560_40mbar_t4p0ns_green,
                        plot_name=r"tscan_250917_Bdel560_40mbar_t4p0ns",
                        plot_title=r"tscan_250917_Bdel560_40mbar_t4p0ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=True,
                        repair_abel=True,
                        xlims_um_R=[100,1300],
                        ylims_um_R=[150,600],
                        xlims_um_G=[300,1300],
                        ylims_um_G=[300,650],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.constants as sc  # for sc.k

# ------------------------------ Data lists ------------------------------
res_250917_40mbar_Bdel560 = [
    res_tscan_250917_Bdel560_40mbar_t0p0ns,
    res_tscan_250917_Bdel560_40mbar_t1p0ns,
    res_tscan_250917_Bdel560_40mbar_t2p0ns,
    res_tscan_250917_Bdel560_40mbar_t2p5ns,
    res_tscan_250917_Bdel560_40mbar_t3p0ns,
    res_tscan_250917_Bdel560_40mbar_t3p5ns,
    res_tscan_250917_Bdel560_40mbar_t4p0ns
]
dicts = res_250917_40mbar_Bdel560
times = [0.0, 1.0, 2.0, 2.5, 3.0, 3.5, 4.0]  # ns

# ------------------------------ Figure scaffold ------------------------------
fig, ax = plt.subplot_mosaic(
    [['00','01','02','03','04','05','06'],
     ['10','11','12','13','14','15','16']],
    figsize=(14, 4),
    gridspec_kw={'width_ratios': [1,1,1,1,1,1,1], 'height_ratios':[1,1]}
)
top_plots    = ['00', '01', '02', '03', '04', '05', '06']
bottom_plots = ['10', '11', '12', '13', '14', '15', '16']

xlim       = 140
linestyle  = 'dotted'
linewidth  = 1
line_color = 'black'

# ------------------------------ Physics helpers ------------------------------
T_stand = 293.15  # K
Nb_40mbar = 40*(1013.25*100/1000)/(sc.k*T_stand)*1e-6 * 2  # cm^-3 (100% ionization ref for top row)

# ------------------------------ Convex quartic fit ------------------------------
def _weighted_sums(r2, r4, z, w):
    S22 = np.sum(w * (r2 * r2))    # Σ w r^4
    S24 = np.sum(w * (r2 * r4))    # Σ w r^6
    S44 = np.sum(w * (r4 * r4))    # Σ w r^8
    t2  = np.sum(w * (r2 * z))     # Σ w r^2 z
    t4  = np.sum(w * (r4 * z))     # Σ w r^4 z
    return S22, S24, S44, t2, t4

def fit_quartic_convex_from_annulus(x, y, r_in, r_out, y_sigma=None, ridge=0.0, force_pure_quartic=False):
    """
    Fit convex f(r) = -1 + a r^2 + b r^4 using only |r| in [r_in, r_out].
    Enforces convexity via a>=0, b>=0. If force_pure_quartic=True, set a=0 and fit b>=0 only.
    Returns: a, b, f(x over all x), mask_used, R_eff
    """
    x = np.asarray(x, dtype=float)
    y = np.asarray(y, dtype=float)
    mask = (np.abs(x) >= r_in) & (np.abs(x) <= r_out)
    if mask.sum() < 5:
        return np.nan, np.nan, np.full_like(x, np.nan), mask, np.nan

    xr  = x[mask]
    yr  = y[mask]
    z   = yr + 1.0
    r2  = xr**2
    r4  = r2**2

    if y_sigma is not None:
        w = 1.0 / np.clip(np.asarray(y_sigma)[mask], 1e-12, np.inf)**2
    else:
        w = np.ones_like(xr)

    def sse(a, b):
        res = z - (a*r2 + b*r4)
        return np.sum(w * res**2)

    S22, S24, S44, t2, t4 = _weighted_sums(r2, r4, z, w)

    # Boundary fits (convexity-safe)
    a_lin = max(0.0, (t2) / (S22 + ridge)) if (S22 + ridge) > 0 else 0.0
    b_lin = max(0.0, (t4) / (S44 + ridge)) if (S44 + ridge) > 0 else 0.0
    sse_a_only = sse(a_lin, 0.0)
    sse_b_only = sse(0.0, b_lin)

    if force_pure_quartic:
        a, b = 0.0, b_lin
    else:
        # Unconstrained 2x2
        A = np.array([[S22 + ridge, S24],
                      [S24,         S44 + ridge]], dtype=float)
        bvec = np.array([t2, t4], dtype=float)
        try:
            a0, b0 = np.linalg.solve(A, bvec)
        except np.linalg.LinAlgError:
            a0, b0 = np.nan, np.nan

        if np.isfinite(a0) and np.isfinite(b0) and (a0 >= 0.0) and (b0 >= 0.0):
            a, b = a0, b0
        else:
            a, b = (a_lin, 0.0) if sse_a_only <= sse_b_only else (0.0, b_lin)

    f = -1.0 + a*(x**2) + b*(x**4)

    # Effective radius where f(r)=0 (if exists)
    R_eff = np.nan
    if b > 0:
        disc = a*a + 4.0*b
        if disc >= 0:
            s1 = (-a - np.sqrt(disc)) / (2.0*b)
            s2 = (-a + np.sqrt(disc)) / (2.0*b)
            s_pos = [s for s in (s1, s2) if s > 0]
            if s_pos:
                R_eff = np.sqrt(min(s_pos))
    elif a > 0:
        R_eff = 1.0 / np.sqrt(a)

    return a, b, f, mask, R_eff

# ------------------------------ Per-panel annulus bounds ------------------------------
default_bounds = (45.0, 55.0)  # µm
fit_bounds_by_key = {
    '10': (45.0, 55.0),
    '11': (27.0, 31.0),
    '12': (31.0, 36.0),
    '13': (40.0, 43.0),
    '14': (42.0, 45.0),
    '15': (45.0, 48.0),
    '16': (48.0, 51.0),
}
def bounds_for(key):
    return fit_bounds_by_key.get(key, default_bounds)

# ------------------------------ Plot data (top row) ------------------------------
ne_max = 1.2 * np.max(dicts[0]['ne_r'])

for i, top_key, bot_key, d, t in zip(range(len(dicts)), top_plots, bottom_plots, dicts, times):
    axs0 = ax[top_key]
    if d['xabel'] is not None:
        axs0.plot(d['xabel'], d['ne_r']/1e18, color='tab:orange', lw=1.5)

    axs0.set_xlim(-xlim, xlim)
    axs0.set_ylim(0, 2.4)
    axs0.axhline(0, color=line_color, linewidth=linewidth, linestyle=linestyle)
    for yline in [2, 4]:
        axs0.axhline(yline, color=line_color, linewidth=linewidth, linestyle=linestyle, alpha=0.2)
    for xv in [-100,-50, 0, 50,100]:
        axs0.axvline(xv, color=line_color, linewidth=linewidth, linestyle=linestyle, alpha=0.2)
    axs0.set_title(f'{t:g} ns', fontsize=14)

# ------------------------------ Axes cosmetics (bottom row too) ------------------------------
for key in top_plots[1:-1]:
    ax[key].set_yticks([]); ax[key].set_xticks([])
ax[top_plots[-1]].yaxis.tick_right()
ax[top_plots[-1]].yaxis.set_label_position("right")

for key in bottom_plots[1:-1]:
    ax[key].set_yticks([])
ax[bottom_plots[-1]].yaxis.tick_right()
ax[bottom_plots[-1]].yaxis.set_label_position("right")

for key in bottom_plots:
    ax[key].set_xlim(-xlim, xlim)
    ax[key].set_ylim(-1.0, 1.9)
    for yline in [0.0, -1.0]:
        ax[key].axhline(yline, color=line_color, linewidth=linewidth, linestyle=linestyle, alpha=0.2)
    for xv in [-100,-50, 0, 50,100]:
        ax[key].axvline(xv, color=line_color, linewidth=linewidth, linestyle=linestyle, alpha=0.2)

ax['00'].axhline(Nb_40mbar/1e18, color='red', ls='dashed')
ax['00'].set_ylabel(r'$n_e$ [10$^{18}$ cm$^{-3}$]', color='tab:orange', size=14)
ax['10'].set_ylabel(r'$\Delta n/n_0$', color='black', size=14)
ax['10'].set_xlabel(r'r [$\mu$m]')
ax['12'].set_xlabel(r'r [$\mu$m]')
ax['14'].set_xlabel(r'r [$\mu$m]')
ax['16'].set_xlabel(r'r [$\mu$m]')

ax['00'].text(x=-90, y=1.05*Nb_40mbar/1e18, s='100% ionization', color='red')

# ------------------------------ Fits & patched display (bottom row) ------------------------------
fit_summaries = []
force_pure_quartic = False  # True -> enforce a=0, b>=0 (extra-wide convex core)

for bot_key, d, t in zip(bottom_plots, dicts, times):
    axs1 = ax[bot_key]

    # --- Special case: 0 ns bottom plot -> keep axes but draw nothing ---
    if t == 0.0:
        fit_summaries.append((bot_key, np.nan, np.nan, np.nan))
        continue

    if d['xabel'] is None:
        fit_summaries.append((bot_key, np.nan, np.nan, np.nan))
        continue

    x = np.asarray(d['xabel'])
    y = np.asarray(d['dn0_r'])
    y_sigma = d.get('Delta_dn0_r', None)

    r_in, r_out = bounds_for(bot_key)

    a, b, f, mask_used, R_eff = fit_quartic_convex_from_annulus(
        x, y, r_in, r_out, y_sigma=y_sigma, ridge=0.0, force_pure_quartic=force_pure_quartic
    )
    fit_summaries.append((bot_key, a, b, R_eff))

    # ---------- Patch logic with small overlap (no gap/bridge) ----------
    if len(x) > 1:
        dx_med = np.median(np.diff(np.sort(x)))
        eps = 0.5 * dx_med if np.isfinite(dx_med) and dx_med > 0 else 0.5
    else:
        eps = 0.5

    mask_inside  = (np.abs(x) <= (r_out + eps))   # dotted region (a hair wider)
    # Solid outside measured data
    y_outside = y.copy()
    y_outside[mask_inside] = np.nan
    axs1.plot(x, y_outside, color='black', lw=1.2, ls='-',label='data')

    # Dotted fitted curve inside (replacement, slight overlap)
    f_inside = np.full_like(f, np.nan, dtype=float)
    f_inside[mask_inside] = f[mask_inside]
    axs1.plot(x, f_inside, color='black', lw=1.2, ls='dotted',label='fit')

    # ---------- Vertical dotted seam at x = ±r_out ----------
    # Take y-values from the nearest samples to ±r_out
    def nearest_idx(val):
        return int(np.nanargmin(np.abs(x - val)))

    for x_seam in (r_out, -r_out):
        i_seam = nearest_idx(x_seam)
        # Measured (outside) value and fitted (inside) value at the seam
        y_meas = y[i_seam]
        y_fit  = f[i_seam]
        # Only draw if both are finite
        if np.isfinite(y_meas) and np.isfinite(y_fit):
            y0, y1 = (y_meas, y_fit) if y_meas <= y_fit else (y_fit, y_meas)
            axs1.plot([x_seam, x_seam], [y0, y1], color='black', lw=1.2, ls='dotted')


ax['12'].legend()
ax['01'].text(s='electrons',x=-80,y=2.4*0.4,color='tab:orange',size=14)
ax['11'].text(s='neutrals',x=-80,y=0.8,color='black',size=14)
ax['10'].text(s='40 mbar\n ambient\npressure',x=-80,y=0.24,color='black',size=14)
ax['00'].yaxis.set_label_position("left")
plt.subplots_adjust(wspace=0, hspace=0)


# 40 mbar, BM 582:

In [ ]:
tscan_250917_Bdel582_40mbar_t0p0ns_red   = common_path_E + r"/phase_maps/40mbar_Bdel582_t0/Interferometry2"
tscan_250917_Bdel582_40mbar_t0p0ns_green = common_path_E + r"/phase_maps/40mbar_Bdel582_t0/Interferometry1"

res_tscan_250917_Bdel582_40mbar_t0p0ns=plot_2d_data(n0_mbar=40,
                        RED_filepath=tscan_250917_Bdel582_40mbar_t0p0ns_red,
                        GREEN_filepath=tscan_250917_Bdel582_40mbar_t0p0ns_green,
                        plot_name=r"tscan_250917_Bdel582_40mbar_t0p0ns",
                        plot_title=r"tscan_250917_Bdel582_40mbar_t0p0ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=True,
                        repair_abel=True,
                        xlims_um_R=[900,1300],
                        ylims_um_R=[150,550],
                        xlims_um_G=[300,1300],
                        ylims_um_G=[300,650],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
tscan_250917_Bdel582_40mbar_t1p0ns_red   = common_path + r"/phase_maps40mbar_Bdel582_t1.0ns/Interferometry2"
tscan_250917_Bdel582_40mbar_t1p0ns_green = common_path + r"/phase_maps40mbar_Bdel582_t1.0ns/Interferometry1"

res_tscan_250917_Bdel582_40mbar_t1p0ns=plot_2d_data(n0_mbar=40,
                        RED_filepath=tscan_250917_Bdel582_40mbar_t1p0ns_red,
                        GREEN_filepath=tscan_250917_Bdel582_40mbar_t1p0ns_green,
                        plot_name=r"tscan_250917_Bdel582_40mbar_t1p0ns",
                        plot_title=r"tscan_250917_Bdel582_40mbar_t1p0ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=True,
                        repair_abel=True,
                        xlims_um_R=[100,1300],
                        ylims_um_R=[150,600],
                        xlims_um_G=[300,1300],
                        ylims_um_G=[300,650],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
tscan_250917_Bdel582_40mbar_t2p0ns_red   = common_path + r"/phase_maps40mbar_Bdel582_t2.0ns/Interferometry2"
tscan_250917_Bdel582_40mbar_t2p0ns_green = common_path + r"/phase_maps40mbar_Bdel582_t2.0ns/Interferometry1"

res_tscan_250917_Bdel582_40mbar_t2p0ns=plot_2d_data(n0_mbar=40,
                        RED_filepath=tscan_250917_Bdel582_40mbar_t2p0ns_red,
                        GREEN_filepath=tscan_250917_Bdel582_40mbar_t2p0ns_green,
                        plot_name=r"tscan_250917_Bdel582_40mbar_t2p0ns",
                        plot_title=r"tscan_250917_Bdel582_40mbar_t2p0ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=True,
                        repair_abel=True,
                        xlims_um_R=[100,1300],
                        ylims_um_R=[150,600],
                        xlims_um_G=[300,1300],
                        ylims_um_G=[300,650],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
tscan_250917_Bdel582_40mbar_t2p5ns_red   = common_path + r"/phase_maps40mbar_Bdel582_t2.5ns/Interferometry2"
tscan_250917_Bdel582_40mbar_t2p5ns_green = common_path + r"/phase_maps40mbar_Bdel582_t2.5ns/Interferometry1"

res_tscan_250917_Bdel582_40mbar_t2p5ns=plot_2d_data(n0_mbar=40,
                        RED_filepath=tscan_250917_Bdel582_40mbar_t2p5ns_red,
                        GREEN_filepath=tscan_250917_Bdel582_40mbar_t2p5ns_green,
                        plot_name=r"tscan_250917_Bdel582_40mbar_t2p5ns",
                        plot_title=r"tscan_250917_Bdel582_40mbar_t2p5ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=True,
                        repair_abel=True,
                        xlims_um_R=[100,1300],
                        ylims_um_R=[150,600],
                        xlims_um_G=[300,1300],
                        ylims_um_G=[300,650],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
tscan_250917_Bdel582_40mbar_t3p0ns_red   = common_path + r"/phase_maps40mbar_Bdel582_t3.0ns/Interferometry2"
tscan_250917_Bdel582_40mbar_t3p0ns_green = common_path + r"/phase_maps40mbar_Bdel582_t3.0ns/Interferometry1"

res_tscan_250917_Bdel582_40mbar_t3p0ns=plot_2d_data(n0_mbar=40,
                        RED_filepath=tscan_250917_Bdel582_40mbar_t3p0ns_red,
                        GREEN_filepath=tscan_250917_Bdel582_40mbar_t3p0ns_green,
                        plot_name=r"tscan_250917_Bdel582_40mbar_t3p0ns",
                        plot_title=r"tscan_250917_Bdel582_40mbar_t3p0ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=True,
                        repair_abel=True,
                        xlims_um_R=[100,1300],
                        ylims_um_R=[150,600],
                        xlims_um_G=[300,1300],
                        ylims_um_G=[300,650],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
tscan_250917_Bdel582_40mbar_t3p5ns_red   = common_path + r"/phase_maps40mbar_Bdel582_t3.5ns/Interferometry2"
tscan_250917_Bdel582_40mbar_t3p5ns_green = common_path + r"/phase_maps40mbar_Bdel582_t3.5ns/Interferometry1"

res_tscan_250917_Bdel582_40mbar_t3p5ns=plot_2d_data(n0_mbar=40,
                        RED_filepath=tscan_250917_Bdel582_40mbar_t3p5ns_red,
                        GREEN_filepath=tscan_250917_Bdel582_40mbar_t3p5ns_green,
                        plot_name=r"tscan_250917_Bdel582_40mbar_t3p5ns",
                        plot_title=r"tscan_250917_Bdel582_40mbar_t3p5ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=True,
                        repair_abel=True,
                        xlims_um_R=[100,1300],
                        ylims_um_R=[150,600],
                        xlims_um_G=[300,1300],
                        ylims_um_G=[300,650],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
tscan_250917_Bdel582_40mbar_t4p0ns_red   = common_path + r"/phase_maps40mbar_Bdel582_t4.0ns/Interferometry2"
tscan_250917_Bdel582_40mbar_t4p0ns_green = common_path + r"/phase_maps40mbar_Bdel582_t4.0ns/Interferometry1"

res_tscan_250917_Bdel582_40mbar_t4p0ns=plot_2d_data(n0_mbar=40,
                        RED_filepath=tscan_250917_Bdel582_40mbar_t4p0ns_red,
                        GREEN_filepath=tscan_250917_Bdel582_40mbar_t4p0ns_green,
                        plot_name=r"tscan_250917_Bdel582_40mbar_t4p0ns",
                        plot_title=r"tscan_250917_Bdel582_40mbar_t4p0ns", 
                        dev_angle_override_R=2,
                        dev_angle_override_G=0,
                        flip_sign_RED=True,
                        flip_sign_GREEN=True,
                        repair_abel=True,
                        xlims_um_R=[100,1300],
                        ylims_um_R=[150,600],
                        xlims_um_G=[300,1300],
                        ylims_um_G=[300,650],
                        
                        xlims_bg_R=[100,1050],
                        ylims_bg_R=[30,200],
                        xlims_bg_G=[900,1300],
                        ylims_bg_G=[80,250])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.constants as sc  # for sc.k

# ------------------------------ Data lists ------------------------------
res_250917_40mbar_Bdel582 = [
    res_tscan_250917_Bdel582_40mbar_t0p0ns,
    res_tscan_250917_Bdel582_40mbar_t1p0ns,
    res_tscan_250917_Bdel582_40mbar_t2p0ns,
    res_tscan_250917_Bdel582_40mbar_t2p5ns,
    res_tscan_250917_Bdel582_40mbar_t3p0ns,
    res_tscan_250917_Bdel582_40mbar_t3p5ns,
    res_tscan_250917_Bdel582_40mbar_t4p0ns
]
dicts = res_250917_40mbar_Bdel582
times = [0.0, 1.0, 2.0, 2.5, 3.0, 3.5, 4.0]  # ns

# ------------------------------ Figure scaffold ------------------------------
fig, ax = plt.subplot_mosaic(
    [['00','01','02','03','04','05','06'],
     ['10','11','12','13','14','15','16']],
    figsize=(14, 4),
    gridspec_kw={'width_ratios': [1,1,1,1,1,1,1], 'height_ratios':[1,1]}
)
top_plots    = ['00', '01', '02', '03', '04', '05', '06']
bottom_plots = ['10', '11', '12', '13', '14', '15', '16']

xlim       = 140
linestyle  = 'dotted'
linewidth  = 1
line_color = 'black'

# ------------------------------ Physics helpers ------------------------------
T_stand = 293.15  # K
Nb_40mbar = 40*(1013.25*100/1000)/(sc.k*T_stand)*1e-6 * 2  # cm^-3 (100% ionization ref for top row)

# ------------------------------ Convex quartic fit ------------------------------
def _weighted_sums(r2, r4, z, w):
    S22 = np.sum(w * (r2 * r2))    # Σ w r^4
    S24 = np.sum(w * (r2 * r4))    # Σ w r^6
    S44 = np.sum(w * (r4 * r4))    # Σ w r^8
    t2  = np.sum(w * (r2 * z))     # Σ w r^2 z
    t4  = np.sum(w * (r4 * z))     # Σ w r^4 z
    return S22, S24, S44, t2, t4

def fit_quartic_convex_from_annulus(x, y, r_in, r_out, y_sigma=None, ridge=0.0, force_pure_quartic=False):
    """
    Fit convex f(r) = -1 + a r^2 + b r^4 using only |r| in [r_in, r_out].
    Enforces convexity via a>=0, b>=0. If force_pure_quartic=True, set a=0 and fit b>=0 only.
    Returns: a, b, f(x over all x), mask_used, R_eff
    """
    x = np.asarray(x, dtype=float)
    y = np.asarray(y, dtype=float)
    mask = (np.abs(x) >= r_in) & (np.abs(x) <= r_out)
    if mask.sum() < 5:
        return np.nan, np.nan, np.full_like(x, np.nan), mask, np.nan

    xr  = x[mask]
    yr  = y[mask]
    z   = yr + 1.0
    r2  = xr**2
    r4  = r2**2

    if y_sigma is not None:
        w = 1.0 / np.clip(np.asarray(y_sigma)[mask], 1e-12, np.inf)**2
    else:
        w = np.ones_like(xr)

    def sse(a, b):
        res = z - (a*r2 + b*r4)
        return np.sum(w * res**2)

    S22, S24, S44, t2, t4 = _weighted_sums(r2, r4, z, w)

    # Boundary fits (convexity-safe)
    a_lin = max(0.0, (t2) / (S22 + ridge)) if (S22 + ridge) > 0 else 0.0
    b_lin = max(0.0, (t4) / (S44 + ridge)) if (S44 + ridge) > 0 else 0.0
    sse_a_only = sse(a_lin, 0.0)
    sse_b_only = sse(0.0, b_lin)

    if force_pure_quartic:
        a, b = 0.0, b_lin
    else:
        # Unconstrained 2x2
        A = np.array([[S22 + ridge, S24],
                      [S24,         S44 + ridge]], dtype=float)
        bvec = np.array([t2, t4], dtype=float)
        try:
            a0, b0 = np.linalg.solve(A, bvec)
        except np.linalg.LinAlgError:
            a0, b0 = np.nan, np.nan

        if np.isfinite(a0) and np.isfinite(b0) and (a0 >= 0.0) and (b0 >= 0.0):
            a, b = a0, b0
        else:
            a, b = (a_lin, 0.0) if sse_a_only <= sse_b_only else (0.0, b_lin)

    f = -1.0 + a*(x**2) + b*(x**4)

    # Effective radius where f(r)=0 (if exists)
    R_eff = np.nan
    if b > 0:
        disc = a*a + 4.0*b
        if disc >= 0:
            s1 = (-a - np.sqrt(disc)) / (2.0*b)
            s2 = (-a + np.sqrt(disc)) / (2.0*b)
            s_pos = [s for s in (s1, s2) if s > 0]
            if s_pos:
                R_eff = np.sqrt(min(s_pos))
    elif a > 0:
        R_eff = 1.0 / np.sqrt(a)

    return a, b, f, mask, R_eff

# ------------------------------ Per-panel annulus bounds ------------------------------
default_bounds = (45.0, 55.0)  # µm
fit_bounds_by_key = {
    '10': (45.0, 55.0),
    '11': (27.0, 31.0),
    '12': (31.0, 36.0),
    '13': (40.0, 43.0),
    '14': (42.0, 45.0),
    '15': (45.0, 48.0),
    '16': (48.0, 51.0),
}
def bounds_for(key):
    return fit_bounds_by_key.get(key, default_bounds)

# ------------------------------ Plot data (top row) ------------------------------
ne_max = 1.2 * np.max(dicts[0]['ne_r'])

for i, top_key, bot_key, d, t in zip(range(len(dicts)), top_plots, bottom_plots, dicts, times):
    axs0 = ax[top_key]
    if d['xabel'] is not None:
        axs0.plot(d['xabel'], d['ne_r']/1e18, color='tab:orange', lw=1.5)

    axs0.set_xlim(-xlim, xlim)
    axs0.set_ylim(0, 2.4)
    axs0.axhline(0, color=line_color, linewidth=linewidth, linestyle=linestyle)
    for yline in [2, 4]:
        axs0.axhline(yline, color=line_color, linewidth=linewidth, linestyle=linestyle, alpha=0.2)
    for xv in [-100,-50, 0, 50,100]:
        axs0.axvline(xv, color=line_color, linewidth=linewidth, linestyle=linestyle, alpha=0.2)
    axs0.set_title(f'{t:g} ns', fontsize=14)

# ------------------------------ Axes cosmetics (bottom row too) ------------------------------
for key in top_plots[1:-1]:
    ax[key].set_yticks([]); ax[key].set_xticks([])
ax[top_plots[-1]].yaxis.tick_right()
ax[top_plots[-1]].yaxis.set_label_position("right")

for key in bottom_plots[1:-1]:
    ax[key].set_yticks([])
ax[bottom_plots[-1]].yaxis.tick_right()
ax[bottom_plots[-1]].yaxis.set_label_position("right")

for key in bottom_plots:
    ax[key].set_xlim(-xlim, xlim)
    ax[key].set_ylim(-1.0, 1.9)
    for yline in [0.0, -1.0]:
        ax[key].axhline(yline, color=line_color, linewidth=linewidth, linestyle=linestyle, alpha=0.2)
    for xv in [-100,-50, 0, 50,100]:
        ax[key].axvline(xv, color=line_color, linewidth=linewidth, linestyle=linestyle, alpha=0.2)

ax['00'].axhline(Nb_40mbar/1e18, color='red', ls='dashed')
ax['00'].set_ylabel(r'$n_e$ [10$^{18}$ cm$^{-3}$]', color='tab:orange', size=14)
ax['10'].set_ylabel(r'$\Delta n/n_0$', color='black', size=14)
ax['10'].set_xlabel(r'r [$\mu$m]')
ax['12'].set_xlabel(r'r [$\mu$m]')
ax['14'].set_xlabel(r'r [$\mu$m]')
ax['16'].set_xlabel(r'r [$\mu$m]')

ax['00'].text(x=-90, y=1.05*Nb_40mbar/1e18, s='100% ionization', color='red')

# ------------------------------ Fits & patched display (bottom row) ------------------------------
fit_summaries = []
force_pure_quartic = False  # True -> enforce a=0, b>=0 (extra-wide convex core)

for bot_key, d, t in zip(bottom_plots, dicts, times):
    axs1 = ax[bot_key]

    # --- Special case: 0 ns bottom plot -> keep axes but draw nothing ---
    if t == 0.0:
        fit_summaries.append((bot_key, np.nan, np.nan, np.nan))
        continue

    if d['xabel'] is None:
        fit_summaries.append((bot_key, np.nan, np.nan, np.nan))
        continue

    x = np.asarray(d['xabel'])
    y = np.asarray(d['dn0_r'])
    y_sigma = d.get('Delta_dn0_r', None)

    r_in, r_out = bounds_for(bot_key)

    a, b, f, mask_used, R_eff = fit_quartic_convex_from_annulus(
        x, y, r_in, r_out, y_sigma=y_sigma, ridge=0.0, force_pure_quartic=force_pure_quartic
    )
    fit_summaries.append((bot_key, a, b, R_eff))

    # ---------- Patch logic with small overlap (no gap/bridge) ----------
    if len(x) > 1:
        dx_med = np.median(np.diff(np.sort(x)))
        eps = 0.5 * dx_med if np.isfinite(dx_med) and dx_med > 0 else 0.5
    else:
        eps = 0.5

    mask_inside  = (np.abs(x) <= (r_out + eps))   # dotted region (a hair wider)
    # Solid outside measured data
    y_outside = y.copy()
    y_outside[mask_inside] = np.nan
    axs1.plot(x, y_outside, color='black', lw=1.2, ls='-',label='data')

    # Dotted fitted curve inside (replacement, slight overlap)
    f_inside = np.full_like(f, np.nan, dtype=float)
    f_inside[mask_inside] = f[mask_inside]
    axs1.plot(x, f_inside, color='black', lw=1.2, ls='dotted',label='fit')

    # ---------- Vertical dotted seam at x = ±r_out ----------
    # Take y-values from the nearest samples to ±r_out
    def nearest_idx(val):
        return int(np.nanargmin(np.abs(x - val)))

    for x_seam in (r_out, -r_out):
        i_seam = nearest_idx(x_seam)
        # Measured (outside) value and fitted (inside) value at the seam
        y_meas = y[i_seam]
        y_fit  = f[i_seam]
        # Only draw if both are finite
        if np.isfinite(y_meas) and np.isfinite(y_fit):
            y0, y1 = (y_meas, y_fit) if y_meas <= y_fit else (y_fit, y_meas)
            axs1.plot([x_seam, x_seam], [y0, y1], color='black', lw=1.2, ls='dotted')


ax['12'].legend()
ax['01'].text(s='electrons',x=-80,y=2.4*0.4,color='tab:orange',size=14)
ax['11'].text(s='neutrals',x=-80,y=0.8,color='black',size=14)
ax['10'].text(s='40 mbar\n ambient\npressure',x=-80,y=0.24,color='black',size=14)
ax['00'].yaxis.set_label_position("left")
plt.subplots_adjust(wspace=0, hspace=0)
